# 21_3 Descriptive Analysis - Weather

Let us now take a closer look on the characterisitcs of our aggregrated data.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
import hashlib
import holidays
from shapely import Point, wkt
import scipy.stats
import seaborn as sns

## Load and prepare data

In [2]:
data_month_double = '07'

In [3]:
# read final data
data = pd.read_csv("../Data/20_Final_Data.csv")

/tmp/ipykernel_2576417/974816303.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../../../20_Final_Data_NEW_TEST.csv")


In [4]:
tier = pd.read_parquet("../Dat/12_tier_part2.parquet")
next = pd.read_parquet("../Data/15_nextbike_part2.parquet")

In [5]:
tier['trip_duration_seconds'] = tier['trip_duration'].dt.total_seconds()
next['trip_duration_seconds'] = next['trip_duration'].dt.total_seconds()

In [6]:
# transform geometry
data['station_point'] = data['station_point'].apply(wkt.loads)

In [7]:
data['buffer_zone'] = shapely.wkt.loads(data['buffer_zone'])

In [8]:
data = gpd.GeoDataFrame(data, geometry='station_point')

In [9]:
data = data.loc[:, ~data.columns.str.contains('Unnamed')]

In [10]:
#data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'], inplace=True)

In [11]:
data['actual_arrival_time'] = pd.to_datetime(data['actual_arrival_time'])
data['actual_departure_time'] = pd.to_datetime(data['actual_departure_time'])
data['scheduled_arrival_time'] = pd.to_datetime(data['scheduled_arrival_time'])
data['scheduled_departure_time'] = pd.to_datetime(data['scheduled_departure_time'])

In [12]:
data = gpd.GeoDataFrame(data, geometry='buffer_zone')

In [13]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2564861 entries, 0 to 2564860
Data columns (total 56 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   route_id                             int64         
 1   agency_id                            int64         
 2   route_short_name                     object        
 3   route_type                           int64         
 4   route_type_name                      object        
 5   agency_name                          object        
 6   service_id                           int64         
 7   trip_id                              object        
 8   trip_headsign                        object        
 9   direction_id                         int64         
 10  shape_id                             int64         
 11  stop_id                              int64         
 12  actual_arrival_time                  datetime64[ns]
 13  actual_departure_ti

In [14]:
data['date'] = pd.to_datetime(data['date'])

In [15]:
data['arrival_delay'] = pd.to_timedelta(data['arrival_delay'])
data['departure_delay'] = pd.to_timedelta(data['departure_delay'])

In [16]:
data['arrival_delay_float'] = data['arrival_delay'].astype(str)
data['arrival_delay_float'] = data['arrival_delay_float'].str.split(' ', n=2).str[-1]
data['arrival_delay_float'] = data['arrival_delay_float'].astype(str)
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float'])
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float']).dt.total_seconds() / 60

In [17]:
data['departure_delay_float'] = data['departure_delay'].astype(str)
data['departure_delay_float'] = data['departure_delay_float'].str.split(' ', n=2).str[-1]
data['departure_delay_float'] = data['departure_delay_float'].astype(str)
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float'])
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float']).dt.total_seconds() / 60

In [18]:

#NEW FEATURES
data['delay_category'] = data['arrival_delay'].apply(lambda x: 1 if x > pd.Timedelta(0) else 0 if x == pd.Timedelta(0) else -1)
data['cancelled_trip'] = np.where((data['scheduled_arrival_time'].notna() & data['actual_arrival_time'].isna() & data['scheduled_departure_time'].notna() & data['actual_departure_time'].isna()), 1, 0)

## Data Description

### Definition of functions

In [19]:
def calc_stats(data, column_name):

    #generated by Copilot

    rows = len(data.index)

    sum = data[column_name].sum()

#Central Tendency

    # Calculate mean (average)
    mean = data[column_name].mean()
    # Calculate median (middle value)
    median = data[column_name].median()
    # Calculate mode (most frequent value)
    mode = data[column_name].mode()
    #calc maximum
    max = data[column_name].max()
    #calc minimum
    min = data[column_name].min()

    # Calculate range
    data_range = data[column_name].max() - data[column_name].min()
    # Calculate coefficient of variation of range
    cvr = data_range / mean
    # Calculate coefficient of variation of mean
    cvm = mean / mean
    # Calculate coefficient of variation of median
    cvmed = median / mean
    # Calculate coefficient of variation of mode
    cvmode = mode / mean
    # Calculate coefficient of variation of standard deviation
   

# Variability
# Measure of how spread out the values in a data set are
    
    # Calculate standard deviation (average amount of variability in a dataset; tells how far each score lies from the mean; the larger the standard deviation, the more variable the data)
    std_dev = data[column_name].std()
    # Calculate variance (average squared deviation from the mean; square of standard deviation; units of varriance are much larger than those of a typical value in the dataset)
    variance = data[column_name].var()
    # Calculate coefficient of variation
    cv = std_dev / mean # standard deviation divided by the mean; measures the relative variability of a dataset; allows comparison of variability of datasets with different units of measurement
    # Calculate 25th percentile
    percentile_25 = data[column_name].quantile(0.25)
    # Calculate 75th percentile
    percentile_75 = data[column_name].quantile(0.75)
    # Calculate interquartile range
    iqr = percentile_75 - percentile_25 # difference between the 75th and 25th percentiles; measures the spread of the middle 50% of values in a dataset
    # Calculate coefficient of quartile deviation
    qd = iqr / (percentile_75 + percentile_25)
    # Calculate coefficient of variation of coefficient of variation
    cvcv = std_dev / mean
    # Calculate coefficient of variation of coefficient of quartile deviation
    cvqd = iqr / (percentile_75 + percentile_25)
    # Calculate coefficient of variation of range

#Measure of Shape

    # Calculate skewness - measure of how much the probability distribution of a random variable deviates from the normal distribution
    skewness = data[column_name].skew()
    
    # Calculate kurtosis
    kurtosis = data[column_name].kurtosis()

    # Calculate coefficient of skewness
    cs = skewness / std_dev
    # Calculate coefficient of kurtosis
    ck = kurtosis / std_dev
    # Calculate coefficient of variation of skewness
    cvs = skewness / std_dev
    # Calculate coefficient of variation of kurtosis
    cvk = kurtosis / std_dev
    # Calculate coefficient of variation of coefficient of skewness
    cvcs = skewness / std_dev
    # Calculate coefficient of variation of coefficient of kurtosis
    cvck = kurtosis / std_dev
    
    
    
    #return "column: " + str(column_name), "mean: " + str(mean), "median: " + str(median), "mode: " + str(mode), "std_dev: " + str(std_dev), "variance: " + str(variance), "skewness: "+ str(skewness), "kurtosis: " + str(kurtosis), "percentile_25: "+ str(percentile_25), "percentile_75: "+ str(percentile_75), "iqr: " + str(iqr), "range: "+ str(data_range), "cv"+ str(cv), "qd: "+str(qd), "cs: "+ str(cs), "ck: " + str(ck), "csv: "+ str(cvs), "cvk: "+ str(cvk), "cvcs: "+ str(cvcs), "cvck: " +str(cvck), "cvcv: "+ str(cvcv), "cvqd: "+ str (cvqd), "cvr: "+ str(cvr), "cvm: " + str(cvm), "cvmed: " + str(cvmed), "cvmode: "+ str(cvmode)

    return pd.DataFrame({'column': [column_name],
                            'rows': [rows],
                         'sum': [sum],
                         'mean': [mean],
                         'median': [median],
                         'mode': [mode],
                         'max': [max],
                         'min': [min],
                         'std_dev': [std_dev],
                         'variance': [variance],
                         'skewness': [skewness],
                         'kurtosis': [kurtosis],
                         'percentile_25': [percentile_25],
                         'percentile_75': [percentile_75],
                         'iqr': [iqr],
                         'range': [data_range],
                         'cv': [cv],
                         'qd': [qd],
                         'cs': [cs],
                         'ck': [ck],
                         'cvs': [cvs],
                         'cvk': [cvk],
                         'cvcs': [cvcs],
                         'cvck': [cvck],
                         'cvcv': [cvcv],
                         'cvqd': [cvqd],
                         'cvr': [cvr],
                         'cvm': [cvm],
                         'cvmed': [cvmed],
                         'cvmode': [cvmode]}).T

In [20]:
# create a shorter dataframe with most interesting columns - for better overview
def show_short_df(dataframe):
    delay_short = dataframe[['route_id', 'trip_id', 'stop_id', 'stop_name', 'scheduled_arrival_time', 'actual_arrival_time', 'scheduled_departure_time', 'actual_departure_time', 'arrival_delay', 'departure_delay', 'tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'buffer_zone', 'arrival_delay_float', 'departure_delay_float']]
    return delay_short

In [21]:
# transform geometry
point_start = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["start_location"], crs=4326))
point_end = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["end_location"], crs=4326))
# drop geometry columns in wrong format
tier = tier.drop(columns=["start_location", "end_location"])
# replace geometry columns with correct format
tier['start_location'] = point_start
tier['end_location'] = point_end

In [22]:
# get the data from the tier / nextbike dataset
def check_micromobility_datasets(type, data):
    
    if type == 'tier':

        return_data = tier.copy()
        return_data = return_data.iloc[0:0] # empty dataframe

        for item in data['tier_trips_id'].str.split(' ').explode().dropna():   # iterate over all tier ids in the dataframe
            
            id = tier[tier['tier_trips_id'] == int(float(item))]   # get the rows with the specific id
            return_data = pd.concat([return_data, id])  # add the rows to the return dataframe

            return_data.drop_duplicates(inplace=True)
            
    if type == 'next':

        return_data = next.copy()
        return_data = return_data.iloc[0:0] # empty dataframe

        for item in data['nextbike_trips_id'].str.split(' ').explode().dropna():   # iterate over all nextbike ids in the dataframe
            
            id = next[next['nextbike_trips_id'] == int(float(item))]  # get the rows with the specific id
            return_data = pd.concat([return_data, id]) # add the rows to the return dataframe

            return_data.drop_duplicates(inplace=True)

    return return_data

In [23]:
def get_station_to_station(data):
    t = check_micromobility_datasets('tier', data)
    n = check_micromobility_datasets('nextbike', data)

    t = t[t['end_stop_id'].notna() & t['start_stop_id'].notna()]
    n = n[n['end_stop_id'].notna() & n['start_stop_id'].notna()]
    
    return t, n

In [24]:
# map the start and end locations of the trips on a map and connect them with a line

def map_buffer_zones(dataset):
    
    micromobility_data = check_micromobility_datasets('tier', dataset)
    
    # Create a folium map object
    m = folium.Map(location=[50.73743, 7.09821], zoom_start=12)

    # Iterate over each row in the micromobility data
    for index, row in micromobility_data.iterrows():
        # Get the start and end locations
        start_location = row['start_location']
        end_location = row['end_location']
        
        # Get the start and end stop names
        start_stop_name = row['start_stop_name']
        end_stop_name = row['end_stop_name']
        # Add markers for the start and end locations
        folium.Marker(location=[start_location.y, start_location.x], popup=start_stop_name).add_to(m)
        folium.Marker(location=[end_location.y, end_location.x], popup=end_stop_name).add_to(m)
        # Create a line connecting the start and end locations
        folium.PolyLine(locations=[[start_location.y, start_location.x], [end_location.y, end_location.x]], color='blue').add_to(m)
        
    # Display the map
    return m

In [25]:
def get_stats(data, var):
    
    df = pd.DataFrame()

    for i in var:
        df = pd.concat([df, calc_stats(data, i)], axis=1)

    df.columns = df.iloc[0]
    df = df[1:]
    
    return df

## Create Delay, No_Delay, and Nan-Delay Datasets

In [26]:
delay = data[data['departure_delay'] > pd.Timedelta(0)]


In [27]:
#no_delay = data[data['arrival_delay'] == pd.Timedelta(0)]
no_delay = data[
    (data['departure_delay'] == pd.Timedelta(0)) | 
    (data['scheduled_arrival_time'].notna() & 
     data['actual_arrival_time'].isna() & 
     data['scheduled_departure_time'].notna() & 
     data['actual_departure_time'].notna())
]

In [28]:
#nan_delay = data[data['arrival_delay'].isna()]
nan_delay = data[data['scheduled_arrival_time'].notna() & data['actual_arrival_time'].isna() & data['scheduled_departure_time'].notna() & data['actual_departure_time'].isna()]
#nan_delay = data[data['arrival_delay'].isna()]
nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
41,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,255-551-008-2071.2.22:033200-34-157_C0741D60-B...,Bonn Hbf,1,...,0,0,0,0,0,1,0,0,-1,1
42,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
43,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
44,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
45,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564854,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564856,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564857,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564858,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1


### Weather

Also weather seems to have influence on micromobility usage according to literature. The question is, if this can also be observed in combination with PT.

#### Temperature

In [29]:
cold_delay = delay[delay['current_temp'] < 10]
cold_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip


In [30]:
get_stats(cold_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,0,0,0,0,0,0
sum,0,0,0,0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
median,NaN,NaN,NaN,NaN,NaN,NaN
mode,"Series([], Name: tier_trips_count, dtype: int64)","Series([], Name: tier_trips_end_at_station_cou...","Series([], Name: nextbike_trips_count, dtype: ...","Series([], Name: nextbike_trips_end_at_station...","Series([], Name: current_temp, dtype: float64)","Series([], Name: current_precipitation_volume,..."
max,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
std_dev,NaN,NaN,NaN,NaN,NaN,NaN
variance,NaN,NaN,NaN,NaN,NaN,NaN
skewness,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
cold_delay_tier = check_micromobility_datasets('tier', cold_delay)

In [32]:
cold_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
min,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
max,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [33]:
get_stats(cold_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,0,0
sum,0.0,0.0
mean,NaN,NaN
median,NaN,NaN
mode,"Series([], Name: trip_duration_seconds, dtype:...","Series([], Name: trip_distance_kilometers, dty..."
max,NaN,NaN
min,NaN,NaN
std_dev,NaN,NaN
variance,NaN,NaN
skewness,NaN,NaN


In [34]:
# get entries where trips start and end at station
cold_delay_tier_station_station = cold_delay_tier[cold_delay_tier['end_stop_id'].notna() & cold_delay_tier['start_stop_id'].notna()]
cold_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location


In [35]:
cold_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
min,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
max,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [36]:
# get statistics for the tier trips that start and end at a station
get_stats(cold_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,0,0
sum,0.0,0.0
mean,NaN,NaN
median,NaN,NaN
mode,"Series([], Name: trip_duration_seconds, dtype:...","Series([], Name: trip_distance_kilometers, dty..."
max,NaN,NaN
min,NaN,NaN
std_dev,NaN,NaN
variance,NaN,NaN
skewness,NaN,NaN


In [37]:
cold_delay_next = check_micromobility_datasets('next', cold_delay)

In [38]:
cold_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
min,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
max,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [39]:
get_stats(cold_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,0,0
sum,0.0,0.0
mean,NaN,NaN
median,NaN,NaN
mode,"Series([], Name: trip_duration_seconds, dtype:...","Series([], Name: trip_distance_kilometers, dty..."
max,NaN,NaN
min,NaN,NaN
std_dev,NaN,NaN
variance,NaN,NaN
skewness,NaN,NaN


In [40]:
# get entries where trips start and end at station
cold_delay_next_station_station = cold_delay_next[cold_delay_next['end_stop_id'].notna() & cold_delay_next['start_stop_id'].notna()]
cold_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds


In [41]:
cold_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
min,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
max,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [42]:
# get statistics for the tier trips that start and end at a station
get_stats(cold_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,0,0
sum,0.0,0.0
mean,NaN,NaN
median,NaN,NaN
mode,"Series([], Name: trip_duration_seconds, dtype:...","Series([], Name: trip_distance_kilometers, dty..."
max,NaN,NaN
min,NaN,NaN
std_dev,NaN,NaN
variance,NaN,NaN
skewness,NaN,NaN


In [43]:
cold_no_delay = no_delay[no_delay['current_temp'] < 10]
cold_no_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip


In [44]:
get_stats(cold_no_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,0,0,0,0,0,0
sum,0,0,0,0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
median,NaN,NaN,NaN,NaN,NaN,NaN
mode,"Series([], Name: tier_trips_count, dtype: int64)","Series([], Name: tier_trips_end_at_station_cou...","Series([], Name: nextbike_trips_count, dtype: ...","Series([], Name: nextbike_trips_end_at_station...","Series([], Name: current_temp, dtype: float64)","Series([], Name: current_precipitation_volume,..."
max,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
std_dev,NaN,NaN,NaN,NaN,NaN,NaN
variance,NaN,NaN,NaN,NaN,NaN,NaN
skewness,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
cold_nan_delay = nan_delay[nan_delay['current_temp'] < 10]
cold_nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip


In [46]:
get_stats(cold_nan_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,0,0,0,0,0,0
sum,0,0,0,0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
median,NaN,NaN,NaN,NaN,NaN,NaN
mode,"Series([], Name: tier_trips_count, dtype: int64)","Series([], Name: tier_trips_end_at_station_cou...","Series([], Name: nextbike_trips_count, dtype: ...","Series([], Name: nextbike_trips_end_at_station...","Series([], Name: current_temp, dtype: float64)","Series([], Name: current_precipitation_volume,..."
max,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
std_dev,NaN,NaN,NaN,NaN,NaN,NaN
variance,NaN,NaN,NaN,NaN,NaN,NaN
skewness,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
cold_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
cold_no_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
cold_nan_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
warm_delay = delay[(delay['current_temp'] >= 10) & (delay['current_temp'] < 20)]
warm_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
0,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
1,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
2,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
3,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
4,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564839,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,1,0
2564841,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,1,0
2564843,688,6,N8,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6880005-688-006-687.2.32:263500-36-1_5350170A-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,1,0
2564847,688,6,N8,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6880005-688-006-687.2.32:263500-36-1_5350170A-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,0,0


In [51]:
get_stats(warm_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,399470,399470,399470,399470,399470,399470
sum,4003,3279,10275,8761,6910948.24,37757.75
mean,0.010021,0.008208,0.025722,0.021932,17.300293,0.476577
median,0.0,0.0,0.0,0.0,17.73,0.34
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 19.72 Name: current_temp, dtype: float64","0 0.21 Name: current_precipitation_volume, ..."
max,6,6,8,7,19.99,3.39
min,0,0,0,0,11.13,0.11
std_dev,0.107529,0.096838,0.176644,0.161142,1.907178,0.362539
variance,0.011563,0.009378,0.031203,0.025967,3.637329,0.131434
skewness,12.986143,14.225637,8.507717,8.707383,-0.835132,1.757396


In [52]:
warm_delay_tier = check_micromobility_datasets('tier', warm_delay)

In [53]:
warm_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.766000e+03,2766,2766,2766,2766.000000,2766.000000,1953.000000,1533.000000,2766.000000
mean,8.011129e+08,2023-07-16 22:55:41.214750464,2023-07-16 23:09:34.403470848,0 days 00:13:53.188720173,1.716961,1716.961384,1470.249360,1386.523157,833.188720
min,7.822645e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,7.823960e+08,2023-07-05 18:40:00,2023-07-05 18:55:00,0 days 00:10:00,0.752236,752.235869,1104.000000,1102.000000,600.000000
50%,8.086608e+08,2023-07-20 05:25:00,2023-07-20 05:35:00,0 days 00:10:00,1.360880,1360.879906,1145.000000,1140.000000,600.000000
75%,8.087665e+08,2023-07-25 22:05:00,2023-07-25 22:20:00,0 days 00:15:00,2.299294,2299.293678,1302.000000,1293.000000,900.000000
max,8.323115e+08,2023-08-01 01:45:00,2023-08-01 02:05:00,0 days 01:45:00,13.908704,13908.703513,9780.000000,9703.000000,6300.000000
std,1.698108e+07,NaN,NaN,0 days 00:09:26.539342480,1.371607,1371.607264,1466.117365,1447.900505,566.539342


In [54]:
get_stats(warm_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,2766,2766
sum,2304600.0,4749.115188
mean,833.18872,1.716961
median,600.0,1.36088
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.100631 2 0...
max,6300.0,13.908704
min,300.0,0.100174
std_dev,566.539342,1.371607
variance,320966.826578,1.881306
skewness,3.797153,1.795782


In [55]:
# get entries where trips start and end at station
warm_delay_tier_station_station = warm_delay_tier[warm_delay_tier['end_stop_id'].notna() & warm_delay_tier['start_stop_id'].notna()]
warm_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3171653,782356027,2023-06-30 01:45:00,2023-06-30 02:05:00,e-scooter,0 days 00:20:00,3.598056,3598.055590,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1406.0,Bonn Kopenhagener Str.,801621 ...,1200.0,POINT (7.09506 50.73298),POINT (7.07331 50.75712)
3209527,782391260,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.670973,670.973151,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10180 50.73728),POINT (7.09852 50.73217)
3210269,782391943,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.838693,838.692953,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10410 50.73699),POINT (7.09815 50.73232)
3234236,782413798,2023-07-01 06:40:00,2023-07-01 06:45:00,e-scooter,0 days 00:05:00,0.844529,844.529088,1131.0,Bonn Weberstr.,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10205 50.72540),POINT (7.09679 50.73093)
3164060,782350073,2023-07-01 07:10:00,2023-07-01 07:15:00,e-scooter,0 days 00:05:00,0.941159,941.158971,1182.0,Bonn Chlodwigplatz,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,300.0,POINT (7.08933 50.74278),POINT (7.08230 50.73803)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422531,832205256,2023-07-31 23:25:00,2023-08-01 00:00:00,e-scooter,0 days 00:35:00,1.787106,1787.106290,1153.0,Bonn Heerstr.,801621 POLYGON ((7.157885983913106 50.7227...,1190.0,Bonn Zeisigweg,801621 ...,2100.0,POINT (7.09015 50.73736),POINT (7.07489 50.74243)
3485603,832265823,2023-07-31 23:25:00,2023-08-01 00:00:00,e-scooter,0 days 00:35:00,1.774445,1774.445116,1153.0,Bonn Heerstr.,801621 POLYGON ((7.157885983913106 50.7227...,1190.0,Bonn Zeisigweg,801621 ...,2100.0,POINT (7.09011 50.73738),POINT (7.07496 50.74244)
3398184,832181971,2023-08-01 00:10:00,2023-08-01 00:20:00,e-scooter,0 days 00:10:00,0.649892,649.892254,1255.0,Bonn Graf-Stauffenberg-Str.,801621 POLYGON ((7.157885983913106 50.7227...,6994.0,Bonn Heinrich-Lützeler-Str.,801621 ...,600.0,POINT (7.10419 50.71648),POINT (7.09839 50.71719)
3326567,832125594,2023-08-01 01:45:00,2023-08-01 02:05:00,e-scooter,0 days 00:20:00,0.308480,308.479763,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1111.0,Bonn Beethovenhalle Und Swb,801621 ...,1200.0,POINT (7.10401 50.73732),POINT (7.10429 50.74010)


In [56]:
warm_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.277000e+03,1277,1277,1277,1277.000000,1277.000000,1277.000000,1277.000000,1277.000000
mean,8.012866e+08,2023-07-17 00:33:29.553641472,2023-07-17 00:46:49.710258432,0 days 00:13:20.156617071,1.455440,1455.439729,1433.056382,1416.772122,800.156617
min,7.822651e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,7.824001e+08,2023-07-05 19:45:00,2023-07-05 19:50:00,0 days 00:10:00,0.634687,634.687315,1106.000000,1102.000000,600.000000
50%,8.086632e+08,2023-07-19 08:25:00,2023-07-19 08:30:00,0 days 00:10:00,1.141575,1141.575098,1144.000000,1145.000000,600.000000
75%,8.087753e+08,2023-07-26 02:20:00,2023-07-26 02:30:00,0 days 00:15:00,1.989234,1989.233767,1255.000000,1293.000000,900.000000
max,8.323115e+08,2023-08-01 01:45:00,2023-08-01 02:05:00,0 days 01:40:00,6.428679,6428.679229,9703.000000,9703.000000,6000.000000
std,1.717610e+07,NaN,NaN,0 days 00:09:13.246310565,1.119678,1119.678415,1359.529551,1454.880339,553.246311


In [57]:
# get statistics for the tier trips that start and end at a station
get_stats(warm_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,1277,1277
sum,1021800.0,1858.596534
mean,800.156617,1.45544
median,600.0,1.141575
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.100631 2 0.10...
max,6000.0,6.428679
min,300.0,0.100174
std_dev,553.246311,1.119678
variance,306081.480154,1.25368
skewness,3.714119,1.3616


In [58]:
warm_delay_next = check_micromobility_datasets('next', warm_delay)

In [59]:
warm_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,7.121000e+03,7121,7121,7121,7121.000000,7121.000000,5199.000000,4187.000000,7121.000000
mean,3.794049e+07,2023-07-17 12:32:08.265693184,2023-07-17 12:46:34.596264448,0 days 00:14:26.330571548,1.777143,1777.143131,1363.827467,1272.556007,866.330572
min,3.771653e+07,2023-07-01 05:16:00,2023-07-01 05:46:00,0 days 00:05:00,0.100260,100.260461,43.000000,43.000000,300.000000
25%,3.782829e+07,2023-07-06 05:56:00,2023-07-06 06:11:00,0 days 00:10:00,0.794074,794.074378,1103.000000,687.500000,600.000000
50%,3.794222e+07,2023-07-20 10:01:00,2023-07-20 10:16:00,0 days 00:15:00,1.400796,1400.796409,1150.000000,1141.000000,900.000000
75%,3.804928e+07,2023-07-25 23:21:00,2023-07-25 23:36:00,0 days 00:15:00,2.353547,2353.546672,1224.000000,1221.000000,900.000000
max,3.817172e+07,2023-08-01 00:41:00,2023-08-01 00:46:00,0 days 02:00:00,13.172239,13172.239331,9780.000000,9780.000000,7200.000000
std,1.283350e+05,NaN,NaN,0 days 00:08:46.661871815,1.429663,1429.663461,1332.547700,1300.074556,526.661872


In [60]:
get_stats(warm_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,7121,7121
sum,6169140.0,12655.036232
mean,866.330572,1.777143
median,900.0,1.400796
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,7200.0,13.172239
min,300.0,0.10026
std_dev,526.661872,1.429663
variance,277372.727224,2.043938
skewness,3.523064,1.964694


In [61]:
# get entries where trips start and end at station
warm_delay_next_station_station = warm_delay_next[warm_delay_next['end_stop_id'].notna() & warm_delay_next['start_stop_id'].notna()]
warm_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1417189,37961351,2023-07-01 05:41:00,2023-07-01 05:56:00,False,False,bike,0 days 00:15:00,1.671334,1671.334291,"b'\x01\x01\x00\x00\x00\xd9\n\x9a\x96XY\x1c@""\x...",b'\x01\x01\x00\x00\x00\x82\x8d\xeb\xdf\xf5I\x1...,1151.0,Bonn Frankenbad/Kunstverein,215356 ...,43.0,Bonn Propsthof Nord,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1220393,37717932,2023-07-01 05:56:00,2023-07-01 06:11:00,False,False,bike,0 days 00:15:00,1.377444,1377.444442,b'\x01\x01\x00\x00\x00\xcfj\x81=&R\x1c@\x7f\xa...,b'\x01\x01\x00\x00\x00\x9f\xe4\x0e\x9b\xc8L\x1...,8437.0,Bonn An Der Josefshöhe,215356 ...,1190.0,Bonn Zeisigweg,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1235649,37726317,2023-07-01 06:06:00,2023-07-01 06:21:00,False,False,bike,0 days 00:15:00,2.055591,2055.591133,"b""\x01\x01\x00\x00\x00\xf0\xf8\xf6\xaeA_\x1c@\...",b'\x01\x01\x00\x00\x00B\xd1<\x80En\x1c@\x84*5{...,1182.0,Bonn Chlodwigplatz,215356 ...,685.0,Bonn Juridicum,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1267255,37827523,2023-07-01 06:06:00,2023-07-01 06:16:00,False,False,bike,0 days 00:10:00,1.000358,1000.357907,b'\x01\x01\x00\x00\x00\xc0\x06D\x88+g\x1c@\xef...,b'\x01\x01\x00\x00\x00\x8f\x89\x94f\xf3h\x1c@f...,1160.0,Bonn Nordstr.,215356 ...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1638081,37769500,2023-07-01 06:11:00,2023-07-01 06:21:00,False,True,bike,0 days 00:10:00,0.914302,914.302426,b'\x01\x01\x00\x00\x00\xc5V\xd0\xb4\xc4Z\x1c@\...,b'\x01\x01\x00\x00\x00\x06.\x8f5#c\x1c@\x9dJ\x...,1153.0,Bonn Heerstr.,215356 ...,1102.0,Bonn Friedensplatz,215356 POLYGON ((7.120193524138334 50.7395...,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333820,37861619,2023-07-31 22:16:00,2023-07-31 22:26:00,False,False,bike,0 days 00:10:00,0.864220,864.220203,b'\x01\x01\x00\x00\x00h\x06\xf1\x81\x1d_\x1c@\...,b'\x01\x01\x00\x00\x00\x1fh\x05\x86\xac^\x1c@\...,1104.0,Bonn Stadthaus,215356 ...,1175.0,Bonn Lvr-Klinik,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1476595,38040111,2023-07-31 22:26:00,2023-07-31 22:41:00,False,True,bike,0 days 00:15:00,1.658928,1658.927837,b'\x01\x01\x00\x00\x00\xb1\xa3q\xa8\xdfe\x1c@r...,b'\x01\x01\x00\x00\x00\x1c`\xe6;\xf8Y\x1c@.\xe...,1143.0,Bonn Poppelsdorfer Allee,215356 ...,1240.0,Bonn Poppelsdorfer Platz,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1443891,38014658,2023-07-31 22:51:00,2023-07-31 23:06:00,True,False,bike,0 days 00:15:00,2.202721,2202.720792,b'\x01\x01\x00\x00\x00\x06.\x8f5#c\x1c@\x9dJ\x...,b'\x01\x01\x00\x00\x00\x0fc\xd2\xdfKQ\x1c@~\x1...,1102.0,Bonn Friedensplatz,215356 ...,1620.0,Bonn Max-Bruch-Str.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1441769,38012536,2023-07-31 23:21:00,2023-07-31 23:41:00,False,False,bike,0 days 00:20:00,3.058475,3058.475311,b'\x01\x01\x00\x00\x00}\xca1Y\xdc\x7f\x1c@ILP\...,b'\x01\x01\x00\x00\x00\x8b\xc0X\xdf\xc0d\x1c@&...,1502.0,Bonn Beueler Bahnhofsplatz,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,1200.0


In [62]:
warm_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.352000e+03,3352,3352,3352,3352.000000,3352.000000,3352.000000,3352.000000,3352.000000
mean,3.794290e+07,2023-07-17 09:07:52.285202688,2023-07-17 09:21:05.405727744,0 days 00:13:13.120525059,1.464728,1464.727675,1376.714200,1322.736277,793.120525
min,3.771653e+07,2023-07-01 05:41:00,2023-07-01 05:56:00,0 days 00:05:00,0.100260,100.260461,43.000000,43.000000,300.000000
25%,3.783232e+07,2023-07-05 23:32:15,2023-07-05 23:48:30,0 days 00:10:00,0.739146,739.146391,1103.000000,689.000000,600.000000
50%,3.794622e+07,2023-07-20 09:33:30,2023-07-20 09:43:30,0 days 00:10:00,1.206854,1206.853535,1145.000000,1143.000000,600.000000
75%,3.804994e+07,2023-07-25 20:51:00,2023-07-25 21:01:00,0 days 00:15:00,1.905394,1905.394178,1223.000000,1221.000000,900.000000
max,3.817172e+07,2023-07-31 23:31:00,2023-07-31 23:56:00,0 days 01:20:00,9.640671,9640.671235,9780.000000,9780.000000,4800.000000
std,1.276849e+05,NaN,NaN,0 days 00:06:59.193125595,1.077596,1077.596043,1349.357814,1385.248025,419.193126


In [63]:
# get statistics for the tier trips that start and end at a station
get_stats(warm_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,3352,3352
sum,2658540.0,4909.767166
mean,793.120525,1.464728
median,600.0,1.206854
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,4800.0,9.640671
min,300.0,0.10026
std_dev,419.193126,1.077596
variance,175722.876547,1.161213
skewness,2.708901,1.654504


In [64]:
warm_no_delay = no_delay[(no_delay['current_temp'] >= 10) & (no_delay['current_temp'] < 20)]
warm_no_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
0,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
1,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
2,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
3,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
4,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564809,688,6,N8,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6880005-688-006-687.2.32:263500-36-1_5350170A-...,Bonn Hbf,0,...,0,0,1,early morning,8,0,early morning,8,0,0
2564811,687,6,N7,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6870005-687-006-687.2.24:263500-34-1_F0CE2D00-...,Bad Godesberg Bf / Rheinallee,0,...,0,0,1,early morning,8,0,early morning,8,0,0
2564817,684,6,N4,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6840005-684-006-687.2.32:263500-36-1_7F56FE12-...,Am Waldrand,0,...,0,0,1,early morning,8,0,early morning,8,0,0
2564834,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,early morning,8,0,early morning,8,0,0


In [65]:
get_stats(warm_no_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,258974,258974,258974,258974,258974,258974
sum,6130,4978,14067,11929,4473054.49,24459.96
mean,0.02367,0.019222,0.054318,0.046063,17.272215,0.475228
median,0.0,0.0,0.0,0.0,17.72,0.34
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 19.72 Name: current_temp, dtype: float64","0 0.21 Name: current_precipitation_volume, ..."
max,6,6,10,8,19.99,3.39
min,0,0,0,0,11.13,0.11
std_dev,0.173246,0.15363,0.293437,0.263474,1.917817,0.362685
variance,0.030014,0.023602,0.086105,0.069419,3.678022,0.13154
skewness,9.210194,9.901414,8.06291,7.982867,-0.807623,1.776126


In [66]:
warm_no_delay_tier = check_micromobility_datasets('tier', warm_no_delay)

In [67]:
warm_no_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.462000e+03,2462,2462,2462,2462.000000,2462.000000,1927.000000,1258.000000,2462.000000
mean,8.011909e+08,2023-07-17 00:26:57.977254400,2023-07-17 00:40:59.585702656,0 days 00:14:01.608448415,1.793789,1793.789206,1149.546964,1397.444356,841.608448
min,7.822648e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,7.823931e+08,2023-07-05 18:35:00,2023-07-05 18:47:30,0 days 00:10:00,0.768549,768.548967,687.000000,1102.250000,600.000000
50%,8.086627e+08,2023-07-20 22:32:30,2023-07-20 22:42:30,0 days 00:10:00,1.347843,1347.842564,1106.000000,1144.000000,600.000000
75%,8.087660e+08,2023-07-26 06:33:45,2023-07-26 06:47:30,0 days 00:15:00,2.382268,2382.268116,1255.000000,1255.000000,900.000000
max,8.323097e+08,2023-08-01 00:10:00,2023-08-01 00:20:00,0 days 02:00:00,11.376707,11376.707371,9703.000000,9703.000000,7200.000000
std,1.682731e+07,NaN,NaN,0 days 00:09:54.555834046,1.488459,1488.458827,1040.068440,1418.090421,594.555834


In [68]:
get_stats(warm_no_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,2462,2462
sum,2072040.0,4416.309026
mean,841.608448,1.793789
median,600.0,1.347843
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.100712 2 0...
max,7200.0,11.376707
min,300.0,0.100174
std_dev,594.555834,1.488459
variance,353496.639799,2.21551
skewness,4.063277,1.763237


In [69]:
# get entries where trips start and end at station
warm_no_delay_tier_station_station = warm_no_delay_tier[warm_no_delay_tier['end_stop_id'].notna() & warm_no_delay_tier['start_stop_id'].notna()]
warm_no_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3171653,782356027,2023-06-30 01:45:00,2023-06-30 02:05:00,e-scooter,0 days 00:20:00,3.598056,3598.055590,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1406.0,Bonn Kopenhagener Str.,801621 ...,1200.0,POINT (7.09506 50.73298),POINT (7.07331 50.75712)
3209527,782391260,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.670973,670.973151,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10180 50.73728),POINT (7.09852 50.73217)
3210269,782391943,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.838693,838.692953,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10410 50.73699),POINT (7.09815 50.73232)
3254146,782431910,2023-07-01 01:50:00,2023-07-01 02:00:00,e-scooter,0 days 00:10:00,1.779253,1779.253071,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,600.0,POINT (7.09818 50.73206),POINT (7.08279 50.73648)
2973348,782286043,2023-07-01 04:35:00,2023-07-01 04:50:00,e-scooter,0 days 00:15:00,3.658388,3658.388003,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1433.0,Bonn Paulusplatz,801621 ...,900.0,POINT (7.09660 50.73269),POINT (7.06609 50.74510)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485596,832265817,2023-07-31 22:25:00,2023-07-31 22:45:00,e-scooter,0 days 00:20:00,1.171579,1171.578745,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,1200.0,POINT (7.10025 50.73326),POINT (7.09058 50.73749)
3518645,832297525,2023-07-31 22:55:00,2023-07-31 23:00:00,e-scooter,0 days 00:05:00,0.704362,704.361980,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1161.0,Bonn An Der Esche,801621 ...,300.0,POINT (7.10309 50.73744),POINT (7.10055 50.74329)
3318827,832120317,2023-07-31 23:10:00,2023-08-01 00:20:00,e-scooter,0 days 01:10:00,0.117849,117.848655,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,4200.0,POINT (7.10346 50.73716),POINT (7.10245 50.73747)
3421744,832204504,2023-07-31 23:15:00,2023-07-31 23:25:00,e-scooter,0 days 00:10:00,1.002491,1002.491082,1504.0,Bonn Beuel Bf,801621 POLYGON ((7.157885983913106 50.7227...,1126.0,Bonn Gerhardstr.,801621 ...,600.0,POINT (7.12768 50.73829),POINT (7.13061 50.74688)


In [70]:
warm_no_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.114000e+03,1114,1114,1114,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000
mean,8.006238e+08,2023-07-16 17:42:51.059245824,2023-07-16 17:56:09.210053888,0 days 00:13:18.150807899,1.525602,1525.601676,1175.940754,1420.019749,798.150808
min,7.822651e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,7.823851e+08,2023-07-05 17:25:00,2023-07-05 17:50:00,0 days 00:10:00,0.670302,670.302471,687.000000,1106.000000,600.000000
50%,8.086641e+08,2023-07-19 22:30:00,2023-07-19 22:42:30,0 days 00:10:00,1.155788,1155.787883,1115.000000,1145.000000,600.000000
75%,8.087657e+08,2023-07-25 22:55:00,2023-07-25 23:05:00,0 days 00:15:00,2.051486,2051.485903,1184.000000,1268.000000,900.000000
max,8.323097e+08,2023-08-01 00:10:00,2023-08-01 00:20:00,0 days 01:40:00,6.762358,6762.357911,9703.000000,9703.000000,6000.000000
std,1.690863e+07,NaN,NaN,0 days 00:09:36.777140167,1.209878,1209.878412,1012.867618,1422.112771,576.777140


In [71]:
# get statistics for the tier trips that start and end at a station
get_stats(warm_no_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,1114,1114
sum,889140.0,1699.520268
mean,798.150808,1.525602
median,600.0,1.155788
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.101712 2 0.10...
max,6000.0,6.762358
min,300.0,0.100174
std_dev,576.77714,1.209878
variance,332671.86942,1.463806
skewness,4.159499,1.442005


In [72]:
warm_no_delay_next = check_micromobility_datasets('next', warm_no_delay)

In [73]:
warm_no_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,6.197000e+03,6197,6197,6197,6197.000000,6197.000000,5113.000000,3546.000000,6197.000000
mean,3.794012e+07,2023-07-17 05:23:13.148297728,2023-07-17 05:37:32.425367040,0 days 00:14:19.277069549,1.746348,1746.348141,1084.203012,1241.440778,859.277070
min,3.771653e+07,2023-07-01 01:31:00,2023-07-01 01:46:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782828e+07,2023-07-05 22:26:00,2023-07-05 22:36:00,0 days 00:10:00,0.818457,818.457406,687.000000,700.000000,600.000000
50%,3.794218e+07,2023-07-19 22:51:00,2023-07-19 23:01:00,0 days 00:10:00,1.352970,1352.970078,1106.000000,1144.000000,600.000000
75%,3.804843e+07,2023-07-25 21:01:00,2023-07-25 21:16:00,0 days 00:15:00,2.273033,2273.033292,1184.000000,1221.000000,900.000000
max,3.817174e+07,2023-08-01 00:46:00,2023-08-01 01:06:00,0 days 02:00:00,13.172239,13172.239331,9780.000000,9780.000000,7200.000000
std,1.287104e+05,NaN,NaN,0 days 00:08:57.580831712,1.398818,1398.817889,876.735336,1146.302973,537.580832


In [74]:
get_stats(warm_no_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,6197,6197
sum,5324940.0,10822.119432
mean,859.27707,1.746348
median,600.0,1.35297
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.46698 Name: trip_distance_kilometers, d..."
max,7200.0,13.172239
min,300.0,0.100004
std_dev,537.580832,1.398818
variance,288993.150625,1.956691
skewness,3.962666,1.960641


In [75]:
# get entries where trips start and end at station
warm_no_delay_next_station_station = warm_no_delay_next[warm_no_delay_next['end_stop_id'].notna() & warm_no_delay_next['start_stop_id'].notna()]
warm_no_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1292973,37874063,2023-07-01 01:31:00,2023-07-01 01:46:00,False,False,bike,0 days 00:15:00,2.239755,2239.754526,"b""\x01\x01\x00\x00\x00{-\xe8\xbd1d\x1c@R'\xa0\...",b'\x01\x01\x00\x00\x00L\x8ce\xfa%b\x1c@;\xe0\x...,687.0,Bonn Hbf,215356 ...,1173.0,Bonn Pädagogische Fakultät,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1311815,37850623,2023-07-01 01:31:00,2023-07-01 01:46:00,False,False,bike,0 days 00:15:00,1.331223,1331.223283,b'\x01\x01\x00\x00\x00\x81\xcd9x&d\x1c@g\x7f\x...,b'\x01\x01\x00\x00\x00f\xf7\xe4a\xa1f\x1c@}\xc...,687.0,Bonn Hbf,215356 ...,1160.0,Bonn Nordstr.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1513945,38069151,2023-07-01 01:31:00,2023-07-01 01:46:00,False,False,bike,0 days 00:15:00,1.392093,1392.093481,b'\x01\x01\x00\x00\x00V\xb9P\xf9\xd7b\x1c@b\xd...,b'\x01\x01\x00\x00\x00)\x93\x1a\xda\x00\\\x1c@...,687.0,Bonn Hbf,215356 ...,1240.0,Bonn Poppelsdorfer Platz,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1444489,38015256,2023-07-01 05:31:00,2023-07-01 05:36:00,False,True,bike,0 days 00:05:00,0.109052,109.052108,b'\x01\x01\x00\x00\x00\x0c@\xa3t\xe9/\x1c@\x1c...,b'\x01\x01\x00\x00\x00W\xb2c#\x10/\x1c@ap\xcd\...,694.0,Bonn Tannenbusch Mitte,215356 ...,694.0,Bonn Tannenbusch Mitte,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1433891,38006735,2023-07-01 06:21:00,2023-07-01 06:36:00,False,False,bike,0 days 00:15:00,1.550245,1550.245342,b'\x01\x01\x00\x00\x00\xa9lXSYT\x1c@\xb0\xe6\x...,b'\x01\x01\x00\x00\x00\xdapX\x1a\xf8a\x1c@(\x0...,1221.0,Bonn Kaufmannstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539405,38089363,2023-07-31 23:31:00,2023-07-31 23:41:00,False,False,bike,0 days 00:10:00,1.037686,1037.686290,b'\x01\x01\x00\x00\x00\xf6z\xf7\xc7{e\x1c@\xfe...,b'\x01\x01\x00\x00\x00\x12\xdar.\xc5e\x1c@r\xf...,687.0,Bonn Hbf,215356 ...,1133.0,Bonn Luisenstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1466477,38032070,2023-07-31 23:36:00,2023-07-31 23:56:00,False,False,bike,0 days 00:20:00,2.853534,2853.534454,b'\x01\x01\x00\x00\x00\xb2\xb8\xff\xc8th\x1c@\...,b'\x01\x01\x00\x00\x00=Fy\xe6\xe5P\x1c@i5$\xee...,1103.0,Bonn Markt,215356 ...,1231.0,Bonn Sebastianstr.,215356 POLYGON ((7.120193524138334 50.7395...,1200.0
1360121,37933007,2023-07-31 23:51:00,2023-07-31 23:56:00,False,True,bike,0 days 00:05:00,0.126078,126.077777,b'\x01\x01\x00\x00\x00\xa7\xe7\xddXPh\x1c@`\x1...,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,686.0,Bonn Universität/Markt,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1360122,37933008,2023-07-31 23:56:00,2023-08-01 00:01:00,True,False,bike,0 days 00:05:00,0.115054,115.054399,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,b'\x01\x01\x00\x00\x00\xc4AB\x94/h\x1c@D\xbf\x...,1103.0,Bonn Markt,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,300.0


In [76]:
warm_no_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.087000e+03,3087,3087,3087,3087.000000,3087.000000,3087.000000,3087.000000,3087.000000
mean,3.794291e+07,2023-07-16 23:27:45.694849280,2023-07-16 23:41:07.871719936,0 days 00:13:22.176870748,1.467905,1467.904827,1109.795270,1272.039521,802.176871
min,3.771653e+07,2023-07-01 01:31:00,2023-07-01 01:46:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.783568e+07,2023-07-05 18:26:00,2023-07-05 18:36:00,0 days 00:10:00,0.767884,767.884271,687.000000,1102.000000,600.000000
50%,3.794453e+07,2023-07-19 22:16:00,2023-07-19 22:31:00,0 days 00:10:00,1.216908,1216.907610,1115.000000,1150.000000,600.000000
75%,3.804842e+07,2023-07-25 19:26:00,2023-07-25 19:43:30,0 days 00:15:00,1.851771,1851.771246,1182.000000,1221.000000,900.000000
max,3.817163e+07,2023-08-01 00:36:00,2023-08-01 00:41:00,0 days 02:00:00,9.640671,9640.671235,9780.000000,9780.000000,7200.000000
std,1.279293e+05,NaN,NaN,0 days 00:08:21.546465033,1.101745,1101.744983,862.814869,1185.642161,501.546465


In [77]:
# get statistics for the tier trips that start and end at a station
get_stats(warm_no_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,3087,3087
sum,2476320.0,4531.422202
mean,802.176871,1.467905
median,600.0,1.216908
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.222838 Name: trip_distance_kilometers, ..."
max,7200.0,9.640671
min,300.0,0.100004
std_dev,501.546465,1.101745
variance,251548.856587,1.213842
skewness,4.786835,2.011118


In [78]:
warm_nan_delay = nan_delay[(nan_delay['current_temp'] >= 10) & (nan_delay['current_temp'] < 20)]
warm_nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
41,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,255-551-008-2071.2.22:033200-34-157_C0741D60-B...,Bonn Hbf,1,...,0,0,0,0,0,1,0,0,-1,1
42,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
43,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
44,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
45,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564854,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564856,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564857,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564858,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1


In [79]:
get_stats(warm_nan_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,431069,431069,431069,431069,431069,431069
sum,6535,5332,16800,14161,7483016.4,42192.13
mean,0.01516,0.012369,0.038973,0.032851,17.359208,0.476107
median,0.0,0.0,0.0,0.0,17.78,0.32
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 19.72 Name: current_temp, dtype: float64","0 0.21 Name: current_precipitation_volume, ..."
max,5,5,11,9,19.99,1.77
min,0,0,0,0,11.13,0.11
std_dev,0.134963,0.120354,0.240886,0.215569,1.909571,0.388452
variance,0.018215,0.014485,0.058026,0.04647,3.646462,0.150895
skewness,11.071256,11.783867,9.440482,9.16431,-0.845778,1.720959


In [80]:
warm_nan_delay_tier = check_micromobility_datasets('tier', warm_nan_delay)

In [81]:
warm_nan_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.339000e+03,3339,3339,3339,3339.000000,3339.000000,2500.000000,1761.000000,3339.000000
mean,8.009782e+08,2023-07-16 22:16:30.566037504,2023-07-16 22:30:24.258760448,0 days 00:13:53.692722371,1.753809,1753.808725,1319.966400,1418.277115,833.692722
min,7.822645e+08,2023-07-01 01:25:00,2023-07-01 01:40:00,0 days 00:05:00,0.100042,100.042262,43.000000,43.000000,300.000000
25%,7.823883e+08,2023-07-05 15:35:00,2023-07-05 16:07:30,0 days 00:10:00,0.736866,736.866000,687.000000,1102.000000,600.000000
50%,8.086604e+08,2023-07-19 22:40:00,2023-07-19 22:50:00,0 days 00:10:00,1.325877,1325.876813,1115.000000,1143.000000,600.000000
75%,8.087659e+08,2023-07-26 10:15:00,2023-07-26 10:27:30,0 days 00:15:00,2.287911,2287.910750,1259.000000,1255.000000,900.000000
max,8.323115e+08,2023-07-31 23:15:00,2023-07-31 23:30:00,0 days 01:45:00,13.908704,13908.703513,9703.000000,9780.000000,6300.000000
std,1.726712e+07,NaN,NaN,0 days 00:09:36.226534267,1.493832,1493.832122,1358.917219,1516.742949,576.226534


In [82]:
get_stats(warm_nan_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,3339,3339
sum,2783700.0,5855.967333
mean,833.692722,1.753809
median,600.0,1.325877
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100042 1 0.100174 2 0...
max,6300.0,13.908704
min,300.0,0.100042
std_dev,576.226534,1.493832
variance,332037.018794,2.231534
skewness,3.735313,1.970441


In [83]:
# get entries where trips start and end at station
warm_nan_delay_tier_station_station = warm_nan_delay_tier[warm_nan_delay_tier['end_stop_id'].notna() & warm_nan_delay_tier['start_stop_id'].notna()]
warm_nan_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3190399,782373031,2023-07-01 01:25:00,2023-07-01 01:40:00,e-scooter,0 days 00:15:00,1.055901,1055.901028,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1110.0,Bonn Wilhelmsplatz,801621 ...,900.0,POINT (7.10146 50.73271),POINT (7.09772 50.74150)
3209527,782391260,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.670973,670.973151,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10180 50.73728),POINT (7.09852 50.73217)
3210269,782391943,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.838693,838.692953,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10410 50.73699),POINT (7.09815 50.73232)
3201996,782383745,2023-07-01 02:20:00,2023-07-01 02:25:00,e-scooter,0 days 00:05:00,0.567479,567.479089,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1103.0,Bonn Markt,801621 ...,300.0,POINT (7.09831 50.73236),POINT (7.10279 50.73481)
3276897,782452649,2023-07-01 02:20:00,2023-07-01 02:25:00,e-scooter,0 days 00:05:00,0.589011,589.011497,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1103.0,Bonn Markt,801621 ...,300.0,POINT (7.09832 50.73240),POINT (7.10306 50.73479)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422524,832205249,2023-07-31 22:25:00,2023-07-31 22:45:00,e-scooter,0 days 00:20:00,1.197806,1197.806121,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,1200.0,POINT (7.10022 50.73309),POINT (7.09036 50.73745)
3485596,832265817,2023-07-31 22:25:00,2023-07-31 22:45:00,e-scooter,0 days 00:20:00,1.171579,1171.578745,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,1200.0,POINT (7.10025 50.73326),POINT (7.09058 50.73749)
3518645,832297525,2023-07-31 22:55:00,2023-07-31 23:00:00,e-scooter,0 days 00:05:00,0.704362,704.361980,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1161.0,Bonn An Der Esche,801621 ...,300.0,POINT (7.10309 50.73744),POINT (7.10055 50.74329)
3420883,832203680,2023-07-31 23:00:00,2023-07-31 23:05:00,e-scooter,0 days 00:05:00,0.369227,369.226899,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,300.0,POINT (7.10130 50.73766),POINT (7.10442 50.73650)


In [84]:
warm_nan_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.523000e+03,1523,1523,1523,1523.000000,1523.000000,1523.000000,1523.000000,1523.000000
mean,8.005461e+08,2023-07-16 17:09:21.588968960,2023-07-16 17:22:32.068286464,0 days 00:13:10.479317137,1.489768,1489.767807,1337.796454,1420.495076,790.479317
min,7.822645e+08,2023-07-01 01:25:00,2023-07-01 01:40:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,7.823869e+08,2023-07-05 14:57:30,2023-07-05 15:07:30,0 days 00:10:00,0.665461,665.460646,691.500000,1103.000000,600.000000
50%,8.086606e+08,2023-07-19 07:45:00,2023-07-19 08:00:00,0 days 00:10:00,1.153521,1153.521300,1115.000000,1144.000000,600.000000
75%,8.087661e+08,2023-07-26 06:27:30,2023-07-26 06:35:00,0 days 00:15:00,1.965522,1965.521621,1232.500000,1245.000000,900.000000
max,8.323115e+08,2023-07-31 23:15:00,2023-07-31 23:25:00,0 days 01:40:00,8.743102,8743.102380,9703.000000,9780.000000,6000.000000
std,1.713102e+07,NaN,NaN,0 days 00:08:32.915931728,1.208192,1208.192096,1343.320341,1461.833095,512.915932


In [85]:
# get statistics for the tier trips that start and end at a station
get_stats(warm_nan_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,1523,1523
sum,1203900.0,2268.916371
mean,790.479317,1.489768
median,600.0,1.153521
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.100973 2 0.10...
max,6000.0,8.743102
min,300.0,0.100174
std_dev,512.915932,1.208192
variance,263082.753021,1.459728
skewness,3.332903,1.736488


In [86]:
warm_nan_delay_next = check_micromobility_datasets('next', warm_nan_delay)

In [87]:
warm_nan_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,8.968000e+03,8968,8968,8968,8968.000000,8968.000000,7138.000000,5230.000000,8968.000000
mean,3.793928e+07,2023-07-17 02:21:32.294825984,2023-07-17 02:35:49.843889664,0 days 00:14:17.549063336,1.727845,1727.844642,1201.776408,1217.339388,857.549063
min,3.771653e+07,2023-07-01 01:56:00,2023-07-01 02:06:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782894e+07,2023-07-05 19:06:00,2023-07-05 19:24:45,0 days 00:10:00,0.785771,785.771230,687.000000,688.000000,600.000000
50%,3.794153e+07,2023-07-19 21:21:00,2023-07-19 21:41:00,0 days 00:10:00,1.345363,1345.363351,1115.000000,1143.000000,600.000000
75%,3.804850e+07,2023-07-25 20:31:00,2023-07-25 20:48:30,0 days 00:15:00,2.252799,2252.799363,1221.000000,1221.000000,900.000000
max,3.817163e+07,2023-08-01 03:21:00,2023-08-01 03:31:00,0 days 02:00:00,14.104859,14104.859070,9780.000000,9780.000000,7200.000000
std,1.282833e+05,NaN,NaN,0 days 00:08:46.665141565,1.402899,1402.898609,1137.935280,1108.378305,526.665142


In [88]:
get_stats(warm_nan_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,8968,8968
sum,7690500.0,15495.310752
mean,857.549063,1.727845
median,600.0,1.345363
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,7200.0,14.104859
min,300.0,0.100004
std_dev,526.665142,1.402899
variance,277376.171341,1.968125
skewness,3.71172,2.038059


In [89]:
# get entries where trips start and end at station
warm_nan_delay_next_station_station = warm_nan_delay_next[warm_nan_delay_next['end_stop_id'].notna() & warm_nan_delay_next['start_stop_id'].notna()]
warm_nan_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1635116,37766535,2023-07-01 05:16:00,2023-07-01 05:26:00,False,True,bike,0 days 00:10:00,1.003667,1003.666723,"b""\x01\x01\x00\x00\x00Z\xb7A\xed\xb7\x96\x1c@L...",b'\x01\x01\x00\x00\x00t\xef\xe1\x92\xe3\x9e\x1...,8950.0,Bonn Schießbergweg,215356 ...,1584.0,Bonn Ramersdorf,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1444489,38015256,2023-07-01 05:31:00,2023-07-01 05:36:00,False,True,bike,0 days 00:05:00,0.109052,109.052108,b'\x01\x01\x00\x00\x00\x0c@\xa3t\xe9/\x1c@\x1c...,b'\x01\x01\x00\x00\x00W\xb2c#\x10/\x1c@ap\xcd\...,694.0,Bonn Tannenbusch Mitte,215356 ...,694.0,Bonn Tannenbusch Mitte,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1311817,37850625,2023-07-01 06:21:00,2023-07-01 06:36:00,False,False,bike,0 days 00:15:00,1.284558,1284.557726,b'\x01\x01\x00\x00\x00}\xeb\xc3z\xa3f\x1c@}\xc...,b'\x01\x01\x00\x00\x001\x08\xac\x1cZd\x1c@C\x9...,1160.0,Bonn Nordstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1433891,38006735,2023-07-01 06:21:00,2023-07-01 06:36:00,False,False,bike,0 days 00:15:00,1.550245,1550.245342,b'\x01\x01\x00\x00\x00\xa9lXSYT\x1c@\xb0\xe6\x...,b'\x01\x01\x00\x00\x00\xdapX\x1a\xf8a\x1c@(\x0...,1221.0,Bonn Kaufmannstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1471081,38035639,2023-07-01 06:26:00,2023-07-01 06:41:00,False,False,bike,0 days 00:15:00,1.898569,1898.568609,b'\x01\x01\x00\x00\x00\xbb\x99\xd1\x8f\x86c\x1...,b'\x01\x01\x00\x00\x00#.\x00\x8d\xd2e\x1c@\xd5...,687.0,Bonn Hbf,215356 ...,1174.0,Bonn Augustinum,215356 POLYGON ((7.120193524138334 50.7395...,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539405,38089363,2023-07-31 23:31:00,2023-07-31 23:41:00,False,False,bike,0 days 00:10:00,1.037686,1037.686290,b'\x01\x01\x00\x00\x00\xf6z\xf7\xc7{e\x1c@\xfe...,b'\x01\x01\x00\x00\x00\x12\xdar.\xc5e\x1c@r\xf...,687.0,Bonn Hbf,215356 ...,1133.0,Bonn Luisenstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1592910,37753445,2023-07-31 23:31:00,2023-07-31 23:56:00,False,False,bike,0 days 00:25:00,4.197185,4197.185304,b'\x01\x01\x00\x00\x00\xa5\xa2\xb1\xf6wv\x1c@\...,b'\x01\x01\x00\x00\x00\xbb%9`WS\x1c@#2\xac\xe2...,1500.0,Bonn Konrad-Adenauer-Platz,215356 ...,1231.0,Bonn Sebastianstr.,215356 POLYGON ((7.120193524138334 50.7395...,1500.0
1250204,37733796,2023-07-31 23:56:00,2023-08-01 00:11:00,False,False,bike,0 days 00:15:00,2.822479,2822.478797,b'\x01\x01\x00\x00\x00\x8a \xce\xc3\t\\\x1c@Id...,b'\x01\x01\x00\x00\x00\x12/O\xe7\x8aB\x1c@\x02...,1153.0,Bonn Heerstr.,215356 ...,9702.0,Bonn Im Tannenbusch,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1360122,37933008,2023-07-31 23:56:00,2023-08-01 00:01:00,True,False,bike,0 days 00:05:00,0.115054,115.054399,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,b'\x01\x01\x00\x00\x00\xc4AB\x94/h\x1c@D\xbf\x...,1103.0,Bonn Markt,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,300.0


In [90]:
warm_nan_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,4.420000e+03,4420,4420,4420,4420.000000,4420.000000,4420.000000,4420.000000,4420.000000
mean,3.793927e+07,2023-07-17 01:03:32.864253440,2023-07-17 01:16:58.601809920,0 days 00:13:25.737556561,1.446099,1446.099150,1239.268552,1248.039819,805.737557
min,3.771653e+07,2023-07-01 05:16:00,2023-07-01 05:26:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782897e+07,2023-07-05 18:36:00,2023-07-05 18:46:00,0 days 00:10:00,0.731814,731.814020,687.000000,694.000000,600.000000
50%,3.794079e+07,2023-07-19 21:36:00,2023-07-19 21:53:30,0 days 00:10:00,1.192735,1192.734925,1115.000000,1145.000000,600.000000
75%,3.804744e+07,2023-07-25 19:47:15,2023-07-25 20:02:15,0 days 00:15:00,1.831861,1831.860589,1221.000000,1221.000000,900.000000
max,3.817163e+07,2023-08-01 03:21:00,2023-08-01 03:31:00,0 days 02:00:00,9.640671,9640.671235,9780.000000,9780.000000,7200.000000
std,1.274229e+05,NaN,NaN,0 days 00:08:22.761978977,1.096765,1096.764936,1203.425667,1144.499207,502.761979


In [91]:
# get statistics for the tier trips that start and end at a station
get_stats(warm_nan_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,4420,4420
sum,3561360.0,6391.758242
mean,805.737557,1.446099
median,600.0,1.192735
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,7200.0,9.640671
min,300.0,0.100004
std_dev,502.761979,1.096765
variance,252769.607505,1.202893
skewness,4.505201,1.960686


In [92]:
warm_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,399470.000000,399470.000000,399470.000000,399470.000000,399470.000000,399470.000000,399470.000000,386296,399470,399470.000000,...,399470.000000,399470.000000,399470.000000,399470.0,399470.000000,399470.000000,399470.000000,399470.000000,399470.000000,399470.0
mean,563.017488,6.006351,2.733660,2973.282564,0.502018,3640.027096,2328.097872,2023-07-18 14:22:54.774594816,2023-07-18 14:46:44.893696768,120940.458618,...,0.833959,118882.985573,122663.505657,0.0,0.253391,2.366678,0.619493,2.446599,0.631304,0.0
min,18.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,2023-07-01 01:24:50,2023-07-01 01:25:10,804.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0
25%,602.000000,6.000000,3.000000,64.000000,0.000000,1918.000000,1212.000000,2023-07-07 03:11:52.500000,2023-07-07 05:15:30,62306.000000,...,0.250000,70215.000000,71645.000000,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.0
50%,607.000000,6.000000,3.000000,64.000000,1.000000,2734.000000,1441.000000,2023-07-21 17:00:00,2023-07-21 17:08:45,100804.000000,...,0.500000,95830.000000,101530.000000,0.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.0
75%,611.000000,6.000000,3.000000,130.000000,1.000000,5633.000000,1683.000000,2023-07-26 20:31:30,2023-07-26 20:34:30,190305.000000,...,0.750000,185730.000000,191015.000000,0.0,0.000000,4.000000,1.000000,4.000000,1.000000,0.0
max,843.000000,12.000000,3.000000,27690.000000,1.000000,10396.000000,9780.000000,2023-08-01 03:19:00,2023-08-01 03:19:15,235807.000000,...,510.750000,235945.000000,235950.000000,0.0,2.000000,8.000000,3.000000,8.000000,1.000000,0.0
std,159.205842,0.156780,0.853279,8330.065331,0.499997,2796.932699,2427.661880,NaN,NaN,65473.209092,...,5.344589,68504.269899,65831.711419,0.0,0.510886,2.405343,0.993824,2.401140,0.546550,0.0


In [93]:
warm_no_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,258974.000000,258974.000000,258974.000000,258974.000000,258974.000000,258974.000000,258974.000000,185921,258974,258974.000000,...,258974.000000,258974.000000,258974.000000,258974.0,258974.000000,258974.000000,258974.000000,258974.000000,258974.000000,258974.0
mean,550.662584,6.919351,2.563242,5660.785075,0.481678,4215.004027,2141.166465,2023-07-18 08:12:34.097138432,2023-07-18 09:43:05.854579968,119355.813483,...,0.047437,88540.897287,122692.516098,0.0,0.253879,1.798779,0.627565,2.466298,-0.277209,0.0
min,16.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,2023-07-01 00:33:00,2023-07-01 00:32:00,308.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0
25%,551.000000,6.000000,3.000000,64.000000,0.000000,2014.000000,1175.000000,2023-07-06 21:35:45,2023-07-06 22:52:00,61805.000000,...,0.000000,0.000000,71400.000000,0.0,0.000000,0.000000,0.000000,1.000000,-1.000000,0.0
50%,607.000000,6.000000,3.000000,64.000000,0.000000,3858.000000,1406.000000,2023-07-21 09:23:45,2023-07-21 09:45:00,94804.000000,...,0.000000,74245.000000,101200.000000,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.0
75%,614.000000,6.000000,3.000000,151.000000,1.000000,6377.000000,1702.000000,2023-07-26 15:12:45,2023-07-26 15:01:00,190305.000000,...,0.000000,150000.000000,191900.000000,0.0,0.000000,3.000000,1.000000,4.000000,0.000000,0.0
max,884.000000,12.000000,3.000000,29145.000000,1.000000,10396.000000,9780.000000,2023-08-01 03:15:45,2023-08-01 03:16:00,235807.000000,...,510.000000,235945.000000,235950.000000,0.0,2.000000,8.000000,3.000000,8.000000,1.000000,0.0
std,224.725261,2.111964,1.058074,10948.499223,0.499665,2867.025969,2271.457568,NaN,NaN,65432.414743,...,1.148310,79165.094668,65902.325135,0.0,0.511735,2.376443,0.996814,2.423253,0.458387,0.0


In [94]:
warm_nan_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,431069.000000,431069.000000,431069.000000,431069.000000,431069.000000,431069.000000,431069.000000,0,0,431069.000000,...,0.0,431069.0,431069.0,431069.0,431069.000000,431069.0,431069.000000,431069.0,431069.0,431069.0
mean,552.772802,6.279904,2.661590,4137.276566,0.489372,4052.350169,2247.414992,NaT,NaT,111730.828322,...,NaN,0.0,0.0,0.0,0.265897,0.0,0.500435,0.0,-1.0,1.0
min,16.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,NaT,NaT,1305.000000,...,NaN,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,-1.0,1.0
25%,601.000000,6.000000,3.000000,64.000000,0.000000,1970.000000,1213.000000,NaT,NaT,61804.000000,...,NaN,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,-1.0,1.0
50%,607.000000,6.000000,3.000000,64.000000,0.000000,2952.000000,1455.000000,NaT,NaT,91304.000000,...,NaN,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,-1.0,1.0
75%,613.000000,6.000000,3.000000,130.000000,1.000000,6327.000000,1682.000000,NaT,NaT,164804.000000,...,NaN,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,-1.0,1.0
max,884.000000,12.000000,3.000000,29145.000000,1.000000,10396.000000,9780.000000,NaT,NaT,235807.000000,...,NaN,0.0,0.0,0.0,2.000000,0.0,3.000000,0.0,-1.0,1.0
std,185.110249,1.192090,0.949058,9626.064674,0.499888,2906.886362,2336.302005,NaN,NaN,60061.111221,...,NaN,0.0,0.0,0.0,0.523995,0.0,0.888273,0.0,0.0,0.0


In [95]:
hot_delay = delay[delay['current_temp'] >= 20]
hot_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
54948,601,6,601,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6010212-601-006-9478.2.22:160900-44-1_0FB6A3B3...,Uniklinikum Süd,1,...,0-5,0,0,evening rush,5,2,evening rush,5,1,0
54951,609,6,609,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6090196-609-006-1574.2.24:170000-45-1_9B34082F...,Hardthöhe/Südwache,1,...,0-5,0,0,evening rush,5,2,evening rush,5,1,0
54958,610,6,610,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6100186-610-006-1341.2.24:165300-53-1_19B38892...,Giselherstr.,1,...,0-5,0,0,evening rush,5,2,evening rush,5,0,0
54967,630,6,630,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6300107-630-006-1293.2.21:163200-52-1_4852C788...,Agnetendorfer Str.,0,...,0-5,0,0,evening rush,5,2,evening rush,5,0,0
54974,609,6,609,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6090190-609-006-1574.2.24:160000-45-1_4B78162D...,Hardthöhe/Südwache,1,...,0-5,0,0,evening rush,5,2,evening rush,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2533797,601,6,601,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6010048-601-006-9478.2.22:160600-44-1_3D4C1FF4...,Uniklinikum Süd,1,...,0-5,0,1,evening rush,5,0,evening rush,5,1,0
2533801,614,6,614,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,258,6140248-614-006-1607.2.22:165400-19-1_260E1A81...,Im Rosenberg,1,...,0-5,0,1,evening rush,5,0,evening rush,5,0,0
2533803,605,6,605,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6050105-605-006-1341.2.24:164600-40-1_07E67217...,Mondorfer Fähre,0,...,0-5,0,1,evening rush,5,0,evening rush,5,1,0
2533804,602,6,602,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6020095-602-006-1307.2.22:164100-37-1_64412A1E...,Agnetendorfer Str.,0,...,0-5,0,1,evening rush,5,0,evening rush,5,1,0


In [96]:
get_stats(hot_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,550796,550796,550796,550796,550796,550796
sum,8469,6908,23209,19396,13299074.62,39623.94
mean,0.015376,0.012542,0.042137,0.035214,24.145191,0.823355
median,0.0,0.0,0.0,0.0,23.09,0.39
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 21.48 Name: current_temp, dtype: float64","0 0.33 Name: current_precipitation_volume, ..."
max,6,4,8,8,35.37,3.89
min,0,0,0,0,20.05,0.12
std_dev,0.135192,0.121189,0.234464,0.211288,3.586258,0.990738
variance,0.018277,0.014687,0.054973,0.044643,12.861244,0.981561
skewness,10.474103,11.251808,7.085213,7.470592,1.276714,2.020511


In [97]:
hot_delay_tier = check_micromobility_datasets('tier', hot_delay)

In [98]:
hot_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,5.852000e+03,5852,5852,5852,5852.000000,5852.000000,4100.000000,3191.000000,5852.000000
mean,7.970040e+08,2023-07-14 16:55:37.320574464,2023-07-14 17:09:50.792891392,0 days 00:14:13.472317156,1.741757,1741.756718,1429.522439,1377.078345,853.472317
min,7.822646e+08,2023-07-01 17:05:00,2023-07-01 17:15:00,0 days 00:05:00,0.100290,100.289988,43.000000,43.000000,300.000000
25%,7.823623e+08,2023-07-08 14:55:00,2023-07-08 15:10:00,0 days 00:10:00,0.746935,746.934991,1102.000000,1102.000000,600.000000
50%,7.824573e+08,2023-07-12 22:20:00,2023-07-12 22:30:00,0 days 00:10:00,1.324163,1324.162734,1131.000000,1145.000000,600.000000
75%,8.087303e+08,2023-07-20 12:11:15,2023-07-20 12:31:15,0 days 00:15:00,2.315674,2315.673717,1240.000000,1240.000000,900.000000
max,8.323141e+08,2023-07-31 16:45:00,2023-07-31 17:00:00,0 days 01:55:00,11.853086,11853.086389,9780.000000,9780.000000,6900.000000
std,1.646028e+07,NaN,NaN,0 days 00:09:40.969586984,1.457222,1457.221807,1458.000195,1376.383390,580.969587


In [99]:
get_stats(hot_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,5852,5852
sum,4994520.0,10192.760315
mean,853.472317,1.741757
median,600.0,1.324163
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100290 1 0.101082 2 0...
max,6900.0,11.853086
min,300.0,0.10029
std_dev,580.969587,1.457222
variance,337525.661001,2.123495
skewness,3.875676,1.903369


In [100]:
# get entries where trips start and end at station
hot_delay_tier_station_station = hot_delay_tier[hot_delay_tier['end_stop_id'].notna() & hot_delay_tier['start_stop_id'].notna()]
hot_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3035496,782304577,2023-07-01 17:10:00,2023-07-01 17:20:00,e-scooter,0 days 00:10:00,1.576509,1576.508878,1102.0,Bonn Friedensplatz,801621 POLYGON ((7.157885983913106 50.7227...,1192.0,Bonn Ellerstr.,801621 ...,600.0,POINT (7.09504 50.73646),POINT (7.08100 50.73846)
2954000,782279912,2023-07-01 17:15:00,2023-07-01 17:25:00,e-scooter,0 days 00:10:00,1.327481,1327.480919,1104.0,Bonn Stadthaus,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,600.0,POINT (7.09416 50.73632),POINT (7.08233 50.73793)
3160534,782347810,2023-07-01 17:30:00,2023-07-01 17:35:00,e-scooter,0 days 00:05:00,0.409722,409.722289,1405.0,Bonn Nordfriedhof,801621 POLYGON ((7.157885983913106 50.7227...,2738.0,Bonn Hedwigschule,801621 ...,300.0,POINT (7.07156 50.75246),POINT (7.07509 50.75139)
3165488,782351010,2023-07-01 17:30:00,2023-07-01 17:40:00,e-scooter,0 days 00:10:00,0.761879,761.879472,1106.0,Bonn Thomas-Mann-Str.,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,600.0,POINT (7.09514 50.73360),POINT (7.10089 50.73736)
3235072,782414557,2023-07-01 17:30:00,2023-07-01 17:40:00,e-scooter,0 days 00:10:00,0.746957,746.957378,1106.0,Bonn Thomas-Mann-Str.,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,600.0,POINT (7.09515 50.73386),POINT (7.10085 50.73743)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370550,832155781,2023-07-31 16:10:00,2023-07-31 16:20:00,e-scooter,0 days 00:10:00,0.344273,344.272909,1111.0,Bonn Beethovenhalle Und Swb,801621 POLYGON ((7.157885983913106 50.7227...,1161.0,Bonn An Der Esche,801621 ...,600.0,POINT (7.10198 50.74017),POINT (7.10072 50.74302)
3444519,832226390,2023-07-31 16:10:00,2023-07-31 16:25:00,e-scooter,0 days 00:15:00,2.137650,2137.649578,1146.0,Bonn Bachstr.,801621 POLYGON ((7.157885983913106 50.7227...,1255.0,Bonn Graf-Stauffenberg-Str.,801621 ...,900.0,POINT (7.09273 50.73087),POINT (7.10524 50.71616)
3520750,832299536,2023-07-31 16:20:00,2023-07-31 16:35:00,e-scooter,0 days 00:15:00,1.526621,1526.621152,1223.0,Bonn Beringstr.,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,900.0,POINT (7.09417 50.72605),POINT (7.10262 50.73695)
3380897,832165359,2023-07-31 16:30:00,2023-07-31 16:35:00,e-scooter,0 days 00:05:00,0.265767,265.766999,1144.0,Bonn Beethovenstr.,801621 POLYGON ((7.157885983913106 50.7227...,1144.0,Bonn Beethovenstr.,801621 ...,300.0,POINT (7.09561 50.72809),POINT (7.09758 50.72944)


In [101]:
hot_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.623000e+03,2623,2623,2623,2623.000000,2623.000000,2623.000000,2623.000000,2623.000000
mean,7.961386e+08,2023-07-14 10:11:00.274494720,2023-07-14 10:23:41.448722944,0 days 00:12:41.174227983,1.407458,1407.457688,1452.953107,1415.776210,761.174228
min,7.822648e+08,2023-07-01 17:10:00,2023-07-01 17:20:00,0 days 00:05:00,0.100290,100.289988,43.000000,43.000000,300.000000
25%,7.823559e+08,2023-07-08 14:47:30,2023-07-08 14:57:30,0 days 00:10:00,0.620319,620.318848,1103.000000,1104.000000,600.000000
50%,7.824461e+08,2023-07-12 16:45:00,2023-07-12 17:00:00,0 days 00:10:00,1.081036,1081.035705,1133.000000,1151.000000,600.000000
75%,8.087204e+08,2023-07-19 13:52:30,2023-07-19 14:05:00,0 days 00:15:00,1.911442,1911.442170,1231.000000,1240.000000,900.000000
max,8.323115e+08,2023-07-31 16:45:00,2023-07-31 17:00:00,0 days 01:45:00,9.093761,9093.761020,9780.000000,9780.000000,6300.000000
std,1.635612e+07,NaN,NaN,0 days 00:07:59.729290311,1.129293,1129.292632,1499.761410,1416.715781,479.729290


In [102]:
# get statistics for the tier trips that start and end at a station
get_stats(hot_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,2623,2623
sum,1996560.0,3691.761516
mean,761.174228,1.407458
median,600.0,1.081036
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100290 1 0.101082 2 0.10...
max,6300.0,9.093761
min,300.0,0.10029
std_dev,479.72929,1.129293
variance,230140.191982,1.275302
skewness,4.183235,1.743969


In [103]:
hot_delay_next = check_micromobility_datasets('next', hot_delay)

In [104]:
hot_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.510800e+04,15108,15108,15108,15108.000000,15108.000000,11054.000000,8697.000000,15108.000000
mean,3.793930e+07,2023-07-14 20:07:52.239873024,2023-07-14 20:22:54.610802176,0 days 00:15:02.370929308,1.711512,1711.512271,1280.318165,1302.805105,902.370929
min,3.771656e+07,2023-07-01 17:06:00,2023-07-01 17:16:00,0 days 00:05:00,0.100057,100.057033,43.000000,43.000000,300.000000
25%,3.782821e+07,2023-07-08 17:26:00,2023-07-08 17:44:45,0 days 00:10:00,0.758444,758.444003,1102.000000,1102.000000,600.000000
50%,3.794130e+07,2023-07-13 16:28:30,2023-07-13 16:41:00,0 days 00:15:00,1.325598,1325.598307,1136.000000,1145.000000,900.000000
75%,3.804847e+07,2023-07-19 18:16:00,2023-07-19 18:31:00,0 days 00:15:00,2.215521,2215.521119,1223.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 17:01:00,2023-07-31 17:51:00,0 days 02:00:00,14.143293,14143.293036,9780.000000,9780.000000,7200.000000
std,1.280565e+05,NaN,NaN,0 days 00:10:18.177692991,1.443414,1443.413845,1184.559908,1289.227091,618.177693


In [105]:
get_stats(hot_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,15108,15108
sum,13633020.0,25857.527394
mean,902.370929,1.711512
median,900.0,1.325598
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.870199 Name: trip_distance_kilometers, ..."
max,7200.0,14.143293
min,300.0,0.100057
std_dev,618.177693,1.443414
variance,382143.660112,2.083444
skewness,4.022451,2.294629


In [106]:
# get entries where trips start and end at station
hot_delay_next_station_station = hot_delay_next[hot_delay_next['end_stop_id'].notna() & hot_delay_next['start_stop_id'].notna()]
hot_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1287486,37872114,2023-07-01 17:06:00,2023-07-01 17:16:00,False,False,bike,0 days 00:10:00,0.148911,148.911114,b'\x01\x01\x00\x00\x00\xea<*\xfe\xefh\x1c@uYLl...,b'\x01\x01\x00\x00\x00.py\xac\x19i\x1c@\x11p\x...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 ...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1668989,37800408,2023-07-01 17:06:00,2023-07-01 17:16:00,True,False,bike,0 days 00:10:00,1.313219,1313.218705,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,b'\x01\x01\x00\x00\x00j\x15\xfd\xa1\x99g\x1c@\...,1103.0,Bonn Markt,215356 ...,1160.0,Bonn Nordstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1458370,37973456,2023-07-01 17:11:00,2023-07-01 17:16:00,True,False,bike,0 days 00:05:00,0.431904,431.903635,b'\x01\x01\x00\x00\x00\x1c`\xe6;\xf8Y\x1c@.\xe...,b'\x01\x01\x00\x00\x00\xf9-:Yj]\x1c@To\rl\x95\...,1240.0,Bonn Poppelsdorfer Platz,215356 ...,1224.0,Bonn Am Botanischen Garten,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1334805,37894609,2023-07-01 17:11:00,2023-07-01 17:26:00,False,False,bike,0 days 00:15:00,2.293355,2293.354991,"b'\x01\x01\x00\x00\x00\xb55""\x18\x07g\x1c@.\x1...",b'\x01\x01\x00\x00\x00_\x089\xef\xffS\x1c@\x8a...,686.0,Bonn Universität/Markt,215356 ...,1231.0,Bonn Sebastianstr.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1220395,37717934,2023-07-01 17:16:00,2023-07-01 17:26:00,False,False,bike,0 days 00:10:00,2.084287,2084.287000,b'\x01\x01\x00\x00\x00\x0c\xcc\nE\xba?\x1c@\xf...,b'\x01\x01\x00\x00\x00\x84\x9c\xf7\xffqR\x1c@u...,9702.0,Bonn Im Tannenbusch,215356 ...,688.0,Bonn West,215356 POLYGON ((7.120193524138334 50.7395...,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259554,37823592,2023-07-31 16:46:00,2023-07-31 16:56:00,False,False,bike,0 days 00:10:00,2.773151,2773.150597,b'\x01\x01\x00\x00\x00\xd3P\xa3\x90df\x1c@\x83...,b'\x01\x01\x00\x00\x004f\x12\xf5\x82\x7f\x1c@O...,1102.0,Bonn Friedensplatz,215356 ...,1502.0,Bonn Beueler Bahnhofsplatz,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1638348,37769767,2023-07-31 16:51:00,2023-07-31 17:01:00,False,False,bike,0 days 00:10:00,0.585929,585.928564,b'\x01\x01\x00\x00\x00\x89%\xe5\xees|\x1c@\x10...,b'\x01\x01\x00\x00\x00\xce\x8b\x13_\xedx\x1c@\...,1501.0,Bonn Beuel Rathaus,215356 ...,1513.0,Bonn Rheindorfer Str.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1274292,37831022,2023-07-31 16:51:00,2023-07-31 17:11:00,False,False,bike,0 days 00:20:00,1.427347,1427.346859,b'\x01\x01\x00\x00\x00\xf0\xa5\xf0\xa0\xd9e\x1...,"b'\x01\x01\x00\x00\x00\x89\x08\xff""h\\\x1c@\x1...",1143.0,Bonn Poppelsdorfer Allee,215356 ...,1240.0,Bonn Poppelsdorfer Platz,215356 POLYGON ((7.120193524138334 50.7395...,1200.0
1332550,37860349,2023-07-31 16:56:00,2023-07-31 17:51:00,False,False,bike,0 days 00:55:00,0.123326,123.325782,b'\x01\x01\x00\x00\x00\xa7\xe8H.\xff1\x1c@*S\x...,b'\x01\x01\x00\x00\x00eT\x19\xc6\xdd0\x1c@@\x1...,1441.0,Bonn Chemnitzer Weg,215356 ...,1441.0,Bonn Chemnitzer Weg,215356 POLYGON ((7.120193524138334 50.7395...,3300.0


In [107]:
hot_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,7.068000e+03,7068,7068,7068,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000
mean,3.794077e+07,2023-07-14 15:50:36.528013568,2023-07-14 16:04:19.066213888,0 days 00:13:42.538200339,1.415837,1415.836850,1313.630023,1329.936191,822.538200
min,3.771659e+07,2023-07-01 17:06:00,2023-07-01 17:16:00,0 days 00:05:00,0.100057,100.057033,43.000000,43.000000,300.000000
25%,3.782926e+07,2023-07-08 17:26:00,2023-07-08 17:39:45,0 days 00:10:00,0.695255,695.254853,1102.000000,1102.000000,600.000000
50%,3.794203e+07,2023-07-13 14:16:00,2023-07-13 14:23:30,0 days 00:10:00,1.163160,1163.159703,1140.000000,1150.000000,600.000000
75%,3.804924e+07,2023-07-19 16:06:00,2023-07-19 16:16:00,0 days 00:15:00,1.864079,1864.079189,1223.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 17:01:00,2023-07-31 17:51:00,0 days 02:00:00,8.519064,8519.064426,9780.000000,9780.000000,7200.000000
std,1.279567e+05,NaN,NaN,0 days 00:08:56.720075912,1.075680,1075.679743,1232.230490,1320.705926,536.720076


In [108]:
# get statistics for the tier trips that start and end at a station
get_stats(hot_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,7068,7068
sum,5813700.0,10007.134859
mean,822.5382,1.415837
median,600.0,1.16316
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,7200.0,8.519064
min,300.0,0.100057
std_dev,536.720076,1.07568
variance,288068.439888,1.157087
skewness,4.539711,1.92056


In [109]:
hot_no_delay = no_delay[no_delay['current_temp'] >= 20]
hot_no_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
54949,845,12,845,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,130,1035-845-012-687.2.28:164800-29-1_D9CBC066-3E0...,Fronhof,0,...,0,0,0,0,0,2,evening rush,5,-1,0
54952,630,6,630,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6300109-630-006-1293.2.21:170200-52-1_9656F03F...,Agnetendorfer Str.,0,...,0,0,0,0,0,2,evening rush,5,-1,0
54961,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,26803,660439-66-006-1584.1.11:170200-12-178_6294B012...,Stadthaus,0,...,0,0,0,0,0,2,evening rush,5,-1,0
54968,603,6,603,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6030372-603-006-2304.2.22:165100-41-1_788E386E...,Röttgen Schleife,1,...,0,0,0,evening rush,5,2,evening rush,5,0,0
54973,63,6,63,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,130,630181-63-006-371.1.11:165500-18-1_99C2A22A-99...,Tannenbusch Mitte,0,...,0,0,0,evening rush,5,2,evening rush,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2533787,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6040081-604-006-1382.2.21:161700-49-1_C943A97A...,Hersel,0,...,0,0,1,evening rush,5,0,evening rush,5,0,0
2533789,632,6,632,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6320048-632-006-8832.2.23:165100-26-1_74299D22...,Uniklinikum Süd,1,...,0,0,1,evening rush,5,0,evening rush,5,0,0
2533790,817,12,817,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,28386,62-817-012-2533.2.21:170000-44-1_48D9C082-B062...,Rheinbach Bf,1,...,0,0,1,0,0,0,evening rush,5,-1,0
2533802,613,6,613,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6130126-613-006-1723.2.21:170000-28-1_4763A42E...,Giselherstr.,1,...,0,0,1,evening rush,5,0,evening rush,5,0,0


In [110]:
get_stats(hot_no_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,341204,341204,341204,341204,341204,341204
sum,12050,9674,27138,22502,8223613.56,24620.82
mean,0.035316,0.028353,0.079536,0.065949,24.10175,0.815799
median,0.0,0.0,0.0,0.0,23.04,0.39
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 21.48 Name: current_temp, dtype: float64","0 0.22 Name: current_precipitation_volume, ..."
max,6,5,10,9,35.37,3.89
min,0,0,0,0,20.05,0.12
std_dev,0.216727,0.190618,0.364328,0.323782,3.566736,0.972909
variance,0.04697,0.036335,0.132735,0.104835,12.721607,0.946552
skewness,7.913143,8.333429,6.813867,7.080622,1.292352,2.060908


In [111]:
hot_no_delay_tier = check_micromobility_datasets('tier', hot_no_delay)

In [112]:
hot_no_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,4.762000e+03,4762,4762,4762,4762.000000,4762.000000,3721.000000,2384.000000,4762.000000
mean,7.968593e+08,2023-07-14 15:23:06.425871360,2023-07-14 15:37:22.956740864,0 days 00:14:16.530869382,1.756550,1756.550328,1175.174953,1407.986997,856.530869
min,7.822646e+08,2023-07-01 17:10:00,2023-07-01 17:20:00,0 days 00:05:00,0.100067,100.066815,43.000000,43.000000,300.000000
25%,7.823622e+08,2023-07-08 15:31:15,2023-07-08 15:45:00,0 days 00:10:00,0.748038,748.037782,687.000000,1103.000000,600.000000
50%,7.824561e+08,2023-07-12 20:52:30,2023-07-12 21:02:30,0 days 00:10:00,1.365003,1365.003454,1115.000000,1151.000000,600.000000
75%,8.087285e+08,2023-07-20 11:23:45,2023-07-20 11:37:30,0 days 00:15:00,2.292611,2292.610744,1292.000000,1240.000000,900.000000
max,8.323115e+08,2023-07-31 17:00:00,2023-07-31 18:20:00,0 days 01:55:00,12.402037,12402.036677,9703.000000,9780.000000,6900.000000
std,1.632033e+07,NaN,NaN,0 days 00:09:51.478570873,1.473203,1473.202944,1066.610183,1403.418436,591.478571


In [113]:
get_stats(hot_no_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,4762,4762
sum,4078800.0,8364.692662
mean,856.530869,1.75655
median,600.0,1.365003
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100067 1 0.100253 2 0...
max,6900.0,12.402037
min,300.0,0.100067
std_dev,591.478571,1.473203
variance,349846.899803,2.170327
skewness,4.008182,1.941983


In [114]:
# get entries where trips start and end at station
hot_no_delay_tier_station_station = hot_no_delay_tier[hot_no_delay_tier['end_stop_id'].notna() & hot_no_delay_tier['start_stop_id'].notna()]
hot_no_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3035496,782304577,2023-07-01 17:10:00,2023-07-01 17:20:00,e-scooter,0 days 00:10:00,1.576509,1576.508878,1102.0,Bonn Friedensplatz,801621 POLYGON ((7.157885983913106 50.7227...,1192.0,Bonn Ellerstr.,801621 ...,600.0,POINT (7.09504 50.73646),POINT (7.08100 50.73846)
2954000,782279912,2023-07-01 17:15:00,2023-07-01 17:25:00,e-scooter,0 days 00:10:00,1.327481,1327.480919,1104.0,Bonn Stadthaus,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,600.0,POINT (7.09416 50.73632),POINT (7.08233 50.73793)
3125894,782334920,2023-07-01 17:15:00,2023-07-01 17:25:00,e-scooter,0 days 00:10:00,0.118998,118.997600,694.0,Bonn Tannenbusch Mitte,801621 POLYGON ((7.157885983913106 50.7227...,694.0,Bonn Tannenbusch Mitte,801621 ...,600.0,POINT (7.04656 50.75020),POINT (7.04611 50.74923)
3200651,782382402,2023-07-01 17:20:00,2023-07-01 17:30:00,e-scooter,0 days 00:10:00,0.343392,343.391843,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1144.0,Bonn Beethovenstr.,801621 ...,600.0,POINT (7.09842 50.73095),POINT (7.09538 50.73040)
2954001,782279913,2023-07-01 17:30:00,2023-07-01 17:45:00,e-scooter,0 days 00:15:00,2.267151,2267.150628,688.0,Bonn West,801621 POLYGON ((7.157885983913106 50.7227...,8506.0,Bonn Werftstr.,801621 ...,900.0,POINT (7.08233 50.73793),POINT (7.09003 50.75695)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514366,832293399,2023-07-31 15:55:00,2023-07-31 16:05:00,e-scooter,0 days 00:10:00,0.875062,875.062283,1104.0,Bonn Stadthaus,801621 POLYGON ((7.157885983913106 50.7227...,1144.0,Bonn Beethovenstr.,801621 ...,600.0,POINT (7.09488 50.73777),POINT (7.09699 50.73013)
3518638,832297518,2023-07-31 15:55:00,2023-07-31 16:15:00,e-scooter,0 days 00:20:00,1.765479,1765.479372,1500.0,Bonn Konrad-Adenauer-Platz,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,1200.0,POINT (7.11903 50.73997),POINT (7.10332 50.73766)
3437216,832219346,2023-07-31 16:35:00,2023-07-31 16:40:00,e-scooter,0 days 00:05:00,0.260465,260.465466,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,686.0,Bonn Universität/Markt,801621 ...,300.0,POINT (7.09830 50.73110),POINT (7.10051 50.73186)
3338340,832133612,2023-07-31 16:50:00,2023-07-31 17:00:00,e-scooter,0 days 00:10:00,0.266824,266.823960,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,2667.0,Bonn Brüdergasse/Bertha-Von-Suttner-Platz,801621 ...,600.0,POINT (7.10255 50.73742),POINT (7.10410 50.73558)


In [115]:
hot_no_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.122000e+03,2122,2122,2122,2122.000000,2122.000000,2122.000000,2122.000000,2122.000000
mean,7.965313e+08,2023-07-14 11:37:06.955702272,2023-07-14 11:50:01.555136768,0 days 00:12:54.599434495,1.454874,1454.873507,1217.895382,1440.992460,774.599434
min,7.822648e+08,2023-07-01 17:10:00,2023-07-01 17:20:00,0 days 00:05:00,0.100067,100.066815,43.000000,43.000000,300.000000
25%,7.823558e+08,2023-07-08 15:17:30,2023-07-08 15:23:45,0 days 00:10:00,0.625329,625.329199,687.000000,1106.000000,600.000000
50%,7.824524e+08,2023-07-12 18:20:00,2023-07-12 18:35:00,0 days 00:10:00,1.145465,1145.464979,1115.000000,1151.000000,600.000000
75%,8.087215e+08,2023-07-19 17:08:45,2023-07-19 17:20:00,0 days 00:15:00,1.950278,1950.277619,1223.000000,1240.000000,900.000000
max,8.323095e+08,2023-07-31 17:00:00,2023-07-31 18:20:00,0 days 01:55:00,8.887499,8887.498628,9703.000000,9780.000000,6900.000000
std,1.625937e+07,NaN,NaN,0 days 00:08:23.828300044,1.193219,1193.218856,1093.417140,1443.320639,503.828300


In [116]:
# get statistics for the tier trips that start and end at a station
get_stats(hot_no_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,2122,2122
sum,1643700.0,3087.241582
mean,774.599434,1.454874
median,600.0,1.145465
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100067 1 0.100253 2 0.10...
max,6900.0,8.887499
min,300.0,0.100067
std_dev,503.8283,1.193219
variance,253842.955926,1.423771
skewness,4.894139,1.944952


In [117]:
hot_no_delay_next = check_micromobility_datasets('next', hot_no_delay)

In [118]:
hot_no_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.138800e+04,11388,11388,11388,11388.000000,11388.000000,9451.000000,6412.000000,11388.000000
mean,3.793903e+07,2023-07-15 00:34:11.722866176,2023-07-15 00:49:01.812434176,0 days 00:14:50.089567966,1.682316,1682.315681,1099.844884,1266.504367,890.089568
min,3.771654e+07,2023-07-01 17:06:00,2023-07-01 17:11:00,0 days 00:05:00,0.100317,100.317073,43.000000,43.000000,300.000000
25%,3.782552e+07,2023-07-08 18:56:00,2023-07-08 19:11:00,0 days 00:10:00,0.778792,778.792468,687.000000,1102.000000,600.000000
50%,3.794055e+07,2023-07-13 18:46:00,2023-07-13 18:56:00,0 days 00:10:00,1.303257,1303.256751,1106.000000,1150.000000,600.000000
75%,3.804921e+07,2023-07-20 11:41:00,2023-07-20 11:52:15,0 days 00:15:00,2.105536,2105.536103,1221.000000,1221.000000,900.000000
max,3.817169e+07,2023-07-31 17:01:00,2023-07-31 17:16:00,0 days 02:00:00,13.602096,13602.096364,9780.000000,9780.000000,7200.000000
std,1.290596e+05,NaN,NaN,0 days 00:09:54.853215209,1.431483,1431.483109,911.583087,1155.966793,594.853215


In [119]:
get_stats(hot_no_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,11388,11388
sum,10136340.0,19158.21097
mean,890.089568,1.682316
median,600.0,1.303257
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.376674 Name: trip_distance_kilometers, ..."
max,7200.0,13.602096
min,300.0,0.100317
std_dev,594.853215,1.431483
variance,353850.347646,2.049144
skewness,3.939195,2.357338


In [120]:
# get entries where trips start and end at station
hot_no_delay_next_station_station = hot_no_delay_next[hot_no_delay_next['end_stop_id'].notna() & hot_no_delay_next['start_stop_id'].notna()]
hot_no_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1555237,38113591,2023-07-01 17:06:00,2023-07-01 17:26:00,False,False,bike,0 days 00:20:00,1.766626,1766.626256,b'\x01\x01\x00\x00\x00u\xc7b\x9bTd\x1c@=\xd5!7...,b'\x01\x01\x00\x00\x00v\xfb\xac2Sj\x1c@T7\x17\...,687.0,Bonn Hbf,215356 ...,1255.0,Bonn Graf-Stauffenberg-Str.,215356 POLYGON ((7.120193524138334 50.7395...,1200.0
1334805,37894609,2023-07-01 17:11:00,2023-07-01 17:26:00,False,False,bike,0 days 00:15:00,2.293355,2293.354991,"b'\x01\x01\x00\x00\x00\xb55""\x18\x07g\x1c@.\x1...",b'\x01\x01\x00\x00\x00_\x089\xef\xffS\x1c@\x8a...,686.0,Bonn Universität/Markt,215356 ...,1231.0,Bonn Sebastianstr.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1293982,37875072,2023-07-01 17:26:00,2023-07-01 17:36:00,True,False,bike,0 days 00:10:00,1.155661,1155.660576,b'\x01\x01\x00\x00\x00\xbf\xf4\xf6\xe7\xa2\x81...,"b'\x01\x01\x00\x00\x00""8.\xe3\xa6\x86\x1c@yX\x...",1503.0,Bonn Obere Wilhelmstr.,215356 ...,1126.0,Bonn Gerhardstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1534484,38085499,2023-07-01 17:26:00,2023-07-01 17:36:00,False,False,bike,0 days 00:10:00,1.048712,1048.712047,b'\x01\x01\x00\x00\x00\x17\xd6\x8dwGf\x1c@\xa2...,"b'\x01\x01\x00\x00\x00,(\x0c\xca4j\x1c@\x9f\x8...",1131.0,Bonn Weberstr.,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1393351,37985229,2023-07-01 17:31:00,2023-07-01 17:41:00,False,False,bike,0 days 00:10:00,1.183792,1183.792223,b'\x01\x01\x00\x00\x00>\xce4a\xfbi\x1c@\xc3\x8...,b'\x01\x01\x00\x00\x00\x18{/\xbeh_\x1c@\x14?\x...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 ...,1150.0,Bonn Dorotheenstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531046,38082061,2023-07-31 16:31:00,2023-07-31 16:41:00,False,True,bike,0 days 00:10:00,0.413295,413.294604,b'\x01\x01\x00\x00\x00u\xccy\xc6\xbed\x1c@wN\x...,b'\x01\x01\x00\x00\x00\x06.\x8f5#c\x1c@\x9dJ\x...,687.0,Bonn Hbf,215356 ...,1102.0,Bonn Friedensplatz,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1391497,37923983,2023-07-31 16:56:00,2023-07-31 17:06:00,False,False,bike,0 days 00:10:00,1.469302,1469.302141,b'\x01\x01\x00\x00\x00\x9b\xaf\x92\x8f\xddU\x1...,b'\x01\x01\x00\x00\x00g\r\xdeW\xe5b\x1c@\x19\x...,1221.0,Bonn Kaufmannstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1531048,38082063,2023-07-31 16:56:00,2023-07-31 17:06:00,False,False,bike,0 days 00:10:00,0.473075,473.075103,b'\x01\x01\x00\x00\x00e\xe2VA\x0cd\x1c@\x96\xc...,b'\x01\x01\x00\x00\x00\x9f\xcd\xaa\xcf\xd5f\x1...,687.0,Bonn Hbf,215356 ...,1135.0,Bonn Sankt Petrus-Krankenhaus,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1638349,37769768,2023-07-31 17:01:00,2023-07-31 17:11:00,False,False,bike,0 days 00:10:00,0.218677,218.676771,b'\x01\x01\x00\x00\x00\xce\x8b\x13_\xedx\x1c@\...,b'\x01\x01\x00\x00\x00|G\x8d\t1w\x1c@\xda\xac\...,1513.0,Bonn Rheindorfer Str.,215356 ...,1958.0,Bonn An Der Wolfsburg,215356 POLYGON ((7.120193524138334 50.7395...,600.0


In [121]:
hot_no_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,5.695000e+03,5695,5695,5695,5695.000000,5695.000000,5695.000000,5695.000000,5695.000000
mean,3.794051e+07,2023-07-14 22:46:00.126426624,2023-07-14 22:59:33.555750656,0 days 00:13:33.429323968,1.388230,1388.230411,1150.150658,1288.564004,813.429324
min,3.771659e+07,2023-07-01 17:06:00,2023-07-01 17:26:00,0 days 00:05:00,0.100317,100.317073,43.000000,43.000000,300.000000
25%,3.782823e+07,2023-07-08 19:18:30,2023-07-08 19:31:00,0 days 00:10:00,0.713790,713.790212,687.000000,1102.000000,600.000000
50%,3.794125e+07,2023-07-13 16:46:00,2023-07-13 16:51:00,0 days 00:10:00,1.153066,1153.066377,1115.000000,1150.000000,600.000000
75%,3.804916e+07,2023-07-20 10:41:00,2023-07-20 10:58:30,0 days 00:15:00,1.725704,1725.703935,1190.000000,1221.000000,900.000000
max,3.817169e+07,2023-07-31 17:01:00,2023-07-31 17:16:00,0 days 02:00:00,8.605918,8605.918179,9780.000000,9780.000000,7200.000000
std,1.285435e+05,NaN,NaN,0 days 00:08:45.527370780,1.070928,1070.927741,1002.920760,1175.291491,525.527371


In [122]:
# get statistics for the tier trips that start and end at a station
get_stats(hot_no_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,5695,5695
sum,4632480.0,7905.972189
mean,813.429324,1.38823
median,600.0,1.153066
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.353333 1 0.982570 2 2.896818 Name...
max,7200.0,8.605918
min,300.0,0.100317
std_dev,525.527371,1.070928
variance,276179.01744,1.146886
skewness,4.623509,2.117209


In [123]:
hot_nan_delay = nan_delay[nan_delay['current_temp'] >= 20]
hot_nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
54946,608,6,608,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6080181-608-006-1374.2.21:160200-46-1_39A8D120...,Gielgen,0,...,0,0,0,0,0,2,0,0,-1,1
54947,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,26814,660427-66-006-1788.1.11:163200-24-178_A2FC505B...,Stadthaus,0,...,0,0,0,0,0,2,0,0,-1,1
54950,608,6,608,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6080181-608-006-1374.2.21:160200-46-1_39A8D120...,Gielgen,0,...,0,0,0,0,0,2,0,0,-1,1
54953,613,6,613,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6130368-613-006-1723.2.21:165600-28-1_EC37774E...,Giselherstr.,1,...,0,0,0,0,0,2,0,0,-1,1
54954,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,130,6040159-604-006-1382.2.21:161800-49-1_95AC1C3A...,Hersel,0,...,0,0,0,0,0,2,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2533800,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,27301,661277-66-006-1788.1.11:163200-37-177_492F9F5F...,Siegburg Bf,0,...,0,0,1,0,0,0,0,0,-1,1
2533806,636,6,636,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6360060-636-006-1802.2.22:164700-18-1_21425944...,Ramersdorf,1,...,0,0,1,0,0,0,0,0,-1,1
2533807,610,6,610,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6100097-610-006-1756.2.21:160600-54-1_8608C1A3...,Duisdorf Bf,0,...,0,0,1,0,0,0,0,0,-1,1
2533809,630,6,630,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6300046-630-006-9478.2.22:165500-50-1_39117666...,Uniklinikum Süd,1,...,0,0,1,0,0,0,0,0,-1,1


In [124]:
get_stats(hot_nan_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,574544,574544,574544,574544,574544,574544
sum,12532,10062,31398,26034,13617406.81,46983.42
mean,0.021812,0.017513,0.054649,0.045312,23.701243,0.854897
median,0.0,0.0,0.0,0.0,22.79,0.33
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 24.53 Name: current_temp, dtype: float64","0 0.22 Name: current_precipitation_volume, ..."
max,6,4,9,9,35.37,3.89
min,0,0,0,0,20.05,0.12
std_dev,0.162859,0.144375,0.289661,0.258917,3.162681,1.008974
variance,0.026523,0.020844,0.083903,0.067038,10.002554,1.018028
skewness,8.931975,9.680618,7.63146,8.138844,1.383495,1.89901


In [125]:
hot_nan_delay_tier = check_micromobility_datasets('tier', hot_nan_delay)

In [126]:
hot_nan_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,6.658000e+03,6658,6658,6658,6658.000000,6658.000000,4989.000000,3454.000000,6658.000000
mean,7.978019e+08,2023-07-14 17:27:49.240012032,2023-07-14 17:42:18.708320512,0 days 00:14:29.468308801,1.783101,1783.101220,1329.271798,1409.917776,869.468309
min,7.822646e+08,2023-07-01 17:10:00,2023-07-01 17:20:00,0 days 00:05:00,0.100246,100.246164,43.000000,43.000000,300.000000
25%,7.823633e+08,2023-07-08 07:47:30,2023-07-08 07:58:45,0 days 00:10:00,0.754282,754.282149,692.000000,1102.000000,600.000000
50%,8.086221e+08,2023-07-13 15:25:00,2023-07-13 15:37:30,0 days 00:10:00,1.368439,1368.439042,1115.000000,1150.000000,600.000000
75%,8.087353e+08,2023-07-20 15:53:45,2023-07-20 16:05:00,0 days 00:15:00,2.343563,2343.563128,1302.000000,1255.000000,900.000000
max,8.323115e+08,2023-07-31 17:05:00,2023-07-31 18:20:00,0 days 01:55:00,13.583168,13583.167599,9703.000000,9780.000000,6900.000000
std,1.651799e+07,NaN,NaN,0 days 00:10:10.533042542,1.522934,1522.933576,1368.993271,1432.131836,610.533043


In [127]:
get_stats(hot_nan_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,6658,6658
sum,5788920.0,11871.887923
mean,869.468309,1.783101
median,600.0,1.368439
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100246 1 0.100253 2 0...
max,6900.0,13.583168
min,300.0,0.100246
std_dev,610.533043,1.522934
variance,372750.596036,2.319327
skewness,3.930875,2.114761


In [128]:
# get entries where trips start and end at station
hot_nan_delay_tier_station_station = hot_nan_delay_tier[hot_nan_delay_tier['end_stop_id'].notna() & hot_nan_delay_tier['start_stop_id'].notna()]
hot_nan_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3035496,782304577,2023-07-01 17:10:00,2023-07-01 17:20:00,e-scooter,0 days 00:10:00,1.576509,1576.508878,1102.0,Bonn Friedensplatz,801621 POLYGON ((7.157885983913106 50.7227...,1192.0,Bonn Ellerstr.,801621 ...,600.0,POINT (7.09504 50.73646),POINT (7.08100 50.73846)
3279241,782454774,2023-07-01 17:10:00,2023-07-01 17:25:00,e-scooter,0 days 00:15:00,1.181511,1181.510808,1182.0,Bonn Chlodwigplatz,801621 POLYGON ((7.157885983913106 50.7227...,1184.0,Bonn Gerhardsplatz,801621 ...,900.0,POINT (7.09033 50.74356),POINT (7.07971 50.74384)
2954000,782279912,2023-07-01 17:15:00,2023-07-01 17:25:00,e-scooter,0 days 00:10:00,1.327481,1327.480919,1104.0,Bonn Stadthaus,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,600.0,POINT (7.09416 50.73632),POINT (7.08233 50.73793)
3125894,782334920,2023-07-01 17:15:00,2023-07-01 17:25:00,e-scooter,0 days 00:10:00,0.118998,118.997600,694.0,Bonn Tannenbusch Mitte,801621 POLYGON ((7.157885983913106 50.7227...,694.0,Bonn Tannenbusch Mitte,801621 ...,600.0,POINT (7.04656 50.75020),POINT (7.04611 50.74923)
3200651,782382402,2023-07-01 17:20:00,2023-07-01 17:30:00,e-scooter,0 days 00:10:00,0.343392,343.391843,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1144.0,Bonn Beethovenstr.,801621 ...,600.0,POINT (7.09842 50.73095),POINT (7.09538 50.73040)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3406811,832190220,2023-07-31 16:45:00,2023-07-31 17:00:00,e-scooter,0 days 00:15:00,3.037464,3037.463725,1221.0,Bonn Kaufmannstr.,801621 POLYGON ((7.157885983913106 50.7227...,684.0,Bonn Bundesrechnungshof/Auswärtiges Amt,801621 ...,900.0,POINT (7.08402 50.72817),POINT (7.11106 50.72429)
3325487,832124849,2023-07-31 16:45:00,2023-07-31 16:50:00,e-scooter,0 days 00:05:00,0.275064,275.063895,1523.0,Bonn Vilich Kloster,801621 POLYGON ((7.157885983913106 50.7227...,1522.0,Bonn Vilich,801621 ...,300.0,POINT (7.12727 50.75225),POINT (7.12940 50.75098)
3338340,832133612,2023-07-31 16:50:00,2023-07-31 17:00:00,e-scooter,0 days 00:10:00,0.266824,266.823960,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,2667.0,Bonn Brüdergasse/Bertha-Von-Suttner-Platz,801621 ...,600.0,POINT (7.10255 50.73742),POINT (7.10410 50.73558)
3305017,832110902,2023-07-31 17:00:00,2023-07-31 18:20:00,e-scooter,0 days 01:20:00,0.257841,257.841257,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1106.0,Bonn Thomas-Mann-Str.,801621 ...,4800.0,POINT (7.09690 50.73260),POINT (7.09462 50.73304)


In [129]:
hot_nan_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.959000e+03,2959,2959,2959,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000
mean,7.974242e+08,2023-07-14 14:31:37.330179072,2023-07-14 14:44:43.291652608,0 days 00:13:05.961473470,1.473089,1473.089359,1383.008787,1417.840487,785.961473
min,7.822648e+08,2023-07-01 17:10:00,2023-07-01 17:20:00,0 days 00:05:00,0.100246,100.246164,43.000000,43.000000,300.000000
25%,7.823586e+08,2023-07-08 08:05:00,2023-07-08 08:17:30,0 days 00:10:00,0.630372,630.372338,1102.000000,1103.000000,600.000000
50%,7.824604e+08,2023-07-13 12:30:00,2023-07-13 12:35:00,0 days 00:10:00,1.181818,1181.818249,1115.000000,1151.000000,600.000000
75%,8.087298e+08,2023-07-20 12:52:30,2023-07-20 13:10:00,0 days 00:15:00,1.973796,1973.796349,1240.000000,1245.000000,900.000000
max,8.323095e+08,2023-07-31 17:00:00,2023-07-31 18:20:00,0 days 01:55:00,13.583168,13583.167599,9703.000000,9780.000000,6900.000000
std,1.658466e+07,NaN,NaN,0 days 00:09:13.795814167,1.210278,1210.277721,1437.368562,1395.892131,553.795814


In [130]:
# get statistics for the tier trips that start and end at a station
get_stats(hot_nan_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,2959,2959
sum,2325660.0,4358.871414
mean,785.961473,1.473089
median,600.0,1.181818
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100246 1 0.100253 2 0...
max,6900.0,13.583168
min,300.0,0.100246
std_dev,553.795814,1.210278
variance,306689.803789,1.464772
skewness,4.803469,2.227482


In [131]:
hot_nan_delay_next = check_micromobility_datasets('next', hot_nan_delay)

In [132]:
hot_nan_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.657300e+04,16573,16573,16573,16573.000000,16573.000000,13202.000000,9588.000000,16573.000000
mean,3.793945e+07,2023-07-14 19:02:55.101671424,2023-07-14 19:17:47.625656320,0 days 00:14:52.523984794,1.693949,1693.948591,1172.412665,1270.275657,892.523985
min,3.771654e+07,2023-07-01 17:06:00,2023-07-01 17:16:00,0 days 00:05:00,0.100057,100.057033,43.000000,43.000000,300.000000
25%,3.782628e+07,2023-07-08 00:21:00,2023-07-08 00:31:00,0 days 00:10:00,0.771752,771.751599,687.000000,1102.000000,600.000000
50%,3.794129e+07,2023-07-13 17:36:00,2023-07-13 17:51:00,0 days 00:10:00,1.302835,1302.835130,1115.000000,1145.000000,600.000000
75%,3.804992e+07,2023-07-20 13:01:00,2023-07-20 13:16:00,0 days 00:15:00,2.148580,2148.580173,1221.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 17:06:00,2023-07-31 17:16:00,0 days 02:00:00,14.013714,14013.713672,9780.000000,9780.000000,7200.000000
std,1.287150e+05,NaN,NaN,0 days 00:10:06.051972677,1.435419,1435.418761,1028.902944,1186.454948,606.051973


In [133]:
get_stats(hot_nan_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,16573,16573
sum,14791800.0,28073.81
mean,892.523985,1.693949
median,600.0,1.302835
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.870199 Name: trip_distance_kilometers, ..."
max,7200.0,14.013714
min,300.0,0.100057
std_dev,606.051973,1.435419
variance,367298.993587,2.060427
skewness,4.065068,2.338478


In [134]:
# get entries where trips start and end at station
hot_nan_delay_next_station_station = hot_nan_delay_next[hot_nan_delay_next['end_stop_id'].notna() & hot_nan_delay_next['start_stop_id'].notna()]
hot_nan_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1574793,37748052,2023-07-01 17:06:00,2023-07-01 17:16:00,False,False,bike,0 days 00:10:00,1.046114,1046.114110,b'\x01\x01\x00\x00\x00`vO\x1e\x16j\x1c@m\xff\x...,b'\x01\x01\x00\x00\x007n1?7d\x1c@l\x94\xf5\x9b...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 ...,1160.0,Bonn Nordstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1555237,38113591,2023-07-01 17:06:00,2023-07-01 17:26:00,False,False,bike,0 days 00:20:00,1.766626,1766.626256,b'\x01\x01\x00\x00\x00u\xc7b\x9bTd\x1c@=\xd5!7...,b'\x01\x01\x00\x00\x00v\xfb\xac2Sj\x1c@T7\x17\...,687.0,Bonn Hbf,215356 ...,1255.0,Bonn Graf-Stauffenberg-Str.,215356 POLYGON ((7.120193524138334 50.7395...,1200.0
1287486,37872114,2023-07-01 17:06:00,2023-07-01 17:16:00,False,False,bike,0 days 00:10:00,0.148911,148.911114,b'\x01\x01\x00\x00\x00\xea<*\xfe\xefh\x1c@uYLl...,b'\x01\x01\x00\x00\x00.py\xac\x19i\x1c@\x11p\x...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 ...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1668989,37800408,2023-07-01 17:06:00,2023-07-01 17:16:00,True,False,bike,0 days 00:10:00,1.313219,1313.218705,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,b'\x01\x01\x00\x00\x00j\x15\xfd\xa1\x99g\x1c@\...,1103.0,Bonn Markt,215356 ...,1160.0,Bonn Nordstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1458370,37973456,2023-07-01 17:11:00,2023-07-01 17:16:00,True,False,bike,0 days 00:05:00,0.431904,431.903635,b'\x01\x01\x00\x00\x00\x1c`\xe6;\xf8Y\x1c@.\xe...,b'\x01\x01\x00\x00\x00\xf9-:Yj]\x1c@To\rl\x95\...,1240.0,Bonn Poppelsdorfer Platz,215356 ...,1224.0,Bonn Am Botanischen Garten,215356 POLYGON ((7.120193524138334 50.7395...,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531048,38082063,2023-07-31 16:56:00,2023-07-31 17:06:00,False,False,bike,0 days 00:10:00,0.473075,473.075103,b'\x01\x01\x00\x00\x00e\xe2VA\x0cd\x1c@\x96\xc...,b'\x01\x01\x00\x00\x00\x9f\xcd\xaa\xcf\xd5f\x1...,687.0,Bonn Hbf,215356 ...,1135.0,Bonn Sankt Petrus-Krankenhaus,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1460547,38027186,2023-07-31 17:01:00,2023-07-31 17:16:00,False,False,bike,0 days 00:15:00,1.106455,1106.455028,b'\x01\x01\x00\x00\x00h\xeau\x8b\xc0h\x1c@\xf5...,b'\x01\x01\x00\x00\x00T9\xed)9g\x1c@\x12\xa2|A...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 ...,1131.0,Bonn Weberstr.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1638349,37769768,2023-07-31 17:01:00,2023-07-31 17:11:00,False,False,bike,0 days 00:10:00,0.218677,218.676771,b'\x01\x01\x00\x00\x00\xce\x8b\x13_\xedx\x1c@\...,b'\x01\x01\x00\x00\x00|G\x8d\t1w\x1c@\xda\xac\...,1513.0,Bonn Rheindorfer Str.,215356 ...,1958.0,Bonn An Der Wolfsburg,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1649035,37780454,2023-07-31 17:01:00,2023-07-31 17:11:00,False,False,bike,0 days 00:10:00,0.559276,559.275923,b'\x01\x01\x00\x00\x00m6Vb\x9ee\x1c@\x80\xf1\x...,b'\x01\x01\x00\x00\x00\xa8\xa9ek}a\x1c@\x95e\x...,1174.0,Bonn Augustinum,215356 ...,1172.0,Bonn Bataverweg,215356 POLYGON ((7.120193524138334 50.7395...,600.0


In [135]:
hot_nan_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,8.223000e+03,8223,8223,8223,8223.000000,8223.000000,8223.000000,8223.000000,8223.000000
mean,3.794099e+07,2023-07-14 18:32:36.913535232,2023-07-14 18:46:14.170010880,0 days 00:13:37.256475738,1.399583,1399.582689,1197.162714,1291.760428,817.256476
min,3.771656e+07,2023-07-01 17:06:00,2023-07-01 17:16:00,0 days 00:05:00,0.100057,100.057033,43.000000,43.000000,300.000000
25%,3.782704e+07,2023-07-08 10:31:00,2023-07-08 10:38:30,0 days 00:10:00,0.706824,706.823926,687.000000,1102.000000,600.000000
50%,3.794457e+07,2023-07-13 16:46:00,2023-07-13 17:01:00,0 days 00:10:00,1.150986,1150.986132,1115.000000,1150.000000,600.000000
75%,3.805120e+07,2023-07-20 12:33:30,2023-07-20 12:51:00,0 days 00:15:00,1.787010,1787.010395,1221.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 17:06:00,2023-07-31 17:16:00,0 days 02:00:00,9.110475,9110.475451,9780.000000,9780.000000,7200.000000
std,1.290244e+05,NaN,NaN,0 days 00:09:00.610809139,1.076943,1076.942703,1046.827557,1208.093595,540.610809


In [136]:
# get statistics for the tier trips that start and end at a station
get_stats(hot_nan_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,8223,8223
sum,6720300.0,11508.76845
mean,817.256476,1.399583
median,600.0,1.150986
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,7200.0,9.110475
min,300.0,0.100057
std_dev,540.610809,1.076943
variance,292260.046958,1.159806
skewness,4.701582,2.077951


In [137]:
hot_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,550796.000000,550796.000000,550796.000000,550796.000000,550796.000000,550796.000000,550796.000000,536194,550796,550796.000000,...,550796.000000,550796.000000,550796.000000,550796.0,550796.000000,550796.000000,550796.000000,550796.000000,550796.000000,550796.0
mean,567.394066,6.005701,2.768980,2782.943879,0.515309,3592.054298,2318.965067,2023-07-15 10:03:14.330083840,2023-07-15 10:31:10.531539968,146616.115520,...,0.687187,148032.503695,151995.410007,0.0,0.106816,3.469143,0.788719,3.563450,0.641025,0.0
min,60.000000,6.000000,0.000000,3.000000,0.000000,6.000000,43.000000,2023-07-01 17:02:00,2023-07-01 17:02:15,4306.000000,...,0.000000,0.000000,15.000000,0.0,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0
25%,602.000000,6.000000,3.000000,64.000000,0.000000,1915.000000,1214.000000,2023-07-09 13:43:15,2023-07-09 13:42:18.750000128,112306.000000,...,0.250000,122015.000000,123515.000000,0.0,0.000000,2.000000,0.000000,3.000000,0.000000,0.0
50%,607.000000,6.000000,3.000000,64.000000,1.000000,2717.000000,1441.000000,2023-07-14 11:46:22.500000,2023-07-14 12:05:45,144304.000000,...,0.500000,152330.000000,153245.000000,0.0,0.000000,4.000000,0.000000,4.000000,1.000000,0.0
75%,611.000000,6.000000,3.000000,130.000000,1.000000,5633.000000,1682.000000,2023-07-20 16:10:30,2023-07-20 16:33:45,180804.000000,...,0.750000,182700.000000,183115.000000,0.0,0.000000,5.000000,2.000000,5.000000,1.000000,0.0
max,843.000000,12.000000,3.000000,27690.000000,1.000000,10396.000000,9780.000000,2023-07-31 17:11:45,2023-07-31 17:12:00,235308.000000,...,112.500000,235930.000000,235950.000000,0.0,2.000000,8.000000,3.000000,8.000000,1.000000,0.0
std,148.470662,0.175135,0.799807,8066.303902,0.499766,2793.966736,2412.816350,NaN,NaN,44499.659885,...,1.842866,46202.518906,39729.182615,0.0,0.366459,1.676717,1.042014,1.595638,0.532103,0.0


In [138]:
hot_no_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,341204.000000,341204.000000,341204.000000,341204.000000,341204.000000,341204.000000,341204.000000,245766,341204,341204.00000,...,341204.000000,341204.000000,341204.000000,341204.0,341204.000000,341204.000000,341204.000000,341204.000000,341204.000000,341204.0
mean,566.819900,7.043757,2.634078,5486.471208,0.501383,4237.475337,2204.467752,2023-07-15 09:46:22.464417280,2023-07-15 10:13:45.841915904,146444.06618,...,0.039981,109673.794533,151851.773968,0.0,0.108243,2.565527,0.803250,3.552461,-0.272043,0.0
min,16.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,2023-07-01 17:01:00,2023-07-01 17:02:00,4306.00000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0
25%,600.000000,6.000000,3.000000,64.000000,0.000000,2014.000000,1178.000000,2023-07-09 12:30:45,2023-07-09 12:37:00,112306.00000,...,0.000000,0.000000,123500.000000,0.0,0.000000,0.000000,0.000000,3.000000,-1.000000,0.0
50%,607.000000,6.000000,3.000000,64.000000,1.000000,3858.000000,1415.000000,2023-07-14 12:36:00,2023-07-14 12:43:00,144304.00000,...,0.000000,131645.000000,153000.000000,0.0,0.000000,3.000000,0.000000,4.000000,0.000000,0.0
75%,630.000000,6.000000,3.000000,151.000000,1.000000,6377.000000,1710.000000,2023-07-20 16:36:45,2023-07-20 17:03:00,180804.00000,...,0.000000,172445.000000,183300.000000,0.0,0.000000,4.000000,2.000000,5.000000,0.000000,0.0
max,884.000000,12.000000,3.000000,29145.000000,1.000000,10396.000000,9780.000000,2023-07-31 17:01:00,2023-07-31 17:10:15,235308.00000,...,111.500000,235945.000000,235945.000000,0.0,2.000000,8.000000,3.000000,8.000000,1.000000,0.0
std,211.589725,2.119405,0.981769,10867.057084,0.499999,2858.997179,2328.004607,NaN,NaN,44332.03668,...,0.531076,76366.743360,40193.180405,0.0,0.368931,2.103531,1.040156,1.605820,0.461920,0.0


In [139]:
hot_nan_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,574544.000000,574544.000000,574544.000000,574544.000000,574544.000000,574544.000000,574544.000000,0,0,574544.000000,...,0.0,574544.0,574544.0,574544.0,574544.000000,574544.0,574544.000000,574544.0,574544.0,574544.0
mean,554.725462,6.303153,2.673581,3988.372128,0.488093,4077.487501,2260.285653,NaT,NaT,136933.427802,...,NaN,0.0,0.0,0.0,0.114522,0.0,0.793654,0.0,-1.0,1.0
min,16.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,NaT,NaT,3304.000000,...,NaN,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,-1.0,1.0
25%,601.000000,6.000000,3.000000,64.000000,0.000000,1975.000000,1221.000000,NaT,NaT,105306.000000,...,NaN,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,-1.0,1.0
50%,607.000000,6.000000,3.000000,64.000000,0.000000,2952.000000,1459.000000,NaT,NaT,135305.000000,...,NaN,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,-1.0,1.0
75%,613.000000,6.000000,3.000000,130.000000,1.000000,6327.000000,1687.000000,NaT,NaT,164304.000000,...,NaN,0.0,0.0,0.0,0.000000,0.0,2.000000,0.0,-1.0,1.0
max,884.000000,12.000000,3.000000,29145.000000,1.000000,10396.000000,9780.000000,NaT,NaT,235308.000000,...,NaN,0.0,0.0,0.0,2.000000,0.0,3.000000,0.0,-1.0,1.0
std,181.897839,1.199013,0.934189,9473.425808,0.499859,2921.125439,2346.635963,NaN,NaN,38337.890496,...,NaN,0.0,0.0,0.0,0.373017,0.0,0.989270,0.0,0.0,0.0


#### Rain

In [140]:
no_rain_delay = delay[(delay['current_description'] == 'clear sky') | (delay['current_description'] == 'few clouds') | (delay['current_description'] == 'scattered clouds') | (delay['current_description'] == 'broken clouds') | (delay['current_description'] == 'overcast clouds') | (delay['current_description'] == 'mist')]
no_rain_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
0,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
1,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
2,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
3,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
4,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522730,611,6,611,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6110068-611-006-1338.2.21:142000-58-1_CB741FED...,Pappelweg,1,...,0-5,0,0,afternoon,4,0,afternoon,4,1,0
2522734,600,6,600,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6000059-600-006-1308.2.21:144400-36-1_505679CA...,Kranenweg,0,...,0-5,0,0,afternoon,4,0,afternoon,4,1,0
2522736,600,6,600,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6000058-600-006-8813.2.22:145800-32-1_925D6264...,Ippendorf Altenheim,1,...,0-5,0,0,afternoon,4,0,afternoon,4,1,0
2522740,610,6,610,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6100085-610-006-1756.2.21:140600-54-1_A3FFA64F...,Duisdorf Bf,0,...,0-5,0,0,afternoon,4,0,afternoon,4,1,0


In [141]:
get_stats(no_rain_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,815264,815264,815264,815264,815264,815264
sum,11048,9046,30086,25235,17567007.66,0.0
mean,0.013551,0.011096,0.036903,0.030953,21.547631,NaN
median,0.0,0.0,0.0,0.0,21.23,NaN
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 19.72 Name: current_temp, dtype: float64","Series([], Name: current_precipitation_volume,..."
max,6,6,8,8,35.37,NaN
min,0,0,0,0,11.13,NaN
std_dev,0.126453,0.113668,0.218088,0.196764,4.607837,NaN
variance,0.01599,0.012921,0.047562,0.038716,21.232162,NaN
skewness,11.212823,12.142137,7.531516,7.863257,0.612602,NaN


In [142]:
no_rain_delay_tier = check_micromobility_datasets('tier', no_rain_delay)

In [143]:
no_rain_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,7.618000e+03,7618,7618,7618,7618.000000,7618.000000,5348.000000,4200.000000,7618.000000
mean,7.969766e+08,2023-07-14 16:43:31.157784064,2023-07-14 16:57:40.609083904,0 days 00:14:09.451299553,1.745784,1745.784092,1437.288519,1365.568095,849.451300
min,7.822645e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100290,100.289988,43.000000,43.000000,300.000000
25%,7.823662e+08,2023-07-07 15:50:00,2023-07-07 16:00:00,0 days 00:10:00,0.751221,751.220931,1103.000000,1102.000000,600.000000
50%,7.824591e+08,2023-07-13 09:32:30,2023-07-13 09:45:00,0 days 00:10:00,1.339369,1339.369451,1135.000000,1145.000000,600.000000
75%,8.087295e+08,2023-07-21 11:15:00,2023-07-21 11:20:00,0 days 00:15:00,2.316255,2316.255064,1245.000000,1240.000000,900.000000
max,8.323141e+08,2023-07-31 15:00:00,2023-07-31 15:15:00,0 days 01:55:00,13.908704,13908.703513,9780.000000,9703.000000,6900.000000
std,1.618954e+07,NaN,NaN,0 days 00:09:42.832366865,1.444909,1444.908919,1452.713794,1352.651913,582.832367


In [144]:
get_stats(no_rain_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,7618,7618
sum,6471120.0,13299.383212
mean,849.4513,1.745784
median,600.0,1.339369
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100290 1 0.100631 2 0...
max,6900.0,13.908704
min,300.0,0.10029
std_dev,582.832367,1.444909
variance,339693.567866,2.087762
skewness,3.878222,1.881001


In [145]:
# get entries where trips start and end at station
no_rain_delay_tier_station_station = no_rain_delay_tier[no_rain_delay_tier['end_stop_id'].notna() & no_rain_delay_tier['start_stop_id'].notna()]
no_rain_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3171653,782356027,2023-06-30 01:45:00,2023-06-30 02:05:00,e-scooter,0 days 00:20:00,3.598056,3598.055590,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1406.0,Bonn Kopenhagener Str.,801621 ...,1200.0,POINT (7.09506 50.73298),POINT (7.07331 50.75712)
3209527,782391260,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.670973,670.973151,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10180 50.73728),POINT (7.09852 50.73217)
3210269,782391943,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.838693,838.692953,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10410 50.73699),POINT (7.09815 50.73232)
3234236,782413798,2023-07-01 06:40:00,2023-07-01 06:45:00,e-scooter,0 days 00:05:00,0.844529,844.529088,1131.0,Bonn Weberstr.,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10205 50.72540),POINT (7.09679 50.73093)
3164060,782350073,2023-07-01 07:10:00,2023-07-01 07:15:00,e-scooter,0 days 00:05:00,0.941159,941.158971,1182.0,Bonn Chlodwigplatz,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,300.0,POINT (7.08933 50.74278),POINT (7.08230 50.73803)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3439234,832221293,2023-07-31 14:15:00,2023-07-31 14:25:00,e-scooter,0 days 00:10:00,0.941351,941.351063,1110.0,Bonn Wilhelmsplatz,801621 POLYGON ((7.157885983913106 50.7227...,686.0,Bonn Universität/Markt,801621 ...,600.0,POINT (7.09791 50.74142),POINT (7.10061 50.73333)
3376248,832160872,2023-07-31 14:20:00,2023-07-31 14:35:00,e-scooter,0 days 00:15:00,1.364117,1364.116656,1620.0,Bonn Max-Bruch-Str.,801621 POLYGON ((7.157885983913106 50.7227...,1144.0,Bonn Beethovenstr.,801621 ...,900.0,POINT (7.08061 50.72682),POINT (7.09252 50.72979)
3508084,832287375,2023-07-31 14:30:00,2023-07-31 14:45:00,e-scooter,0 days 00:15:00,2.525054,2525.054040,1143.0,Bonn Poppelsdorfer Allee,801621 POLYGON ((7.157885983913106 50.7227...,1620.0,Bonn Max-Bruch-Str.,801621 ...,900.0,POINT (7.09900 50.73069),POINT (7.07706 50.72478)
3491611,832271568,2023-07-31 14:30:00,2023-07-31 14:45:00,e-scooter,0 days 00:15:00,2.123778,2123.777937,1525.0,Bonn Geislar Mitte,801621 POLYGON ((7.157885983913106 50.7227...,1513.0,Bonn Rheindorfer Str.,801621 ...,900.0,POINT (7.12531 50.76065),POINT (7.11721 50.74322)


In [146]:
no_rain_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.464000e+03,3464,3464,3464,3464.000000,3464.000000,3464.000000,3464.000000,3464.000000
mean,7.963017e+08,2023-07-14 11:51:09.024249600,2023-07-14 12:04:00.866050816,0 days 00:12:51.841801385,1.426126,1426.126102,1440.631351,1398.688799,771.841801
min,7.822648e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100290,100.289988,43.000000,43.000000,300.000000
25%,7.823613e+08,2023-07-07 16:25:00,2023-07-07 16:43:45,0 days 00:10:00,0.620358,620.358320,1103.000000,1103.000000,600.000000
50%,7.824519e+08,2023-07-12 18:05:00,2023-07-12 18:22:30,0 days 00:10:00,1.106414,1106.413744,1136.000000,1150.000000,600.000000
75%,8.087224e+08,2023-07-20 19:21:15,2023-07-20 19:35:00,0 days 00:15:00,1.937052,1937.052331,1240.000000,1241.000000,900.000000
max,8.323115e+08,2023-07-31 14:35:00,2023-07-31 15:05:00,0 days 01:45:00,9.093761,9093.761020,9780.000000,9703.000000,6300.000000
std,1.613486e+07,NaN,NaN,0 days 00:08:31.430151325,1.136163,1136.162767,1445.392724,1373.397614,511.430151


In [147]:
# get statistics for the tier trips that start and end at a station
get_stats(no_rain_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,3464,3464
sum,2673660.0,4940.100817
mean,771.841801,1.426126
median,600.0,1.106414
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100290 1 0.100631 2 0.10...
max,6300.0,9.093761
min,300.0,0.10029
std_dev,511.430151,1.136163
variance,261560.799684,1.290866
skewness,4.109731,1.631712


In [148]:
no_rain_delay_next = check_micromobility_datasets('next', no_rain_delay)

In [149]:
no_rain_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.985900e+04,19859,19859,19859,19859.000000,19859.000000,14584.000000,11522.000000,19859.000000
mean,3.793963e+07,2023-07-14 23:08:36.742031104,2023-07-14 23:23:29.025630464,0 days 00:14:52.283599375,1.735011,1735.010625,1296.244446,1286.816612,892.283599
min,3.771653e+07,2023-07-01 05:16:00,2023-07-01 05:46:00,0 days 00:05:00,0.100057,100.057033,43.000000,43.000000,300.000000
25%,3.782823e+07,2023-07-07 17:48:30,2023-07-07 18:08:30,0 days 00:10:00,0.770417,770.417442,1102.000000,697.000000,600.000000
50%,3.794164e+07,2023-07-13 18:31:00,2023-07-13 18:46:00,0 days 00:15:00,1.346937,1346.937138,1140.000000,1144.000000,900.000000
75%,3.804870e+07,2023-07-20 20:21:00,2023-07-20 20:31:00,0 days 00:15:00,2.252389,2252.389408,1224.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 15:06:00,2023-07-31 15:26:00,0 days 02:00:00,14.143293,14143.293036,9780.000000,9780.000000,7200.000000
std,1.280206e+05,NaN,NaN,0 days 00:09:54.401954989,1.444429,1444.428654,1212.081184,1275.901321,594.401955


In [150]:
get_stats(no_rain_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,19859,19859
sum,17719860.0,34455.575994
mean,892.283599,1.735011
median,900.0,1.346937
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.870199 Name: trip_distance_kilometers, ..."
max,7200.0,14.143293
min,300.0,0.100057
std_dev,594.401955,1.444429
variance,353313.684096,2.086374
skewness,3.965375,2.215026


In [151]:
# get entries where trips start and end at station
no_rain_delay_next_station_station = no_rain_delay_next[no_rain_delay_next['end_stop_id'].notna() & no_rain_delay_next['start_stop_id'].notna()]
no_rain_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1417189,37961351,2023-07-01 05:41:00,2023-07-01 05:56:00,False,False,bike,0 days 00:15:00,1.671334,1671.334291,"b'\x01\x01\x00\x00\x00\xd9\n\x9a\x96XY\x1c@""\x...",b'\x01\x01\x00\x00\x00\x82\x8d\xeb\xdf\xf5I\x1...,1151.0,Bonn Frankenbad/Kunstverein,215356 ...,43.0,Bonn Propsthof Nord,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1220393,37717932,2023-07-01 05:56:00,2023-07-01 06:11:00,False,False,bike,0 days 00:15:00,1.377444,1377.444442,b'\x01\x01\x00\x00\x00\xcfj\x81=&R\x1c@\x7f\xa...,b'\x01\x01\x00\x00\x00\x9f\xe4\x0e\x9b\xc8L\x1...,8437.0,Bonn An Der Josefshöhe,215356 ...,1190.0,Bonn Zeisigweg,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1235649,37726317,2023-07-01 06:06:00,2023-07-01 06:21:00,False,False,bike,0 days 00:15:00,2.055591,2055.591133,"b""\x01\x01\x00\x00\x00\xf0\xf8\xf6\xaeA_\x1c@\...",b'\x01\x01\x00\x00\x00B\xd1<\x80En\x1c@\x84*5{...,1182.0,Bonn Chlodwigplatz,215356 ...,685.0,Bonn Juridicum,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1267255,37827523,2023-07-01 06:06:00,2023-07-01 06:16:00,False,False,bike,0 days 00:10:00,1.000358,1000.357907,b'\x01\x01\x00\x00\x00\xc0\x06D\x88+g\x1c@\xef...,b'\x01\x01\x00\x00\x00\x8f\x89\x94f\xf3h\x1c@f...,1160.0,Bonn Nordstr.,215356 ...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1638081,37769500,2023-07-01 06:11:00,2023-07-01 06:21:00,False,True,bike,0 days 00:10:00,0.914302,914.302426,b'\x01\x01\x00\x00\x00\xc5V\xd0\xb4\xc4Z\x1c@\...,b'\x01\x01\x00\x00\x00\x06.\x8f5#c\x1c@\x9dJ\x...,1153.0,Bonn Heerstr.,215356 ...,1102.0,Bonn Friedensplatz,215356 POLYGON ((7.120193524138334 50.7395...,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547892,38095768,2023-07-31 14:51:00,2023-07-31 15:01:00,False,False,bike,0 days 00:10:00,0.948854,948.853815,b'\x01\x01\x00\x00\x00\x8bO\x010\x9eQ\x1c@\xe4...,b'\x01\x01\x00\x00\x00H\x17\x9bV\nQ\x1c@\xbb\x...,1180.0,Bonn Sportpark Nord,215356 ...,1192.0,Bonn Ellerstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1593518,38133637,2023-07-31 14:51:00,2023-07-31 15:01:00,False,False,bike,0 days 00:10:00,0.995287,995.287120,b'\x01\x01\x00\x00\x00O\x94\x84D\xdaf\x1c@\xa8...,b'\x01\x01\x00\x00\x00\xdcGnM\xba]\x1c@v\xdf1<...,686.0,Bonn Universität/Markt,215356 ...,1141.0,Bonn Herwarthstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1358517,37908635,2023-07-31 14:51:00,2023-07-31 15:06:00,False,False,bike,0 days 00:15:00,1.685195,1685.195181,b'\x01\x01\x00\x00\x00\xfeF;n\xf8]\x1c@\x98\xa...,b'\x01\x01\x00\x00\x00\x944\x7fLkc\x1c@1\xce\x...,1153.0,Bonn Heerstr.,215356 ...,1173.0,Bonn Pädagogische Fakultät,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1286703,37871331,2023-07-31 15:01:00,2023-07-31 15:06:00,False,False,bike,0 days 00:05:00,0.811857,811.857002,b'\x01\x01\x00\x00\x00>Y1\\\x1d0\x1c@\xb0\xac4...,b'\x01\x01\x00\x00\x00\x15TT\xfdJ7\x1c@\x85\xc...,695.0,Bonn Buschdorf,215356 ...,2534.0,Bonn Peter-Klein-Str.,215356 POLYGON ((7.120193524138334 50.7395...,300.0


In [152]:
no_rain_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,9.366000e+03,9366,9366,9366,9366.000000,9366.000000,9366.000000,9366.000000,9366.000000
mean,3.794137e+07,2023-07-14 19:25:27.219730944,2023-07-14 19:39:01.434977792,0 days 00:13:34.215246636,1.432449,1432.448995,1324.343476,1320.809417,814.215247
min,3.771653e+07,2023-07-01 05:41:00,2023-07-01 05:56:00,0 days 00:05:00,0.100057,100.057033,43.000000,43.000000,300.000000
25%,3.782939e+07,2023-07-07 17:16:00,2023-07-07 17:27:15,0 days 00:10:00,0.713068,713.068146,1102.000000,1102.000000,600.000000
50%,3.794409e+07,2023-07-13 15:31:00,2023-07-13 15:41:00,0 days 00:10:00,1.182477,1182.477076,1143.000000,1145.000000,600.000000
75%,3.805002e+07,2023-07-20 18:31:00,2023-07-20 18:41:00,0 days 00:15:00,1.874713,1874.712664,1223.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 15:06:00,2023-07-31 15:26:00,0 days 02:00:00,9.640671,9640.671235,9780.000000,9780.000000,7200.000000
std,1.280624e+05,NaN,NaN,0 days 00:08:21.961276944,1.072112,1072.111628,1252.193518,1326.252553,501.961277


In [153]:
# get statistics for the tier trips that start and end at a station
get_stats(no_rain_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,9366,9366
sum,7625940.0,13416.317288
mean,814.215247,1.432449
median,600.0,1.182477
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,7200.0,9.640671
min,300.0,0.100057
std_dev,501.961277,1.072112
variance,251965.123552,1.149423
skewness,4.318548,1.853853


In [154]:
no_rain_no_delay = no_delay[(no_delay['current_description'] == 'clear sky') | (no_delay['current_description'] == 'few clouds') | (no_delay['current_description'] == 'scattered clouds') | (no_delay['current_description'] == 'broken clouds') | (no_delay['current_description'] == 'overcast clouds') | (no_delay['current_description'] == 'mist')]
no_rain_no_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
0,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
1,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
2,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
3,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
4,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522735,606,6,606,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6060102-606-006-1584.2.22:143200-44-1_FEF0563C...,Hardtberg Klinikum,1,...,0,0,0,afternoon,4,0,afternoon,4,0,0
2522737,800,12,800,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,64,30-800-012-1690.2.22:142600-26-1_141C1AC4-54EF...,Bonn Hbf,1,...,0,0,0,0,0,0,afternoon,4,-1,0
2522739,611,6,611,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6110113-611-006-1723.2.21:143000-60-1_E2594585...,Lessenich Sportplatz,0,...,0,0,0,afternoon,4,0,afternoon,4,0,0
2522744,606,6,606,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6060100-606-006-1584.2.22:141200-44-1_F1395CA9...,Hardtberg Klinikum,1,...,0,0,0,afternoon,4,0,afternoon,4,0,0


In [155]:
get_stats(no_rain_no_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,513631,513631,513631,513631,513631,513631
sum,16063,13012,36715,30643,11009947.56,0.0
mean,0.031273,0.025333,0.071481,0.05966,21.43552,NaN
median,0.0,0.0,0.0,0.0,20.99,NaN
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 19.72 Name: current_temp, dtype: float64","Series([], Name: current_precipitation_volume,..."
max,6,6,10,9,35.37,NaN
min,0,0,0,0,11.13,NaN
std_dev,0.202378,0.179104,0.345214,0.307006,4.595252,NaN
variance,0.040957,0.032078,0.119172,0.094252,21.116345,NaN
skewness,8.314497,8.774547,7.24754,7.387358,0.621244,NaN


In [156]:
no_rain_no_delay_tier = check_micromobility_datasets('tier', no_rain_no_delay)

In [157]:
no_rain_no_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,6.397000e+03,6397,6397,6397,6397.000000,6397.000000,5000.000000,3234.000000,6397.000000
mean,7.971985e+08,2023-07-14 19:31:37.039237376,2023-07-14 19:45:51.127090432,0 days 00:14:14.087853681,1.781022,1781.022066,1174.340600,1394.750773,854.087854
min,7.822646e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100067,100.066815,43.000000,43.000000,300.000000
25%,7.823643e+08,2023-07-07 17:50:00,2023-07-07 18:05:00,0 days 00:10:00,0.753628,753.627930,687.000000,1103.000000,600.000000
50%,7.824612e+08,2023-07-13 11:50:00,2023-07-13 12:20:00,0 days 00:10:00,1.370798,1370.798468,1110.000000,1150.000000,600.000000
75%,8.087319e+08,2023-07-21 18:15:00,2023-07-21 18:30:00,0 days 00:15:00,2.344850,2344.849787,1292.000000,1244.000000,900.000000
max,8.323097e+08,2023-07-31 14:50:00,2023-07-31 15:15:00,0 days 02:00:00,12.402037,12402.036677,9703.000000,9780.000000,7200.000000
std,1.618907e+07,NaN,NaN,0 days 00:09:57.132610905,1.487761,1487.760793,1078.477086,1380.811369,597.132611


In [158]:
get_stats(no_rain_no_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,6397,6397
sum,5463600.0,11393.198154
mean,854.087854,1.781022
median,600.0,1.370798
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100067 1 0.100253 2 0...
max,7200.0,12.402037
min,300.0,0.100067
std_dev,597.132611,1.487761
variance,356567.355007,2.213432
skewness,4.078018,1.866929


In [159]:
# get entries where trips start and end at station
no_rain_no_delay_tier_station_station = no_rain_no_delay_tier[no_rain_no_delay_tier['end_stop_id'].notna() & no_rain_no_delay_tier['start_stop_id'].notna()]
no_rain_no_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3171653,782356027,2023-06-30 01:45:00,2023-06-30 02:05:00,e-scooter,0 days 00:20:00,3.598056,3598.055590,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1406.0,Bonn Kopenhagener Str.,801621 ...,1200.0,POINT (7.09506 50.73298),POINT (7.07331 50.75712)
3209527,782391260,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.670973,670.973151,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10180 50.73728),POINT (7.09852 50.73217)
3210269,782391943,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.838693,838.692953,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10410 50.73699),POINT (7.09815 50.73232)
3254146,782431910,2023-07-01 01:50:00,2023-07-01 02:00:00,e-scooter,0 days 00:10:00,1.779253,1779.253071,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,600.0,POINT (7.09818 50.73206),POINT (7.08279 50.73648)
3194413,782376718,2023-07-01 05:55:00,2023-07-01 06:05:00,e-scooter,0 days 00:10:00,0.177056,177.056127,701.0,Bonn Plittersdorfer Str.,801621 POLYGON ((7.157885983913106 50.7227...,701.0,Bonn Plittersdorfer Str.,801621 ...,600.0,POINT (7.15233 50.68885),POINT (7.15228 50.68725)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3386231,832170491,2023-07-31 14:10:00,2023-07-31 14:25:00,e-scooter,0 days 00:15:00,2.302870,2302.870354,1102.0,Bonn Friedensplatz,801621 POLYGON ((7.157885983913106 50.7227...,1500.0,Bonn Konrad-Adenauer-Platz,801621 ...,900.0,POINT (7.09628 50.73706),POINT (7.11691 50.73893)
3439234,832221293,2023-07-31 14:15:00,2023-07-31 14:25:00,e-scooter,0 days 00:10:00,0.941351,941.351063,1110.0,Bonn Wilhelmsplatz,801621 POLYGON ((7.157885983913106 50.7227...,686.0,Bonn Universität/Markt,801621 ...,600.0,POINT (7.09791 50.74142),POINT (7.10061 50.73333)
3441793,832223754,2023-07-31 14:20:00,2023-07-31 14:30:00,e-scooter,0 days 00:10:00,1.026721,1026.720700,1500.0,Bonn Konrad-Adenauer-Platz,801621 POLYGON ((7.157885983913106 50.7227...,1504.0,Bonn Beuel Bf,801621 ...,600.0,POINT (7.11818 50.73933),POINT (7.12737 50.73843)
3523189,832301874,2023-07-31 14:25:00,2023-07-31 14:40:00,e-scooter,0 days 00:15:00,1.388031,1388.030832,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1136.0,Bonn Geißlerstr.,801621 ...,900.0,POINT (7.09934 50.73201),POINT (7.10314 50.72002)


In [160]:
no_rain_no_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.869000e+03,2869,2869,2869,2869.000000,2869.000000,2869.000000,2869.000000,2869.000000
mean,7.967402e+08,2023-07-14 14:42:15.726734080,2023-07-14 14:55:16.626001920,0 days 00:13:00.899268037,1.487202,1487.201910,1210.996166,1424.329035,780.899268
min,7.822648e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100067,100.066815,43.000000,43.000000,300.000000
25%,7.823561e+08,2023-07-07 17:55:00,2023-07-07 18:05:00,0 days 00:10:00,0.637220,637.219782,687.000000,1106.000000,600.000000
50%,7.824552e+08,2023-07-12 22:30:00,2023-07-12 22:40:00,0 days 00:10:00,1.152548,1152.547962,1115.000000,1151.000000,600.000000
75%,8.087294e+08,2023-07-21 15:15:00,2023-07-21 15:25:00,0 days 00:15:00,2.010168,2010.167540,1223.000000,1244.000000,900.000000
max,8.323097e+08,2023-07-31 14:50:00,2023-07-31 14:55:00,0 days 01:55:00,8.887499,8887.498628,9703.000000,9780.000000,6900.000000
std,1.617781e+07,NaN,NaN,0 days 00:08:51.521486505,1.209648,1209.647533,1077.946388,1407.059012,531.521487


In [161]:
# get statistics for the tier trips that start and end at a station
get_stats(no_rain_no_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,2869,2869
sum,2240400.0,4266.782279
mean,780.899268,1.487202
median,600.0,1.152548
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100067 1 0.100253 2 0.10...
max,6900.0,8.887499
min,300.0,0.100067
std_dev,531.521487,1.209648
variance,282515.090617,1.463247
skewness,4.58828,1.761272


In [162]:
no_rain_no_delay_next = check_micromobility_datasets('next', no_rain_no_delay)

In [163]:
no_rain_no_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.561800e+04,15618,15618,15618,15618.000000,15618.000000,12960.000000,8857.000000,15618.000000
mean,3.793945e+07,2023-07-15 00:42:54.199001088,2023-07-15 00:57:34.118325248,0 days 00:14:39.919323857,1.710067,1710.066619,1091.813117,1251.760980,879.919324
min,3.771653e+07,2023-07-01 01:31:00,2023-07-01 01:46:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782666e+07,2023-07-07 19:06:00,2023-07-07 19:21:00,0 days 00:10:00,0.794260,794.259574,687.000000,1102.000000,600.000000
50%,3.794164e+07,2023-07-13 21:13:30,2023-07-13 21:26:00,0 days 00:10:00,1.318544,1318.544493,1106.000000,1145.000000,600.000000
75%,3.804865e+07,2023-07-21 09:16:00,2023-07-21 09:29:45,0 days 00:15:00,2.170299,2170.299144,1221.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 15:06:00,2023-07-31 15:26:00,0 days 02:00:00,13.602096,13602.096364,9780.000000,9780.000000,7200.000000
std,1.286583e+05,NaN,NaN,0 days 00:09:31.054430955,1.429783,1429.783411,883.666399,1139.375629,571.054431


In [164]:
get_stats(no_rain_no_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,15618,15618
sum,13742580.0,26707.820454
mean,879.919324,1.710067
median,600.0,1.318544
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.46698 Name: trip_distance_kilometers, d..."
max,7200.0,13.602096
min,300.0,0.100004
std_dev,571.054431,1.429783
variance,326103.163114,2.044281
skewness,3.78664,2.240093


In [165]:
# get entries where trips start and end at station
no_rain_no_delay_next_station_station = no_rain_no_delay_next[no_rain_no_delay_next['end_stop_id'].notna() & no_rain_no_delay_next['start_stop_id'].notna()]
no_rain_no_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1292973,37874063,2023-07-01 01:31:00,2023-07-01 01:46:00,False,False,bike,0 days 00:15:00,2.239755,2239.754526,"b""\x01\x01\x00\x00\x00{-\xe8\xbd1d\x1c@R'\xa0\...",b'\x01\x01\x00\x00\x00L\x8ce\xfa%b\x1c@;\xe0\x...,687.0,Bonn Hbf,215356 ...,1173.0,Bonn Pädagogische Fakultät,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1311815,37850623,2023-07-01 01:31:00,2023-07-01 01:46:00,False,False,bike,0 days 00:15:00,1.331223,1331.223283,b'\x01\x01\x00\x00\x00\x81\xcd9x&d\x1c@g\x7f\x...,b'\x01\x01\x00\x00\x00f\xf7\xe4a\xa1f\x1c@}\xc...,687.0,Bonn Hbf,215356 ...,1160.0,Bonn Nordstr.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1513945,38069151,2023-07-01 01:31:00,2023-07-01 01:46:00,False,False,bike,0 days 00:15:00,1.392093,1392.093481,b'\x01\x01\x00\x00\x00V\xb9P\xf9\xd7b\x1c@b\xd...,b'\x01\x01\x00\x00\x00)\x93\x1a\xda\x00\\\x1c@...,687.0,Bonn Hbf,215356 ...,1240.0,Bonn Poppelsdorfer Platz,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1444489,38015256,2023-07-01 05:31:00,2023-07-01 05:36:00,False,True,bike,0 days 00:05:00,0.109052,109.052108,b'\x01\x01\x00\x00\x00\x0c@\xa3t\xe9/\x1c@\x1c...,b'\x01\x01\x00\x00\x00W\xb2c#\x10/\x1c@ap\xcd\...,694.0,Bonn Tannenbusch Mitte,215356 ...,694.0,Bonn Tannenbusch Mitte,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1433891,38006735,2023-07-01 06:21:00,2023-07-01 06:36:00,False,False,bike,0 days 00:15:00,1.550245,1550.245342,b'\x01\x01\x00\x00\x00\xa9lXSYT\x1c@\xb0\xe6\x...,b'\x01\x01\x00\x00\x00\xdapX\x1a\xf8a\x1c@(\x0...,1221.0,Bonn Kaufmannstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301122,37878906,2023-07-31 14:56:00,2023-07-31 15:06:00,False,False,bike,0 days 00:10:00,1.354012,1354.011648,b'\x01\x01\x00\x00\x00\xa1\x9f\xa9\xd7-b\x1c@\...,"b'\x01\x01\x00\x00\x00""\xa7\xaf\xe7kV\x1c@;\x8...",687.0,Bonn Hbf,215356 ...,1221.0,Bonn Kaufmannstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1467817,37975497,2023-07-31 14:56:00,2023-07-31 15:06:00,False,False,bike,0 days 00:10:00,0.443949,443.949088,b'\x01\x01\x00\x00\x00)\x94\x85\xaf\xafe\x1c@h...,b'\x01\x01\x00\x00\x00\xd4\xd2\xdc\nae\x1c@\xb...,687.0,Bonn Hbf,215356 ...,1102.0,Bonn Friedensplatz,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1533238,38084253,2023-07-31 14:56:00,2023-07-31 15:16:00,False,False,bike,0 days 00:20:00,1.684933,1684.933448,b'\x01\x01\x00\x00\x00\xccD\x11R\xb7c\x1c@>w\x...,b'\x01\x01\x00\x00\x00(eRC\x1b`\x1c@K\x1f\xba\...,687.0,Bonn Hbf,215356 ...,1176.0,Bonn Husarenstr.,215356 POLYGON ((7.120193524138334 50.7395...,1200.0
1506797,38063030,2023-07-31 15:01:00,2023-07-31 15:16:00,False,False,bike,0 days 00:15:00,3.304795,3304.795051,b'\x01\x01\x00\x00\x00\xdf\xf8\xda3Kb\x1c@\x9e...,b'\x01\x01\x00\x00\x00P4\x0f`\x91\x7f\x1c@\\\x...,687.0,Bonn Hbf,215356 ...,1502.0,Bonn Beueler Bahnhofsplatz,215356 POLYGON ((7.120193524138334 50.7395...,900.0


In [166]:
no_rain_no_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,7.841000e+03,7841,7841,7841,7841.000000,7841.000000,7841.000000,7841.000000,7841.000000
mean,3.794191e+07,2023-07-14 20:58:35.031246080,2023-07-14 21:12:03.359265536,0 days 00:13:28.328019385,1.420519,1420.518973,1136.561536,1275.553373,808.328019
min,3.771653e+07,2023-07-01 01:31:00,2023-07-01 01:46:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.783098e+07,2023-07-07 16:41:00,2023-07-07 16:56:00,0 days 00:10:00,0.737467,737.466947,687.000000,1102.000000,600.000000
50%,3.794452e+07,2023-07-13 16:41:00,2023-07-13 16:51:00,0 days 00:10:00,1.183764,1183.764041,1115.000000,1150.000000,600.000000
75%,3.804916e+07,2023-07-21 07:01:00,2023-07-21 07:11:00,0 days 00:15:00,1.784507,1784.506774,1184.000000,1221.000000,900.000000
max,3.817169e+07,2023-07-31 15:06:00,2023-07-31 15:16:00,0 days 02:00:00,9.640671,9640.671235,9780.000000,9780.000000,7200.000000
std,1.280289e+05,NaN,NaN,0 days 00:08:22.825052285,1.079999,1079.998977,955.575586,1163.944526,502.825052


In [167]:
# get statistics for the tier trips that start and end at a station
get_stats(no_rain_no_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,7841,7841
sum,6338100.0,11138.289265
mean,808.328019,1.420519
median,600.0,1.183764
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.222838 1 0.982570 Name: trip_distanc...
max,7200.0,9.640671
min,300.0,0.100004
std_dev,502.825052,1.079999
variance,252833.033206,1.166398
skewness,4.428703,2.074253


In [168]:
no_rain_nan_delay = nan_delay[(nan_delay['current_description'] == 'clear sky') | (nan_delay['current_description'] == 'few clouds') | (nan_delay['current_description'] == 'scattered clouds') | (nan_delay['current_description'] == 'broken clouds') | (nan_delay['current_description'] == 'overcast clouds') | (nan_delay['current_description'] == 'mist')]
no_rain_nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
41,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,255-551-008-2071.2.22:033200-34-157_C0741D60-B...,Bonn Hbf,1,...,0,0,0,0,0,1,0,0,-1,1
42,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
43,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
44,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
45,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522729,608,6,608,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6080090-608-006-1574.2.24:144800-47-1_56E2C36D...,Hardthöhe/Südwache,1,...,0,0,0,0,0,0,0,0,-1,1
2522732,608,6,608,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6080087-608-006-1374.2.21:150100-46-1_E92E01D6...,Gielgen,0,...,0,0,0,0,0,0,0,0,-1,1
2522738,605,6,605,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6050094-605-006-1415.2.22:142200-43-1_AC8E987F...,Duisdorf Bf,1,...,0,0,0,0,0,0,0,0,-1,1
2522742,633,6,633,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,6330043-633-006-8832.2.23:142500-26-1_D84D0A28...,Ramersdorf,0,...,0,0,0,0,0,0,0,0,-1,1


In [169]:
get_stats(no_rain_nan_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,853143,853143,853143,853143,853143,853143
sum,16570,13437,42752,35604,18137312.27,0.0
mean,0.019422,0.01575,0.050111,0.041733,21.259405,NaN
median,0.0,0.0,0.0,0.0,20.91,NaN
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 19.72 Name: current_temp, dtype: float64","Series([], Name: current_precipitation_volume,..."
max,6,5,11,9,35.37,NaN
min,0,0,0,0,11.13,NaN
std_dev,0.153249,0.136746,0.277691,0.247751,4.22836,NaN
variance,0.023485,0.0187,0.077112,0.06138,17.879026,NaN
skewness,9.526205,10.333837,8.207235,8.452619,0.509636,NaN


In [170]:
no_rain_nan_delay_tier = check_micromobility_datasets('tier', no_rain_nan_delay)

In [171]:
no_rain_nan_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,8.720000e+03,8720,8720,8720,8720.000000,8720.000000,6537.000000,4575.000000,8720.000000
mean,7.973436e+08,2023-07-14 15:26:40.871559424,2023-07-14 15:41:00.082568960,0 days 00:14:19.211009174,1.793982,1793.981967,1324.788741,1406.193443,859.211009
min,7.822645e+08,2023-07-01 01:25:00,2023-07-01 01:40:00,0 days 00:05:00,0.100042,100.042262,43.000000,43.000000,300.000000
25%,7.823613e+08,2023-07-07 11:43:45,2023-07-07 11:55:00,0 days 00:10:00,0.751581,751.580740,691.000000,1102.000000,600.000000
50%,7.824624e+08,2023-07-13 13:35:00,2023-07-13 13:52:30,0 days 00:10:00,1.363337,1363.337426,1115.000000,1146.000000,600.000000
75%,8.087316e+08,2023-07-21 16:06:15,2023-07-21 16:22:30,0 days 00:15:00,2.350962,2350.961637,1301.000000,1255.000000,900.000000
max,8.323115e+08,2023-07-31 14:50:00,2023-07-31 16:40:00,0 days 01:55:00,13.908704,13908.703513,9703.000000,9780.000000,6900.000000
std,1.626492e+07,NaN,NaN,0 days 00:10:00.007586042,1.537483,1537.483434,1353.529133,1433.611489,600.007586


In [172]:
get_stats(no_rain_nan_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,8720,8720
sum,7492320.0,15643.522749
mean,859.211009,1.793982
median,600.0,1.363337
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100042 1 0.100246 2 0...
max,6900.0,13.908704
min,300.0,0.100042
std_dev,600.007586,1.537483
variance,360009.103308,2.363855
skewness,3.866193,2.067425


In [173]:
# get entries where trips start and end at station
no_rain_nan_delay_tier_station_station = no_rain_nan_delay_tier[no_rain_nan_delay_tier['end_stop_id'].notna() & no_rain_nan_delay_tier['start_stop_id'].notna()]
no_rain_nan_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3190399,782373031,2023-07-01 01:25:00,2023-07-01 01:40:00,e-scooter,0 days 00:15:00,1.055901,1055.901028,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1110.0,Bonn Wilhelmsplatz,801621 ...,900.0,POINT (7.10146 50.73271),POINT (7.09772 50.74150)
3209527,782391260,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.670973,670.973151,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10180 50.73728),POINT (7.09852 50.73217)
3210269,782391943,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.838693,838.692953,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10410 50.73699),POINT (7.09815 50.73232)
3201996,782383745,2023-07-01 02:20:00,2023-07-01 02:25:00,e-scooter,0 days 00:05:00,0.567479,567.479089,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1103.0,Bonn Markt,801621 ...,300.0,POINT (7.09831 50.73236),POINT (7.10279 50.73481)
3276897,782452649,2023-07-01 02:20:00,2023-07-01 02:25:00,e-scooter,0 days 00:05:00,0.589011,589.011497,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1103.0,Bonn Markt,801621 ...,300.0,POINT (7.09832 50.73240),POINT (7.10306 50.73479)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3523189,832301874,2023-07-31 14:25:00,2023-07-31 14:40:00,e-scooter,0 days 00:15:00,1.388031,1388.030832,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1136.0,Bonn Geißlerstr.,801621 ...,900.0,POINT (7.09934 50.73201),POINT (7.10314 50.72002)
3508084,832287375,2023-07-31 14:30:00,2023-07-31 14:45:00,e-scooter,0 days 00:15:00,2.525054,2525.054040,1143.0,Bonn Poppelsdorfer Allee,801621 POLYGON ((7.157885983913106 50.7227...,1620.0,Bonn Max-Bruch-Str.,801621 ...,900.0,POINT (7.09900 50.73069),POINT (7.07706 50.72478)
3491611,832271568,2023-07-31 14:30:00,2023-07-31 14:45:00,e-scooter,0 days 00:15:00,2.123778,2123.777937,1525.0,Bonn Geislar Mitte,801621 POLYGON ((7.157885983913106 50.7227...,1513.0,Bonn Rheindorfer Str.,801621 ...,900.0,POINT (7.12531 50.76065),POINT (7.11721 50.74322)
3482733,832263064,2023-07-31 14:35:00,2023-07-31 14:40:00,e-scooter,0 days 00:05:00,0.574873,574.872838,1162.0,Bonn Rosental,801621 POLYGON ((7.157885983913106 50.7227...,1111.0,Bonn Beethovenhalle Und Swb,801621 ...,300.0,POINT (7.09703 50.74234),POINT (7.10219 50.74209)


In [174]:
no_rain_nan_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.930000e+03,3930,3930,3930,3930.000000,3930.000000,3930.000000,3930.000000,3930.000000
mean,7.969405e+08,2023-07-14 11:45:45.648854784,2023-07-14 11:58:53.297709568,0 days 00:13:07.648854961,1.489328,1489.327994,1368.325445,1412.240204,787.648855
min,7.822645e+08,2023-07-01 01:25:00,2023-07-01 01:40:00,0 days 00:05:00,0.100246,100.246164,43.000000,43.000000,300.000000
25%,7.823576e+08,2023-07-07 12:45:00,2023-07-07 12:56:15,0 days 00:10:00,0.644933,644.932621,700.000000,1103.000000,600.000000
50%,7.824558e+08,2023-07-13 07:25:00,2023-07-13 07:32:30,0 days 00:10:00,1.180775,1180.774505,1115.000000,1151.000000,600.000000
75%,8.087295e+08,2023-07-21 11:33:45,2023-07-21 11:38:45,0 days 00:15:00,1.986798,1986.798176,1240.000000,1255.000000,900.000000
max,8.323115e+08,2023-07-31 14:50:00,2023-07-31 15:05:00,0 days 01:55:00,13.583168,13583.167599,9703.000000,9780.000000,6900.000000
std,1.629934e+07,NaN,NaN,0 days 00:09:05.975366098,1.222871,1222.871048,1399.499880,1390.141605,545.975366


In [175]:
# get statistics for the tier trips that start and end at a station
get_stats(no_rain_nan_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,3930,3930
sum,3095460.0,5853.059016
mean,787.648855,1.489328
median,600.0,1.180775
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100246 1 0.100253 2 0...
max,6900.0,13.583168
min,300.0,0.100246
std_dev,545.975366,1.222871
variance,298089.100387,1.495414
skewness,4.404873,2.090722


In [176]:
no_rain_nan_delay_next = check_micromobility_datasets('next', no_rain_nan_delay)

In [177]:
no_rain_nan_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.272600e+04,22726,22726,22726,22726.000000,22726.000000,18135.000000,13159.000000,22726.000000
mean,3.793914e+07,2023-07-14 17:40:13.388189696,2023-07-14 17:54:53.468274176,0 days 00:14:40.080084484,1.709678,1709.677783,1177.969506,1245.138232,880.080084
min,3.771653e+07,2023-07-01 01:56:00,2023-07-01 02:06:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782665e+07,2023-07-07 10:21:00,2023-07-07 10:36:00,0 days 00:10:00,0.779328,779.327622,687.000000,699.000000,600.000000
50%,3.794129e+07,2023-07-13 16:36:00,2023-07-13 16:48:30,0 days 00:10:00,1.319725,1319.725102,1115.000000,1144.000000,600.000000
75%,3.804921e+07,2023-07-21 08:46:00,2023-07-21 09:01:00,0 days 00:15:00,2.194924,2194.923648,1221.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 15:06:00,2023-07-31 15:26:00,0 days 02:00:00,14.104859,14104.859070,9780.000000,9780.000000,7200.000000
std,1.286203e+05,NaN,NaN,0 days 00:09:37.953243595,1.427946,1427.945912,1054.819504,1145.144657,577.953244


In [178]:
get_stats(no_rain_nan_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,22726,22726
sum,20000700.0,38854.137293
mean,880.080084,1.709678
median,600.0,1.319725
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.870199 Name: trip_distance_kilometers, ..."
max,7200.0,14.104859
min,300.0,0.100004
std_dev,577.953244,1.427946
variance,334029.951782,2.03903
skewness,3.973865,2.256743


In [179]:
# get entries where trips start and end at station
no_rain_nan_delay_next_station_station = no_rain_nan_delay_next[no_rain_nan_delay_next['end_stop_id'].notna() & no_rain_nan_delay_next['start_stop_id'].notna()]
no_rain_nan_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1635116,37766535,2023-07-01 05:16:00,2023-07-01 05:26:00,False,True,bike,0 days 00:10:00,1.003667,1003.666723,"b""\x01\x01\x00\x00\x00Z\xb7A\xed\xb7\x96\x1c@L...",b'\x01\x01\x00\x00\x00t\xef\xe1\x92\xe3\x9e\x1...,8950.0,Bonn Schießbergweg,215356 ...,1584.0,Bonn Ramersdorf,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1444489,38015256,2023-07-01 05:31:00,2023-07-01 05:36:00,False,True,bike,0 days 00:05:00,0.109052,109.052108,b'\x01\x01\x00\x00\x00\x0c@\xa3t\xe9/\x1c@\x1c...,b'\x01\x01\x00\x00\x00W\xb2c#\x10/\x1c@ap\xcd\...,694.0,Bonn Tannenbusch Mitte,215356 ...,694.0,Bonn Tannenbusch Mitte,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1311817,37850625,2023-07-01 06:21:00,2023-07-01 06:36:00,False,False,bike,0 days 00:15:00,1.284558,1284.557726,b'\x01\x01\x00\x00\x00}\xeb\xc3z\xa3f\x1c@}\xc...,b'\x01\x01\x00\x00\x001\x08\xac\x1cZd\x1c@C\x9...,1160.0,Bonn Nordstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1433891,38006735,2023-07-01 06:21:00,2023-07-01 06:36:00,False,False,bike,0 days 00:15:00,1.550245,1550.245342,b'\x01\x01\x00\x00\x00\xa9lXSYT\x1c@\xb0\xe6\x...,b'\x01\x01\x00\x00\x00\xdapX\x1a\xf8a\x1c@(\x0...,1221.0,Bonn Kaufmannstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1471081,38035639,2023-07-01 06:26:00,2023-07-01 06:41:00,False,False,bike,0 days 00:15:00,1.898569,1898.568609,b'\x01\x01\x00\x00\x00\xbb\x99\xd1\x8f\x86c\x1...,b'\x01\x01\x00\x00\x00#.\x00\x8d\xd2e\x1c@\xd5...,687.0,Bonn Hbf,215356 ...,1174.0,Bonn Augustinum,215356 POLYGON ((7.120193524138334 50.7395...,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301122,37878906,2023-07-31 14:56:00,2023-07-31 15:06:00,False,False,bike,0 days 00:10:00,1.354012,1354.011648,b'\x01\x01\x00\x00\x00\xa1\x9f\xa9\xd7-b\x1c@\...,"b'\x01\x01\x00\x00\x00""\xa7\xaf\xe7kV\x1c@;\x8...",687.0,Bonn Hbf,215356 ...,1221.0,Bonn Kaufmannstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1467817,37975497,2023-07-31 14:56:00,2023-07-31 15:06:00,False,False,bike,0 days 00:10:00,0.443949,443.949088,b'\x01\x01\x00\x00\x00)\x94\x85\xaf\xafe\x1c@h...,b'\x01\x01\x00\x00\x00\xd4\xd2\xdc\nae\x1c@\xb...,687.0,Bonn Hbf,215356 ...,1102.0,Bonn Friedensplatz,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1533238,38084253,2023-07-31 14:56:00,2023-07-31 15:16:00,False,False,bike,0 days 00:20:00,1.684933,1684.933448,b'\x01\x01\x00\x00\x00\xccD\x11R\xb7c\x1c@>w\x...,b'\x01\x01\x00\x00\x00(eRC\x1b`\x1c@K\x1f\xba\...,687.0,Bonn Hbf,215356 ...,1176.0,Bonn Husarenstr.,215356 POLYGON ((7.120193524138334 50.7395...,1200.0
1506797,38063030,2023-07-31 15:01:00,2023-07-31 15:16:00,False,False,bike,0 days 00:15:00,3.304795,3304.795051,b'\x01\x01\x00\x00\x00\xdf\xf8\xda3Kb\x1c@\x9e...,b'\x01\x01\x00\x00\x00P4\x0f`\x91\x7f\x1c@\\\x...,687.0,Bonn Hbf,215356 ...,1502.0,Bonn Beueler Bahnhofsplatz,215356 POLYGON ((7.120193524138334 50.7395...,900.0


In [180]:
no_rain_nan_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.124900e+04,11249,11249,11249,11249.000000,11249.000000,11249.000000,11249.000000,11249.000000
mean,3.794051e+07,2023-07-14 16:15:48.254956032,2023-07-14 16:29:19.479064576,0 days 00:13:31.224108809,1.417337,1417.337245,1209.940084,1268.677216,811.224109
min,3.771653e+07,2023-07-01 05:16:00,2023-07-01 05:26:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782758e+07,2023-07-07 11:31:00,2023-07-07 11:46:00,0 days 00:10:00,0.717244,717.243706,687.000000,1102.000000,600.000000
50%,3.794341e+07,2023-07-13 15:26:00,2023-07-13 15:36:00,0 days 00:10:00,1.170184,1170.184456,1115.000000,1145.000000,600.000000
75%,3.805003e+07,2023-07-21 06:46:00,2023-07-21 06:56:00,0 days 00:15:00,1.804864,1804.864468,1221.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 15:06:00,2023-07-31 15:16:00,0 days 02:00:00,9.640671,9640.671235,9780.000000,9780.000000,7200.000000
std,1.286909e+05,NaN,NaN,0 days 00:08:37.048265259,1.077539,1077.538814,1098.729674,1169.490682,517.048265


In [181]:
# get statistics for the tier trips that start and end at a station
get_stats(no_rain_nan_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,11249,11249
sum,9125460.0,15943.626664
mean,811.224109,1.417337
median,600.0,1.170184
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,7200.0,9.640671
min,300.0,0.100004
std_dev,517.048265,1.077539
variance,267338.908608,1.16109
skewness,4.56572,2.042411


In [182]:
no_rain_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,815264.000000,815264.000000,815264.000000,815264.000000,815264.000000,815264.000000,815264.000000,791847,815264,815264.000000,...,815264.000000,815264.000000,815264.000000,815264.0,815264.0,815264.000000,815264.000000,815264.000000,815264.000000,815264.0
mean,566.265997,6.006043,2.757642,2848.239423,0.508991,3612.982760,2323.774718,2023-07-15 15:33:35.827151104,2023-07-15 15:57:43.068754688,135923.870690,...,0.759014,135321.885379,139120.977456,0.0,0.0,2.993617,0.689806,3.079972,0.638212,0.0
min,18.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,2023-07-01 01:24:50,2023-07-01 01:25:10,804.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,-1.000000,0.0
25%,602.000000,6.000000,3.000000,64.000000,0.000000,1915.000000,1214.000000,2023-07-08 10:18:00,2023-07-08 10:14:37.500000,90306.000000,...,0.250000,93200.000000,95715.000000,0.0,0.0,1.000000,0.000000,1.000000,0.000000,0.0
50%,607.000000,6.000000,3.000000,64.000000,1.000000,2717.000000,1441.000000,2023-07-14 20:00:30,2023-07-14 20:28:45,134305.000000,...,0.500000,140215.000000,141445.000000,0.0,0.0,3.000000,0.000000,3.000000,1.000000,0.0
75%,611.000000,6.000000,3.000000,130.000000,1.000000,5633.000000,1682.000000,2023-07-22 08:03:30,2023-07-22 08:54:15,182304.000000,...,0.750000,182730.000000,183330.000000,0.0,0.0,5.000000,1.000000,5.000000,1.000000,0.0
max,843.000000,12.000000,3.000000,27690.000000,1.000000,10396.000000,9780.000000,2023-07-31 15:11:45,2023-07-31 15:12:00,235807.000000,...,510.750000,235945.000000,235950.000000,0.0,0.0,8.000000,3.000000,8.000000,1.000000,0.0
std,152.181677,0.168878,0.817520,8159.353649,0.499919,2798.735581,2419.451711,NaN,NaN,55973.570104,...,3.967743,58281.312096,54200.804799,0.0,0.0,2.074824,1.002859,2.037980,0.536977,0.0


In [183]:
no_rain_no_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,513631.000000,513631.000000,513631.000000,513631.000000,513631.000000,513631.000000,513631.000000,369950,513631,513631.000000,...,513631.000000,513631.000000,513631.000000,513631.0,513631.0,513631.000000,513631.000000,513631.000000,513631.000000,513631.0
mean,560.443192,6.990812,2.606507,5572.223474,0.493159,4230.464113,2176.790291,2023-07-15 13:29:50.454365696,2023-07-15 14:12:05.181626112,134914.958235,...,0.043472,100412.457023,138869.276512,0.0,0.0,2.229714,0.700557,3.073860,-0.272912,0.0
min,16.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,2023-07-01 00:33:00,2023-07-01 00:32:00,308.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,-1.000000,0.0
25%,551.000000,6.000000,3.000000,64.000000,0.000000,2014.000000,1177.000000,2023-07-08 06:57:45,2023-07-08 08:28:00,85805.000000,...,0.000000,0.000000,95100.000000,0.0,0.0,0.000000,0.000000,1.000000,-1.000000,0.0
50%,607.000000,6.000000,3.000000,64.000000,0.000000,3858.000000,1414.000000,2023-07-14 19:59:00,2023-07-14 20:18:00,133305.000000,...,0.000000,105445.000000,141000.000000,0.0,0.0,2.000000,0.000000,3.000000,0.000000,0.0
75%,614.000000,6.000000,3.000000,151.000000,1.000000,6377.000000,1702.000000,2023-07-22 07:23:45,2023-07-22 07:59:00,182304.000000,...,0.000000,165820.000000,183700.000000,0.0,0.0,4.000000,1.000000,5.000000,0.000000,0.0
max,884.000000,12.000000,3.000000,29145.000000,1.000000,10396.000000,9780.000000,2023-07-31 15:01:00,2023-07-31 15:02:00,235807.000000,...,510.000000,235945.000000,235950.000000,0.0,0.0,8.000000,3.000000,8.000000,1.000000,0.0
std,217.088572,2.122587,1.012741,10908.123740,0.499954,2863.330959,2302.793290,NaN,NaN,56230.692252,...,0.904502,78044.487943,54663.434814,0.0,0.0,2.244839,1.003814,2.057229,0.460521,0.0


In [184]:
no_rain_nan_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,853143.000000,853143.000000,853143.000000,853143.000000,853143.000000,853143.000000,853143.000000,0,0,853143.000000,...,0.0,853143.0,853143.0,853143.0,853143.0,853143.0,853143.000000,853143.0,853143.0,853143.0
mean,553.990139,6.294802,2.668181,4097.355757,0.487680,4074.799473,2259.396644,NaT,NaT,125636.871490,...,NaN,0.0,0.0,0.0,0.0,0.0,0.646780,0.0,-1.0,1.0
min,16.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,NaT,NaT,1305.000000,...,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.0,1.0
25%,601.000000,6.000000,3.000000,64.000000,0.000000,1974.500000,1214.000000,NaT,NaT,83805.000000,...,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.0,1.0
50%,607.000000,6.000000,3.000000,64.000000,0.000000,2952.000000,1458.000000,NaT,NaT,123804.000000,...,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.0,1.0
75%,613.000000,6.000000,3.000000,130.000000,1.000000,6327.000000,1686.000000,NaT,NaT,164305.000000,...,NaN,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,-1.0,1.0
max,884.000000,12.000000,3.000000,29145.000000,1.000000,10396.000000,9780.000000,NaT,NaT,235807.000000,...,NaN,0.0,0.0,0.0,0.0,0.0,3.000000,0.0,-1.0,1.0
std,183.502153,1.204362,0.940933,9581.484094,0.499848,2914.588115,2346.981025,NaN,NaN,50503.808713,...,NaN,0.0,0.0,0.0,0.0,0.0,0.940689,0.0,0.0,0.0


In [185]:
data['current_description'].unique()

array(['few clouds', 'light intensity shower rain', 'broken clouds',
       'overcast clouds', 'clear sky', 'light rain', 'mist',
       'scattered clouds', 'light intensity drizzle',
       'thunderstorm with heavy rain', 'shower rain', 'moderate rain',
       'thunderstorm with rain', 'light intensity drizzle rain',
       'drizzle', 'heavy intensity shower rain'], dtype=object)

In [186]:
#rain_delay = delay[delay['current_description'].str.contains('rain') & ~delay['current_description'].str.contains('light rain') & ~delay['current_description'].str.contains('drizzle')]
rain_delay = delay[delay['current_description'].str.contains('rain') | ~delay['current_description'].str.contains('thunderstorm with heavy rain') | ~delay['current_description'].str.contains('shower rain') | ~delay['current_description'].str.contains('shower rain') | ~delay['current_description'].str.contains('shower rain') | ~delay['current_description'].str.contains('moderate rain') | ~delay['current_description'].str.contains('shower rain') | ~delay['current_description'].str.contains('thunderstorm with rain') | ~delay['current_description'].str.contains('drizzle') | ~delay['current_description'].str.contains('heavy intensity shower rain')]
rain_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
0,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
1,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
2,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
3,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
4,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564839,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,1,0
2564841,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,1,0
2564843,688,6,N8,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6880005-688-006-687.2.32:263500-36-1_5350170A-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,1,0
2564847,688,6,N8,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6880005-688-006-687.2.32:263500-36-1_5350170A-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,0,0


In [187]:
get_stats(rain_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,950266,950266,950266,950266,950266,950266
sum,12472,10187,33484,28157,20210022.86,77381.69
mean,0.013125,0.01072,0.035236,0.029631,21.267753,0.607621
median,0.0,0.0,0.0,0.0,20.77,0.36
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 21.48 Name: current_temp, dtype: float64","0 0.49 Name: current_precipitation_volume, ..."
max,6,6,8,8,35.37,3.89
min,0,0,0,0,11.13,0.11
std_dev,0.124343,0.111622,0.212242,0.191924,4.516619,0.693509
variance,0.015461,0.012459,0.045047,0.036835,20.399845,0.480954
skewness,11.346815,12.263409,7.628142,7.969663,0.709853,3.012741


In [188]:
rain_delay_tier = check_micromobility_datasets('tier', rain_delay)

In [189]:
rain_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,8.611000e+03,8611,8611,8611,8611.000000,8611.000000,6048.000000,4720.000000,8611.000000
mean,7.983235e+08,2023-07-15 10:18:24.366508288,2023-07-15 10:32:31.564278272,0 days 00:14:07.197770293,1.734128,1734.127757,1442.906085,1380.241102,847.197770
min,7.822645e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,7.823768e+08,2023-07-07 19:52:30,2023-07-07 20:05:00,0 days 00:10:00,0.747713,747.713301,1103.000000,1102.000000,600.000000
50%,8.086263e+08,2023-07-13 19:30:00,2023-07-13 20:00:00,0 days 00:10:00,1.337000,1337.000032,1136.000000,1144.000000,600.000000
75%,8.087460e+08,2023-07-22 13:35:00,2023-07-22 13:47:30,0 days 00:15:00,2.308697,2308.697478,1255.000000,1241.000000,900.000000
max,8.323141e+08,2023-08-01 01:45:00,2023-08-01 02:05:00,0 days 01:55:00,13.908704,13908.703513,9780.000000,9780.000000,6900.000000
std,1.674117e+07,NaN,NaN,0 days 00:09:36.571536418,1.430703,1430.702728,1461.181087,1400.427191,576.571536


In [190]:
get_stats(rain_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,8611,8611
sum,7295220.0,14932.574117
mean,847.19777,1.734128
median,600.0,1.337
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.100290 2 0...
max,6900.0,13.908704
min,300.0,0.100174
std_dev,576.571536,1.430703
variance,332434.736608,2.04691
skewness,3.850633,1.875194


In [191]:
# get entries where trips start and end at station
rain_delay_tier_station_station = rain_delay_tier[rain_delay_tier['end_stop_id'].notna() & rain_delay_tier['start_stop_id'].notna()]
rain_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3171653,782356027,2023-06-30 01:45:00,2023-06-30 02:05:00,e-scooter,0 days 00:20:00,3.598056,3598.055590,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1406.0,Bonn Kopenhagener Str.,801621 ...,1200.0,POINT (7.09506 50.73298),POINT (7.07331 50.75712)
3209527,782391260,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.670973,670.973151,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10180 50.73728),POINT (7.09852 50.73217)
3210269,782391943,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.838693,838.692953,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10410 50.73699),POINT (7.09815 50.73232)
3234236,782413798,2023-07-01 06:40:00,2023-07-01 06:45:00,e-scooter,0 days 00:05:00,0.844529,844.529088,1131.0,Bonn Weberstr.,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10205 50.72540),POINT (7.09679 50.73093)
3164060,782350073,2023-07-01 07:10:00,2023-07-01 07:15:00,e-scooter,0 days 00:05:00,0.941159,941.158971,1182.0,Bonn Chlodwigplatz,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,300.0,POINT (7.08933 50.74278),POINT (7.08230 50.73803)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422531,832205256,2023-07-31 23:25:00,2023-08-01 00:00:00,e-scooter,0 days 00:35:00,1.787106,1787.106290,1153.0,Bonn Heerstr.,801621 POLYGON ((7.157885983913106 50.7227...,1190.0,Bonn Zeisigweg,801621 ...,2100.0,POINT (7.09015 50.73736),POINT (7.07489 50.74243)
3485603,832265823,2023-07-31 23:25:00,2023-08-01 00:00:00,e-scooter,0 days 00:35:00,1.774445,1774.445116,1153.0,Bonn Heerstr.,801621 POLYGON ((7.157885983913106 50.7227...,1190.0,Bonn Zeisigweg,801621 ...,2100.0,POINT (7.09011 50.73738),POINT (7.07496 50.74244)
3398184,832181971,2023-08-01 00:10:00,2023-08-01 00:20:00,e-scooter,0 days 00:10:00,0.649892,649.892254,1255.0,Bonn Graf-Stauffenberg-Str.,801621 POLYGON ((7.157885983913106 50.7227...,6994.0,Bonn Heinrich-Lützeler-Str.,801621 ...,600.0,POINT (7.10419 50.71648),POINT (7.09839 50.71719)
3326567,832125594,2023-08-01 01:45:00,2023-08-01 02:05:00,e-scooter,0 days 00:20:00,0.308480,308.479763,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1111.0,Bonn Beethovenhalle Und Swb,801621 ...,1200.0,POINT (7.10401 50.73732),POINT (7.10429 50.74010)


In [192]:
rain_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.896000e+03,3896,3896,3896,3896.000000,3896.000000,3896.000000,3896.00000,3896.000000
mean,7.978198e+08,2023-07-15 06:35:34.111909632,2023-07-15 06:48:28.228952576,0 days 00:12:54.117043121,1.423031,1423.030582,1446.765144,1416.25770,774.117043
min,7.822648e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.00000,300.000000
25%,7.823733e+08,2023-07-07 19:58:45,2023-07-07 20:10:00,0 days 00:10:00,0.626023,626.022761,1103.000000,1103.00000,600.000000
50%,7.824646e+08,2023-07-13 14:27:30,2023-07-13 14:35:00,0 days 00:10:00,1.102653,1102.653303,1140.000000,1150.00000,600.000000
75%,8.087422e+08,2023-07-22 07:21:15,2023-07-22 07:30:00,0 days 00:15:00,1.934300,1934.300411,1240.000000,1243.00000,900.000000
max,8.323115e+08,2023-08-01 01:45:00,2023-08-01 02:05:00,0 days 01:45:00,9.093761,9093.761020,9780.000000,9780.00000,6300.000000
std,1.680557e+07,NaN,NaN,0 days 00:08:25.424125112,1.126577,1126.577326,1455.861211,1429.85439,505.424125


In [193]:
# get statistics for the tier trips that start and end at a station
get_stats(rain_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,3896,3896
sum,3015960.0,5544.127147
mean,774.117043,1.423031
median,600.0,1.102653
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.100290 2 0.10...
max,6300.0,9.093761
min,300.0,0.100174
std_dev,505.424125,1.126577
variance,255453.546246,1.269176
skewness,4.015907,1.61936


In [194]:
rain_delay_next = check_micromobility_datasets('next', rain_delay)

In [195]:
rain_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.221200e+04,22212,22212,22212,22212.000000,22212.000000,16240.000000,12875.000000,22212.000000
mean,3.793969e+07,2023-07-15 16:51:50.780658944,2023-07-15 17:06:41.680172800,0 days 00:14:50.899513776,1.732635,1732.635207,1307.213239,1293.153786,890.899514
min,3.771653e+07,2023-07-01 05:16:00,2023-07-01 05:46:00,0 days 00:05:00,0.100057,100.057033,43.000000,43.000000,300.000000
25%,3.782821e+07,2023-07-08 10:01:00,2023-07-08 10:16:00,0 days 00:10:00,0.768501,768.500796,1102.000000,697.000000,600.000000
50%,3.794191e+07,2023-07-14 15:41:00,2023-07-14 15:56:00,0 days 00:15:00,1.346888,1346.887697,1143.000000,1144.000000,900.000000
75%,3.804865e+07,2023-07-22 11:11:00,2023-07-22 11:31:00,0 days 00:15:00,2.254992,2254.991567,1224.000000,1221.000000,900.000000
max,3.817174e+07,2023-08-01 00:41:00,2023-08-01 00:46:00,0 days 02:00:00,14.143293,14143.293036,9780.000000,9780.000000,7200.000000
std,1.281510e+05,NaN,NaN,0 days 00:09:50.765916800,1.439398,1439.398121,1234.868278,1293.213850,590.765917


In [196]:
get_stats(rain_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,22212,22212
sum,19788660.0,38485.293226
mean,890.899514,1.732635
median,900.0,1.346888
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.870199 Name: trip_distance_kilometers, ..."
max,7200.0,14.143293
min,300.0,0.100057
std_dev,590.765917,1.439398
variance,349004.368453,2.071867
skewness,3.946175,2.189434


In [197]:
# get entries where trips start and end at station
rain_delay_next_station_station = rain_delay_next[rain_delay_next['end_stop_id'].notna() & rain_delay_next['start_stop_id'].notna()]
rain_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1417189,37961351,2023-07-01 05:41:00,2023-07-01 05:56:00,False,False,bike,0 days 00:15:00,1.671334,1671.334291,"b'\x01\x01\x00\x00\x00\xd9\n\x9a\x96XY\x1c@""\x...",b'\x01\x01\x00\x00\x00\x82\x8d\xeb\xdf\xf5I\x1...,1151.0,Bonn Frankenbad/Kunstverein,215356 ...,43.0,Bonn Propsthof Nord,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1220393,37717932,2023-07-01 05:56:00,2023-07-01 06:11:00,False,False,bike,0 days 00:15:00,1.377444,1377.444442,b'\x01\x01\x00\x00\x00\xcfj\x81=&R\x1c@\x7f\xa...,b'\x01\x01\x00\x00\x00\x9f\xe4\x0e\x9b\xc8L\x1...,8437.0,Bonn An Der Josefshöhe,215356 ...,1190.0,Bonn Zeisigweg,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1235649,37726317,2023-07-01 06:06:00,2023-07-01 06:21:00,False,False,bike,0 days 00:15:00,2.055591,2055.591133,"b""\x01\x01\x00\x00\x00\xf0\xf8\xf6\xaeA_\x1c@\...",b'\x01\x01\x00\x00\x00B\xd1<\x80En\x1c@\x84*5{...,1182.0,Bonn Chlodwigplatz,215356 ...,685.0,Bonn Juridicum,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1267255,37827523,2023-07-01 06:06:00,2023-07-01 06:16:00,False,False,bike,0 days 00:10:00,1.000358,1000.357907,b'\x01\x01\x00\x00\x00\xc0\x06D\x88+g\x1c@\xef...,b'\x01\x01\x00\x00\x00\x8f\x89\x94f\xf3h\x1c@f...,1160.0,Bonn Nordstr.,215356 ...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1638081,37769500,2023-07-01 06:11:00,2023-07-01 06:21:00,False,True,bike,0 days 00:10:00,0.914302,914.302426,b'\x01\x01\x00\x00\x00\xc5V\xd0\xb4\xc4Z\x1c@\...,b'\x01\x01\x00\x00\x00\x06.\x8f5#c\x1c@\x9dJ\x...,1153.0,Bonn Heerstr.,215356 ...,1102.0,Bonn Friedensplatz,215356 POLYGON ((7.120193524138334 50.7395...,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333820,37861619,2023-07-31 22:16:00,2023-07-31 22:26:00,False,False,bike,0 days 00:10:00,0.864220,864.220203,b'\x01\x01\x00\x00\x00h\x06\xf1\x81\x1d_\x1c@\...,b'\x01\x01\x00\x00\x00\x1fh\x05\x86\xac^\x1c@\...,1104.0,Bonn Stadthaus,215356 ...,1175.0,Bonn Lvr-Klinik,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1476595,38040111,2023-07-31 22:26:00,2023-07-31 22:41:00,False,True,bike,0 days 00:15:00,1.658928,1658.927837,b'\x01\x01\x00\x00\x00\xb1\xa3q\xa8\xdfe\x1c@r...,b'\x01\x01\x00\x00\x00\x1c`\xe6;\xf8Y\x1c@.\xe...,1143.0,Bonn Poppelsdorfer Allee,215356 ...,1240.0,Bonn Poppelsdorfer Platz,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1443891,38014658,2023-07-31 22:51:00,2023-07-31 23:06:00,True,False,bike,0 days 00:15:00,2.202721,2202.720792,b'\x01\x01\x00\x00\x00\x06.\x8f5#c\x1c@\x9dJ\x...,b'\x01\x01\x00\x00\x00\x0fc\xd2\xdfKQ\x1c@~\x1...,1102.0,Bonn Friedensplatz,215356 ...,1620.0,Bonn Max-Bruch-Str.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1441769,38012536,2023-07-31 23:21:00,2023-07-31 23:41:00,False,False,bike,0 days 00:20:00,3.058475,3058.475311,b'\x01\x01\x00\x00\x00}\xca1Y\xdc\x7f\x1c@ILP\...,b'\x01\x01\x00\x00\x00\x8b\xc0X\xdf\xc0d\x1c@&...,1502.0,Bonn Beueler Bahnhofsplatz,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,1200.0


In [198]:
rain_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.041400e+04,10414,10414,10414,10414.000000,10414.000000,10414.000000,10414.000000,10414.000000
mean,3.794147e+07,2023-07-15 12:58:35.951603456,2023-07-15 13:12:09.120414976,0 days 00:13:33.168811215,1.431897,1431.897347,1334.124064,1327.774342,813.168811
min,3.771653e+07,2023-07-01 05:41:00,2023-07-01 05:56:00,0 days 00:05:00,0.100057,100.057033,43.000000,43.000000,300.000000
25%,3.782975e+07,2023-07-08 09:36:00,2023-07-08 09:46:00,0 days 00:10:00,0.711259,711.259033,1102.000000,1102.000000,600.000000
50%,3.794299e+07,2023-07-14 11:38:30,2023-07-14 11:48:30,0 days 00:10:00,1.178690,1178.689723,1143.000000,1145.000000,600.000000
75%,3.804971e+07,2023-07-21 22:14:45,2023-07-21 22:24:45,0 days 00:15:00,1.873923,1873.922865,1223.000000,1221.000000,900.000000
max,3.817174e+07,2023-07-31 23:31:00,2023-07-31 23:56:00,0 days 02:00:00,9.640671,9640.671235,9780.000000,9780.000000,7200.000000
std,1.278787e+05,NaN,NaN,0 days 00:08:22.180019361,1.076630,1076.630211,1271.691868,1342.109738,502.180019


In [199]:
# get statistics for the tier trips that start and end at a station
get_stats(rain_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,10414,10414
sum,8468340.0,14911.778977
mean,813.168811,1.431897
median,600.0,1.17869
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,7200.0,9.640671
min,300.0,0.100057
std_dev,502.180019,1.07663
variance,252184.771846,1.159133
skewness,4.284547,1.832994


In [200]:
#rain_no_delay = no_delay[no_delay['current_description'].str.contains('rain') & ~no_delay['current_description'].str.contains('light rain') & ~no_delay['current_description'].str.contains('drizzle')]
rain_no_delay = no_delay[no_delay['current_description'].str.contains('rain') | ~no_delay['current_description'].str.contains('thunderstorm with heavy rain') | ~no_delay['current_description'].str.contains('shower rain') | ~no_delay['current_description'].str.contains('shower rain') | ~no_delay['current_description'].str.contains('shower rain') | ~no_delay['current_description'].str.contains('moderate rain') | ~no_delay['current_description'].str.contains('shower rain') | ~no_delay['current_description'].str.contains('thunderstorm with rain') | ~no_delay['current_description'].str.contains('drizzle') | ~no_delay['current_description'].str.contains('heavy intensity shower rain')]
rain_no_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
0,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
1,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
2,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
3,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
4,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,93-551-008-687.2.36:014000-33-157_49E69401-92E...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564809,688,6,N8,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6880005-688-006-687.2.32:263500-36-1_5350170A-...,Bonn Hbf,0,...,0,0,1,early morning,8,0,early morning,8,0,0
2564811,687,6,N7,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6870005-687-006-687.2.24:263500-34-1_F0CE2D00-...,Bad Godesberg Bf / Rheinallee,0,...,0,0,1,early morning,8,0,early morning,8,0,0
2564817,684,6,N4,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6840005-684-006-687.2.32:263500-36-1_7F56FE12-...,Am Waldrand,0,...,0,0,1,early morning,8,0,early morning,8,0,0
2564834,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,early morning,8,0,early morning,8,0,0


In [201]:
get_stats(rain_no_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,600178,600178,600178,600178,600178,600178
sum,18180,14652,41205,34431,12696668.05,49080.78
mean,0.030291,0.024413,0.068655,0.057368,21.154837,0.601112
median,0.0,0.0,0.0,0.0,20.65,0.36
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 19.72 Name: current_temp, dtype: float64","0 0.49 Name: current_precipitation_volume, ..."
max,6,6,10,9,35.37,3.89
min,0,0,0,0,11.13,0.11
std_dev,0.199216,0.175674,0.335813,0.299416,4.501233,0.678092
variance,0.039687,0.030861,0.11277,0.08965,20.261099,0.459808
skewness,8.424294,8.92557,7.290724,7.456592,0.719582,3.051105


In [202]:
rain_no_delay_tier = check_micromobility_datasets('tier', rain_no_delay)

In [203]:
rain_no_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,7.212000e+03,7212,7212,7212,7212.000000,7212.000000,5637.000000,3634.000000,7212.000000
mean,7.983333e+08,2023-07-15 10:53:45.349417472,2023-07-15 11:07:57.212978432,0 days 00:14:11.863560732,1.770099,1770.098546,1167.060493,1404.812053,851.863561
min,7.822646e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100067,100.066815,43.000000,43.000000,300.000000
25%,7.823747e+08,2023-07-07 22:03:45,2023-07-07 22:15:00,0 days 00:10:00,0.754034,754.034067,687.000000,1103.000000,600.000000
50%,8.086292e+08,2023-07-13 22:45:00,2023-07-13 22:55:00,0 days 00:10:00,1.360769,1360.769376,1110.000000,1150.000000,600.000000
75%,8.087460e+08,2023-07-22 15:25:00,2023-07-22 15:40:00,0 days 00:15:00,2.319081,2319.081467,1292.000000,1244.000000,900.000000
max,8.323115e+08,2023-08-01 00:10:00,2023-08-01 00:20:00,0 days 02:00:00,12.402037,12402.036677,9703.000000,9780.000000,7200.000000
std,1.662206e+07,NaN,NaN,0 days 00:09:52.892043645,1.479372,1479.372395,1058.502833,1409.779158,592.892044


In [204]:
get_stats(rain_no_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,7212,7212
sum,6143640.0,12765.950716
mean,851.863561,1.770099
median,600.0,1.360769
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100067 1 0.100174 2 0...
max,7200.0,12.402037
min,300.0,0.100067
std_dev,592.892044,1.479372
variance,351520.975419,2.188543
skewness,4.022817,1.877442


In [205]:
# get entries where trips start and end at station
rain_no_delay_tier_station_station = rain_no_delay_tier[rain_no_delay_tier['end_stop_id'].notna() & rain_no_delay_tier['start_stop_id'].notna()]
rain_no_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3171653,782356027,2023-06-30 01:45:00,2023-06-30 02:05:00,e-scooter,0 days 00:20:00,3.598056,3598.055590,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1406.0,Bonn Kopenhagener Str.,801621 ...,1200.0,POINT (7.09506 50.73298),POINT (7.07331 50.75712)
3209527,782391260,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.670973,670.973151,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10180 50.73728),POINT (7.09852 50.73217)
3210269,782391943,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.838693,838.692953,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10410 50.73699),POINT (7.09815 50.73232)
3254146,782431910,2023-07-01 01:50:00,2023-07-01 02:00:00,e-scooter,0 days 00:10:00,1.779253,1779.253071,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,600.0,POINT (7.09818 50.73206),POINT (7.08279 50.73648)
2973348,782286043,2023-07-01 04:35:00,2023-07-01 04:50:00,e-scooter,0 days 00:15:00,3.658388,3658.388003,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1433.0,Bonn Paulusplatz,801621 ...,900.0,POINT (7.09660 50.73269),POINT (7.06609 50.74510)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485596,832265817,2023-07-31 22:25:00,2023-07-31 22:45:00,e-scooter,0 days 00:20:00,1.171579,1171.578745,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,1200.0,POINT (7.10025 50.73326),POINT (7.09058 50.73749)
3518645,832297525,2023-07-31 22:55:00,2023-07-31 23:00:00,e-scooter,0 days 00:05:00,0.704362,704.361980,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1161.0,Bonn An Der Esche,801621 ...,300.0,POINT (7.10309 50.73744),POINT (7.10055 50.74329)
3318827,832120317,2023-07-31 23:10:00,2023-08-01 00:20:00,e-scooter,0 days 01:10:00,0.117849,117.848655,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,4200.0,POINT (7.10346 50.73716),POINT (7.10245 50.73747)
3421744,832204504,2023-07-31 23:15:00,2023-07-31 23:25:00,e-scooter,0 days 00:10:00,1.002491,1002.491082,1504.0,Bonn Beuel Bf,801621 POLYGON ((7.157885983913106 50.7227...,1126.0,Bonn Gerhardstr.,801621 ...,600.0,POINT (7.12768 50.73829),POINT (7.13061 50.74688)


In [206]:
rain_no_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.228000e+03,3228,3228,3228,3228.000000,3228.000000,3228.000000,3228.000000,3228.000000
mean,7.979461e+08,2023-07-15 06:27:56.877323264,2023-07-15 06:41:00.037175040,0 days 00:13:03.159851301,1.479934,1479.934143,1204.197646,1434.370818,783.159851
min,7.822648e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100067,100.066815,43.000000,43.000000,300.000000
25%,7.823643e+08,2023-07-07 20:28:45,2023-07-07 20:50:00,0 days 00:10:00,0.639902,639.901887,687.000000,1106.000000,600.000000
50%,8.086225e+08,2023-07-13 17:30:00,2023-07-13 17:47:30,0 days 00:10:00,1.147104,1147.103616,1115.000000,1151.000000,600.000000
75%,8.087446e+08,2023-07-22 13:12:30,2023-07-22 13:18:45,0 days 00:15:00,1.976277,1976.276914,1221.000000,1243.000000,900.000000
max,8.323097e+08,2023-08-01 00:10:00,2023-08-01 00:20:00,0 days 01:55:00,8.887499,8887.498628,9703.000000,9780.000000,6900.000000
std,1.660467e+07,NaN,NaN,0 days 00:08:50.577904527,1.200398,1200.397781,1067.565623,1437.539314,530.577905


In [207]:
# get statistics for the tier trips that start and end at a station
get_stats(rain_no_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,3228,3228
sum,2528040.0,4777.227413
mean,783.159851,1.479934
median,600.0,1.147104
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100067 1 0.100174 2 0.10...
max,6900.0,8.887499
min,300.0,0.100067
std_dev,530.577905,1.200398
variance,281512.912772,1.440955
skewness,4.60062,1.763246


In [208]:
rain_no_delay_next = check_micromobility_datasets('next', rain_no_delay)

In [209]:
rain_no_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.755100e+04,17551,17551,17551,17551.000000,17551.000000,14530.000000,9940.000000,17551.000000
mean,3.793942e+07,2023-07-15 19:18:54.437923584,2023-07-15 19:33:33.782690560,0 days 00:14:39.344766679,1.705482,1705.482351,1095.206607,1257.163883,879.344767
min,3.771653e+07,2023-07-01 01:31:00,2023-07-01 01:46:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782650e+07,2023-07-08 11:13:30,2023-07-08 11:26:00,0 days 00:10:00,0.795139,795.139412,687.000000,1102.000000,600.000000
50%,3.794133e+07,2023-07-14 18:06:00,2023-07-14 18:16:00,0 days 00:10:00,1.317033,1317.033475,1106.000000,1145.000000,600.000000
75%,3.804888e+07,2023-07-22 16:41:00,2023-07-22 16:56:00,0 days 00:15:00,2.167133,2167.132976,1221.000000,1221.000000,900.000000
max,3.817174e+07,2023-08-01 00:46:00,2023-08-01 01:06:00,0 days 02:00:00,13.602096,13602.096364,9780.000000,9780.000000,7200.000000
std,1.289399e+05,NaN,NaN,0 days 00:09:35.873564827,1.421232,1421.232303,900.303036,1151.995622,575.873565


In [210]:
get_stats(rain_no_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,17551,17551
sum,15433380.0,29932.920747
mean,879.344767,1.705482
median,600.0,1.317033
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.46698 Name: trip_distance_kilometers, d..."
max,7200.0,13.602096
min,300.0,0.100004
std_dev,575.873565,1.421232
variance,331630.362668,2.019901
skewness,3.961207,2.220085


In [211]:
# get entries where trips start and end at station
rain_no_delay_next_station_station = rain_no_delay_next[rain_no_delay_next['end_stop_id'].notna() & rain_no_delay_next['start_stop_id'].notna()]
rain_no_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1292973,37874063,2023-07-01 01:31:00,2023-07-01 01:46:00,False,False,bike,0 days 00:15:00,2.239755,2239.754526,"b""\x01\x01\x00\x00\x00{-\xe8\xbd1d\x1c@R'\xa0\...",b'\x01\x01\x00\x00\x00L\x8ce\xfa%b\x1c@;\xe0\x...,687.0,Bonn Hbf,215356 ...,1173.0,Bonn Pädagogische Fakultät,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1311815,37850623,2023-07-01 01:31:00,2023-07-01 01:46:00,False,False,bike,0 days 00:15:00,1.331223,1331.223283,b'\x01\x01\x00\x00\x00\x81\xcd9x&d\x1c@g\x7f\x...,b'\x01\x01\x00\x00\x00f\xf7\xe4a\xa1f\x1c@}\xc...,687.0,Bonn Hbf,215356 ...,1160.0,Bonn Nordstr.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1513945,38069151,2023-07-01 01:31:00,2023-07-01 01:46:00,False,False,bike,0 days 00:15:00,1.392093,1392.093481,b'\x01\x01\x00\x00\x00V\xb9P\xf9\xd7b\x1c@b\xd...,b'\x01\x01\x00\x00\x00)\x93\x1a\xda\x00\\\x1c@...,687.0,Bonn Hbf,215356 ...,1240.0,Bonn Poppelsdorfer Platz,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1444489,38015256,2023-07-01 05:31:00,2023-07-01 05:36:00,False,True,bike,0 days 00:05:00,0.109052,109.052108,b'\x01\x01\x00\x00\x00\x0c@\xa3t\xe9/\x1c@\x1c...,b'\x01\x01\x00\x00\x00W\xb2c#\x10/\x1c@ap\xcd\...,694.0,Bonn Tannenbusch Mitte,215356 ...,694.0,Bonn Tannenbusch Mitte,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1433891,38006735,2023-07-01 06:21:00,2023-07-01 06:36:00,False,False,bike,0 days 00:15:00,1.550245,1550.245342,b'\x01\x01\x00\x00\x00\xa9lXSYT\x1c@\xb0\xe6\x...,b'\x01\x01\x00\x00\x00\xdapX\x1a\xf8a\x1c@(\x0...,1221.0,Bonn Kaufmannstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539405,38089363,2023-07-31 23:31:00,2023-07-31 23:41:00,False,False,bike,0 days 00:10:00,1.037686,1037.686290,b'\x01\x01\x00\x00\x00\xf6z\xf7\xc7{e\x1c@\xfe...,b'\x01\x01\x00\x00\x00\x12\xdar.\xc5e\x1c@r\xf...,687.0,Bonn Hbf,215356 ...,1133.0,Bonn Luisenstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1466477,38032070,2023-07-31 23:36:00,2023-07-31 23:56:00,False,False,bike,0 days 00:20:00,2.853534,2853.534454,b'\x01\x01\x00\x00\x00\xb2\xb8\xff\xc8th\x1c@\...,b'\x01\x01\x00\x00\x00=Fy\xe6\xe5P\x1c@i5$\xee...,1103.0,Bonn Markt,215356 ...,1231.0,Bonn Sebastianstr.,215356 POLYGON ((7.120193524138334 50.7395...,1200.0
1360121,37933007,2023-07-31 23:51:00,2023-07-31 23:56:00,False,True,bike,0 days 00:05:00,0.126078,126.077777,b'\x01\x01\x00\x00\x00\xa7\xe7\xddXPh\x1c@`\x1...,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,686.0,Bonn Universität/Markt,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1360122,37933008,2023-07-31 23:56:00,2023-08-01 00:01:00,True,False,bike,0 days 00:05:00,0.115054,115.054399,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,b'\x01\x01\x00\x00\x00\xc4AB\x94/h\x1c@D\xbf\x...,1103.0,Bonn Markt,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,300.0


In [212]:
rain_no_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,8.764000e+03,8764,8764,8764,8764.000000,8764.000000,8764.000000,8764.000000,8764.000000
mean,3.794138e+07,2023-07-15 16:04:59.137380352,2023-07-15 16:18:28.699224064,0 days 00:13:29.561843906,1.416806,1416.805989,1136.687814,1282.335920,809.561844
min,3.771653e+07,2023-07-01 01:31:00,2023-07-01 01:46:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.783087e+07,2023-07-08 08:24:45,2023-07-08 08:57:15,0 days 00:10:00,0.732615,732.614620,687.000000,1102.000000,600.000000
50%,3.794213e+07,2023-07-14 13:26:00,2023-07-14 13:36:00,0 days 00:10:00,1.175229,1175.228954,1115.000000,1150.000000,600.000000
75%,3.804870e+07,2023-07-22 14:11:00,2023-07-22 14:22:15,0 days 00:15:00,1.779848,1779.847884,1184.000000,1221.000000,900.000000
max,3.817169e+07,2023-08-01 00:36:00,2023-08-01 00:41:00,0 days 02:00:00,9.640671,9640.671235,9780.000000,9780.000000,7200.000000
std,1.283062e+05,NaN,NaN,0 days 00:08:37.587905947,1.083039,1083.039212,956.971934,1178.358921,517.587906


In [213]:
# get statistics for the tier trips that start and end at a station
get_stats(rain_no_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,8764,8764
sum,7095000.0,12416.887685
mean,809.561844,1.416806
median,600.0,1.175229
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.222838 1 0.982570 Name: trip_distanc...
max,7200.0,9.640671
min,300.0,0.100004
std_dev,517.587906,1.083039
variance,267897.240383,1.172974
skewness,4.678091,2.076324


In [214]:
#rain_nan_delay = nan_delay[nan_delay['current_description'].str.contains('rain') & ~nan_delay['current_description'].str.contains('light rain') & ~nan_delay['current_description'].str.contains('drizzle')]
rain_nan_delay = nan_delay[nan_delay['current_description'].str.contains('rain') | ~nan_delay['current_description'].str.contains('thunderstorm with heavy rain') | ~nan_delay['current_description'].str.contains('shower rain') | ~nan_delay['current_description'].str.contains('shower rain') | ~nan_delay['current_description'].str.contains('shower rain') | ~nan_delay['current_description'].str.contains('moderate rain') | ~nan_delay['current_description'].str.contains('shower rain') | ~nan_delay['current_description'].str.contains('thunderstorm with rain') | ~nan_delay['current_description'].str.contains('drizzle') | ~nan_delay['current_description'].str.contains('heavy intensity shower rain')]
rain_nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
41,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,255-551-008-2071.2.22:033200-34-157_C0741D60-B...,Bonn Hbf,1,...,0,0,0,0,0,1,0,0,-1,1
42,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
43,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
44,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
45,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,0,0,0,0,0,1,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564854,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564856,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564857,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564858,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1


In [215]:
get_stats(rain_nan_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,1005613,1005613,1005613,1005613,1005613,1005613
sum,19067,15394,48198,40195,21100423.21,89175.55
mean,0.018961,0.015308,0.047929,0.039971,20.982648,0.621099
median,0.0,0.0,0.0,0.0,20.59,0.33
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 19.72 Name: current_temp, dtype: float64","0 0.21 Name: current_precipitation_volume, ..."
max,6,5,11,9,35.37,3.89
min,0,0,0,0,11.13,0.11
std_dev,0.151567,0.134628,0.269946,0.24137,4.138672,0.718824
variance,0.022973,0.018125,0.072871,0.058259,17.128602,0.516708
skewness,9.693251,10.440013,8.27752,8.551582,0.609189,2.812689


In [216]:
rain_nan_delay_tier = check_micromobility_datasets('tier', rain_nan_delay)

In [217]:
rain_nan_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,9.977000e+03,9977,9977,9977,9977.000000,9977.000000,7472.000000,5205.000000,9977.000000
mean,7.988750e+08,2023-07-15 11:20:33.046005760,2023-07-15 11:34:50.931141632,0 days 00:14:17.885135812,1.774328,1774.328011,1326.862687,1411.824784,857.885136
min,7.822645e+08,2023-07-01 01:25:00,2023-07-01 01:40:00,0 days 00:05:00,0.100042,100.042262,43.000000,43.000000,300.000000
25%,7.823754e+08,2023-07-07 15:15:00,2023-07-07 15:30:00,0 days 00:10:00,0.750246,750.246360,689.000000,1102.000000,600.000000
50%,8.086324e+08,2023-07-14 11:30:00,2023-07-14 11:45:00,0 days 00:10:00,1.354794,1354.794371,1115.000000,1145.000000,600.000000
75%,8.087480e+08,2023-07-22 18:00:00,2023-07-22 18:20:00,0 days 00:15:00,2.331417,2331.417317,1292.000000,1255.000000,900.000000
max,8.323115e+08,2023-07-31 23:15:00,2023-07-31 23:30:00,0 days 01:55:00,13.908704,13908.703513,9703.000000,9780.000000,6900.000000
std,1.683916e+07,NaN,NaN,0 days 00:09:59.928271605,1.514350,1514.349662,1366.966981,1459.135390,599.928272


In [218]:
get_stats(rain_nan_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,9977,9977
sum,8559120.0,17702.47057
mean,857.885136,1.774328
median,600.0,1.354794
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100042 1 0.100174 2 0...
max,6900.0,13.908704
min,300.0,0.100042
std_dev,599.928272,1.51435
variance,359913.931071,2.293255
skewness,3.874451,2.066563


In [219]:
# get entries where trips start and end at station
rain_nan_delay_tier_station_station = rain_nan_delay_tier[rain_nan_delay_tier['end_stop_id'].notna() & rain_nan_delay_tier['start_stop_id'].notna()]
rain_nan_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3190399,782373031,2023-07-01 01:25:00,2023-07-01 01:40:00,e-scooter,0 days 00:15:00,1.055901,1055.901028,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1110.0,Bonn Wilhelmsplatz,801621 ...,900.0,POINT (7.10146 50.73271),POINT (7.09772 50.74150)
3209527,782391260,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.670973,670.973151,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10180 50.73728),POINT (7.09852 50.73217)
3210269,782391943,2023-07-01 01:45:00,2023-07-01 01:50:00,e-scooter,0 days 00:05:00,0.838693,838.692953,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,300.0,POINT (7.10410 50.73699),POINT (7.09815 50.73232)
3201996,782383745,2023-07-01 02:20:00,2023-07-01 02:25:00,e-scooter,0 days 00:05:00,0.567479,567.479089,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1103.0,Bonn Markt,801621 ...,300.0,POINT (7.09831 50.73236),POINT (7.10279 50.73481)
3276897,782452649,2023-07-01 02:20:00,2023-07-01 02:25:00,e-scooter,0 days 00:05:00,0.589011,589.011497,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1103.0,Bonn Markt,801621 ...,300.0,POINT (7.09832 50.73240),POINT (7.10306 50.73479)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422524,832205249,2023-07-31 22:25:00,2023-07-31 22:45:00,e-scooter,0 days 00:20:00,1.197806,1197.806121,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,1200.0,POINT (7.10022 50.73309),POINT (7.09036 50.73745)
3485596,832265817,2023-07-31 22:25:00,2023-07-31 22:45:00,e-scooter,0 days 00:20:00,1.171579,1171.578745,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,1200.0,POINT (7.10025 50.73326),POINT (7.09058 50.73749)
3518645,832297525,2023-07-31 22:55:00,2023-07-31 23:00:00,e-scooter,0 days 00:05:00,0.704362,704.361980,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1161.0,Bonn An Der Esche,801621 ...,300.0,POINT (7.10309 50.73744),POINT (7.10055 50.74329)
3420883,832203680,2023-07-31 23:00:00,2023-07-31 23:05:00,e-scooter,0 days 00:05:00,0.369227,369.226899,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,300.0,POINT (7.10130 50.73766),POINT (7.10442 50.73650)


In [220]:
rain_nan_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,4.472000e+03,4472,4472,4472,4472.000000,4472.000000,4472.000000,4472.000000,4472.000000
mean,7.984864e+08,2023-07-15 07:51:00.845259520,2023-07-15 08:04:08.761180672,0 days 00:13:07.915921288,1.479684,1479.684184,1368.567308,1417.690072,787.915921
min,7.822645e+08,2023-07-01 01:25:00,2023-07-01 01:40:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,7.823711e+08,2023-07-07 15:53:45,2023-07-07 16:08:45,0 days 00:10:00,0.641841,641.841484,699.000000,1103.000000,600.000000
50%,8.086266e+08,2023-07-13 20:10:00,2023-07-13 20:20:00,0 days 00:10:00,1.177382,1177.381995,1115.000000,1150.000000,600.000000
75%,8.087473e+08,2023-07-22 16:45:00,2023-07-22 17:02:30,0 days 00:15:00,1.972956,1972.955729,1240.000000,1245.000000,900.000000
max,8.323115e+08,2023-07-31 23:15:00,2023-07-31 23:25:00,0 days 01:55:00,13.583168,13583.167599,9703.000000,9780.000000,6900.000000
std,1.683536e+07,NaN,NaN,0 days 00:09:00.617823045,1.210390,1210.389767,1407.526431,1416.018773,540.617823


In [221]:
# get statistics for the tier trips that start and end at a station
get_stats(rain_nan_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,4472,4472
sum,3523560.0,6617.14767
mean,787.915921,1.479684
median,600.0,1.177382
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.100246 2 0...
max,6900.0,13.583168
min,300.0,0.100174
std_dev,540.617823,1.21039
variance,292267.630594,1.465043
skewness,4.380747,2.058274


In [222]:
rain_nan_delay_next = check_micromobility_datasets('next', rain_nan_delay)

In [223]:
rain_nan_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.552100e+04,25521,25521,25521,25521.000000,25521.000000,20321.000000,14808.000000,25521.000000
mean,3.793938e+07,2023-07-15 14:26:58.034560,2023-07-15 14:41:38.309627392,0 days 00:14:40.275067591,1.705999,1705.999177,1182.937257,1251.842990,880.275068
min,3.771653e+07,2023-07-01 01:56:00,2023-07-01 02:06:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782718e+07,2023-07-07 14:56:00,2023-07-07 15:11:00,0 days 00:10:00,0.777028,777.028121,687.000000,699.000000,600.000000
50%,3.794134e+07,2023-07-14 14:51:00,2023-07-14 15:11:00,0 days 00:10:00,1.316987,1316.987300,1115.000000,1144.000000,600.000000
75%,3.804921e+07,2023-07-22 17:31:00,2023-07-22 17:41:00,0 days 00:15:00,2.189589,2189.589411,1221.000000,1221.000000,900.000000
max,3.817174e+07,2023-08-01 03:21:00,2023-08-01 03:31:00,0 days 02:00:00,14.104859,14104.859070,9780.000000,9780.000000,7200.000000
std,1.285653e+05,NaN,NaN,0 days 00:09:39.809478120,1.424317,1424.317182,1068.951173,1160.071674,579.809478


In [224]:
get_stats(rain_nan_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,25521,25521
sum,22465500.0,43538.804985
mean,880.275068,1.705999
median,600.0,1.316987
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.870199 Name: trip_distance_kilometers, ..."
max,7200.0,14.104859
min,300.0,0.100004
std_dev,579.809478,1.424317
variance,336179.030918,2.028679
skewness,4.002606,2.236964


In [225]:
# get entries where trips start and end at station
rain_nan_delay_next_station_station = rain_nan_delay_next[rain_nan_delay_next['end_stop_id'].notna() & rain_nan_delay_next['start_stop_id'].notna()]
rain_nan_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1635116,37766535,2023-07-01 05:16:00,2023-07-01 05:26:00,False,True,bike,0 days 00:10:00,1.003667,1003.666723,"b""\x01\x01\x00\x00\x00Z\xb7A\xed\xb7\x96\x1c@L...",b'\x01\x01\x00\x00\x00t\xef\xe1\x92\xe3\x9e\x1...,8950.0,Bonn Schießbergweg,215356 ...,1584.0,Bonn Ramersdorf,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1444489,38015256,2023-07-01 05:31:00,2023-07-01 05:36:00,False,True,bike,0 days 00:05:00,0.109052,109.052108,b'\x01\x01\x00\x00\x00\x0c@\xa3t\xe9/\x1c@\x1c...,b'\x01\x01\x00\x00\x00W\xb2c#\x10/\x1c@ap\xcd\...,694.0,Bonn Tannenbusch Mitte,215356 ...,694.0,Bonn Tannenbusch Mitte,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1311817,37850625,2023-07-01 06:21:00,2023-07-01 06:36:00,False,False,bike,0 days 00:15:00,1.284558,1284.557726,b'\x01\x01\x00\x00\x00}\xeb\xc3z\xa3f\x1c@}\xc...,b'\x01\x01\x00\x00\x001\x08\xac\x1cZd\x1c@C\x9...,1160.0,Bonn Nordstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1433891,38006735,2023-07-01 06:21:00,2023-07-01 06:36:00,False,False,bike,0 days 00:15:00,1.550245,1550.245342,b'\x01\x01\x00\x00\x00\xa9lXSYT\x1c@\xb0\xe6\x...,b'\x01\x01\x00\x00\x00\xdapX\x1a\xf8a\x1c@(\x0...,1221.0,Bonn Kaufmannstr.,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1471081,38035639,2023-07-01 06:26:00,2023-07-01 06:41:00,False,False,bike,0 days 00:15:00,1.898569,1898.568609,b'\x01\x01\x00\x00\x00\xbb\x99\xd1\x8f\x86c\x1...,b'\x01\x01\x00\x00\x00#.\x00\x8d\xd2e\x1c@\xd5...,687.0,Bonn Hbf,215356 ...,1174.0,Bonn Augustinum,215356 POLYGON ((7.120193524138334 50.7395...,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539405,38089363,2023-07-31 23:31:00,2023-07-31 23:41:00,False,False,bike,0 days 00:10:00,1.037686,1037.686290,b'\x01\x01\x00\x00\x00\xf6z\xf7\xc7{e\x1c@\xfe...,b'\x01\x01\x00\x00\x00\x12\xdar.\xc5e\x1c@r\xf...,687.0,Bonn Hbf,215356 ...,1133.0,Bonn Luisenstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1592910,37753445,2023-07-31 23:31:00,2023-07-31 23:56:00,False,False,bike,0 days 00:25:00,4.197185,4197.185304,b'\x01\x01\x00\x00\x00\xa5\xa2\xb1\xf6wv\x1c@\...,b'\x01\x01\x00\x00\x00\xbb%9`WS\x1c@#2\xac\xe2...,1500.0,Bonn Konrad-Adenauer-Platz,215356 ...,1231.0,Bonn Sebastianstr.,215356 POLYGON ((7.120193524138334 50.7395...,1500.0
1250204,37733796,2023-07-31 23:56:00,2023-08-01 00:11:00,False,False,bike,0 days 00:15:00,2.822479,2822.478797,b'\x01\x01\x00\x00\x00\x8a \xce\xc3\t\\\x1c@Id...,b'\x01\x01\x00\x00\x00\x12/O\xe7\x8aB\x1c@\x02...,1153.0,Bonn Heerstr.,215356 ...,9702.0,Bonn Im Tannenbusch,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1360122,37933008,2023-07-31 23:56:00,2023-08-01 00:01:00,True,False,bike,0 days 00:05:00,0.115054,115.054399,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,b'\x01\x01\x00\x00\x00\xc4AB\x94/h\x1c@D\xbf\x...,1103.0,Bonn Markt,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,300.0


In [226]:
rain_nan_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.263300e+04,12633,12633,12633,12633.000000,12633.000000,12633.000000,12633.000000,12633.000000
mean,3.794036e+07,2023-07-15 13:36:22.384231936,2023-07-15 13:49:55.639990528,0 days 00:13:33.255758727,1.416132,1416.132498,1212.093327,1276.789757,813.255759
min,3.771653e+07,2023-07-01 05:16:00,2023-07-01 05:26:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782776e+07,2023-07-07 15:51:00,2023-07-07 16:01:00,0 days 00:10:00,0.715740,715.739804,687.000000,1102.000000,600.000000
50%,3.794221e+07,2023-07-14 12:51:00,2023-07-14 13:01:00,0 days 00:10:00,1.166140,1166.139876,1115.000000,1146.000000,600.000000
75%,3.804974e+07,2023-07-22 14:51:00,2023-07-22 15:06:00,0 days 00:15:00,1.801790,1801.790390,1221.000000,1221.000000,900.000000
max,3.817174e+07,2023-08-01 03:21:00,2023-08-01 03:31:00,0 days 02:00:00,9.640671,9640.671235,9780.000000,9780.000000,7200.000000
std,1.284580e+05,NaN,NaN,0 days 00:08:47.874484806,1.084326,1084.325533,1104.632986,1186.784673,527.874485


In [227]:
# get statistics for the tier trips that start and end at a station
get_stats(rain_nan_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,12633,12633
sum,10273860.0,17890.001853
mean,813.255759,1.416132
median,600.0,1.16614
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.834842 Name: trip_distance_kilometers, ..."
max,7200.0,9.640671
min,300.0,0.100004
std_dev,527.874485,1.084326
variance,278651.47171,1.175762
skewness,4.650003,2.034943


In [228]:
rain_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,950266.000000,950266.000000,950266.000000,950266.000000,950266.000000,950266.000000,950266.000000,922490,950266,950266.000000,...,950266.000000,950266.000000,950266.000000,950266.0,950266.000000,950266.000000,950266.00000,950266.000000,950266.000000,950266.0
mean,565.554253,6.005974,2.754132,2862.957891,0.509721,3612.220960,2322.804288,2023-07-16 18:00:59.627910912,2023-07-16 18:34:38.471684864,135822.659095,...,0.748887,135778.715804,139664.951135,0.0,0.168433,3.005692,0.71758,3.093952,0.636938,0.0
min,18.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,2023-07-01 01:24:50,2023-07-01 01:25:10,804.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,-1.000000,0.0
25%,602.000000,6.000000,3.000000,64.000000,0.000000,1918.000000,1214.000000,2023-07-09 06:04:30,2023-07-09 06:05:30,90305.000000,...,0.250000,93200.000000,95530.000000,0.0,0.000000,1.000000,0.00000,1.000000,0.000000,0.0
50%,607.000000,6.000000,3.000000,64.000000,1.000000,2717.000000,1441.000000,2023-07-17 12:03:15,2023-07-17 12:36:15,134305.000000,...,0.500000,140600.000000,141930.000000,0.0,0.000000,3.000000,0.00000,3.000000,1.000000,0.0
75%,611.000000,6.000000,3.000000,130.000000,1.000000,5633.000000,1682.000000,2023-07-24 12:18:30,2023-07-24 13:02:45,182304.000000,...,0.750000,183215.000000,183830.000000,0.0,0.000000,5.000000,1.00000,5.000000,1.000000,0.0
max,843.000000,12.000000,3.000000,27690.000000,1.000000,10396.000000,9780.000000,2023-08-01 03:19:00,2023-08-01 03:19:15,235807.000000,...,510.750000,235945.000000,235950.000000,0.0,2.000000,8.000000,3.00000,8.000000,1.000000,0.0
std,153.090395,0.167664,0.822893,8178.754925,0.499906,2795.312776,2419.071095,NaN,NaN,55771.456861,...,3.739202,58455.953566,54280.290761,0.0,0.439083,2.087547,1.02544,2.050209,0.538245,0.0


In [229]:
rain_no_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,600178.000000,600178.000000,600178.000000,600178.000000,600178.00000,600178.000000,600178.000000,431687,600178,600178.000000,...,600178.000000,600178.000000,600178.000000,600178.0,600178.000000,600178.000000,600178.000000,600178.000000,600178.000000,600178.0
mean,559.848093,6.990076,2.603513,5561.686826,0.49288,4227.779072,2177.153543,2023-07-16 16:06:31.895155456,2023-07-16 17:04:35.483721216,134755.611832,...,0.043198,100555.048209,139269.690573,0.0,0.171084,2.234679,0.727443,3.083787,-0.274272,0.0
min,16.000000,1.000000,0.000000,3.000000,0.00000,6.000000,43.000000,2023-07-01 00:33:00,2023-07-01 00:32:00,308.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0
25%,551.000000,6.000000,3.000000,64.000000,0.00000,2014.000000,1177.000000,2023-07-08 20:57:40,2023-07-08 22:02:00,85803.000000,...,0.000000,0.000000,94800.000000,0.0,0.000000,0.000000,0.000000,1.000000,-1.000000,0.0
50%,607.000000,6.000000,3.000000,64.000000,0.00000,3858.000000,1414.000000,2023-07-17 11:13:45,2023-07-17 11:31:00,133305.000000,...,0.000000,105300.000000,141330.000000,0.0,0.000000,2.000000,0.000000,3.000000,0.000000,0.0
75%,614.000000,6.000000,3.000000,151.000000,1.00000,6377.000000,1702.000000,2023-07-24 11:11:35,2023-07-24 12:16:00,182304.000000,...,0.000000,170145.000000,184100.000000,0.0,0.000000,4.000000,2.000000,5.000000,0.000000,0.0
max,884.000000,12.000000,3.000000,29145.000000,1.00000,10396.000000,9780.000000,2023-08-01 03:15:45,2023-08-01 03:16:00,235807.000000,...,510.000000,235945.000000,235950.000000,0.0,2.000000,8.000000,3.000000,8.000000,1.000000,0.0
std,217.502117,2.117093,1.016003,10902.606345,0.49995,2862.483577,2303.986491,NaN,NaN,56077.608138,...,0.854009,78289.363412,54781.561853,0.0,0.442243,2.257568,1.025377,2.071034,0.460406,0.0


In [230]:
rain_nan_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,1.005613e+06,1.005613e+06,1.005613e+06,1.005613e+06,1.005613e+06,1.005613e+06,1.005613e+06,0,0,1.005613e+06,...,0.0,1005613.0,1005613.0,1005613.0,1.005613e+06,1005613.0,1.005613e+06,1005613.0,1005613.0,1005613.0
mean,5.538884e+02,6.293187e+00,2.668441e+00,4.052202e+03,4.886413e-01,4.066712e+03,2.254768e+03,NaT,NaT,1.261300e+05,...,NaN,0.0,0.0,0.0,1.794110e-01,0.0,6.679617e-01,0.0,-1.0,1.0
min,1.600000e+01,1.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,6.000000e+00,4.300000e+01,NaT,NaT,1.305000e+03,...,NaN,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,-1.0,1.0
25%,6.010000e+02,6.000000e+00,3.000000e+00,6.400000e+01,0.000000e+00,1.974000e+03,1.214000e+03,NaT,NaT,8.380500e+04,...,NaN,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,-1.0,1.0
50%,6.070000e+02,6.000000e+00,3.000000e+00,6.400000e+01,0.000000e+00,2.952000e+03,1.456000e+03,NaT,NaT,1.243040e+05,...,NaN,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,-1.0,1.0
75%,6.130000e+02,6.000000e+00,3.000000e+00,1.300000e+02,1.000000e+00,6.327000e+03,1.685000e+03,NaT,NaT,1.643050e+05,...,NaN,0.0,0.0,0.0,0.000000e+00,0.0,1.000000e+00,0.0,-1.0,1.0
max,8.840000e+02,1.200000e+01,3.000000e+00,2.914500e+04,1.000000e+00,1.039600e+04,9.780000e+03,NaT,NaT,2.358070e+05,...,NaN,0.0,0.0,0.0,2.000000e+00,0.0,3.000000e+00,0.0,-1.0,1.0
std,1.832842e+02,1.196105e+00,9.406096e-01,9.539435e+03,4.998712e-01,2.915055e+03,2.342219e+03,NaN,NaN,5.041465e+04,...,NaN,0.0,0.0,0.0,4.503408e-01,0.0,9.583452e-01,0.0,0.0,0.0


In [231]:
data['current_description'].unique()

array(['few clouds', 'light intensity shower rain', 'broken clouds',
       'overcast clouds', 'clear sky', 'light rain', 'mist',
       'scattered clouds', 'light intensity drizzle',
       'thunderstorm with heavy rain', 'shower rain', 'moderate rain',
       'thunderstorm with rain', 'light intensity drizzle rain',
       'drizzle', 'heavy intensity shower rain'], dtype=object)

In [232]:
light_rain_delay = delay[delay['current_description'].str.contains('light rain') | delay['current_description'].str.contains('light intensity drizzle') | delay['current_description'].str.contains('light intensity shower rain')]
light_rain_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
8,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
9,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
10,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
11,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
12,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564839,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,1,0
2564841,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,1,0
2564843,688,6,N8,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6880005-688-006-687.2.32:263500-36-1_5350170A-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,1,0
2564847,688,6,N8,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6880005-688-006-687.2.32:263500-36-1_5350170A-...,Bonn Hbf,0,...,0-5,0,1,early morning,8,0,early morning,8,0,0


In [233]:
get_stats(light_rain_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,108219,108219,108219,108219,108219,108219
sum,1117,879,2702,2329,2104686.81,48952.41
mean,0.010322,0.008122,0.024968,0.021521,19.448404,0.478528
median,0.0,0.0,0.0,0.0,18.85,0.32
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 17.11 Name: current_temp, dtype: float64","0 0.21 Name: current_precipitation_volume, ..."
max,5,3,4,4,31.96,3.46
min,0,0,0,0,12.56,0.11
std_dev,0.109917,0.096409,0.170625,0.15757,3.113862,0.521344
variance,0.012082,0.009295,0.029113,0.024828,9.696138,0.2718
skewness,12.6253,13.327813,7.810115,8.291794,1.443515,3.969575


In [234]:
light_rain_delay_tier = check_micromobility_datasets('tier', light_rain_delay)

In [235]:
light_rain_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,8.010000e+02,801,801,801,801.000000,801.000000,566.000000,406.000000,801.000000
mean,8.094964e+08,2023-07-21 10:54:44.644194560,2023-07-21 11:08:37.228464384,0 days 00:13:52.584269662,1.627610,1627.609781,1469.134276,1534.221675,832.584270
min,7.822652e+08,2023-07-01 08:10:00,2023-07-01 08:20:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,8.086312e+08,2023-07-14 16:55:00,2023-07-14 17:05:00,0 days 00:10:00,0.724832,724.832280,1103.000000,1102.000000,600.000000
50%,8.087375e+08,2023-07-25 07:25:00,2023-07-25 07:45:00,0 days 00:10:00,1.279666,1279.665624,1143.500000,1143.500000,600.000000
75%,8.321256e+08,2023-07-29 07:55:00,2023-07-29 08:10:00,0 days 00:15:00,2.180423,2180.422803,1302.000000,1292.750000,900.000000
max,8.323081e+08,2023-08-01 01:45:00,2023-08-01 02:05:00,0 days 01:45:00,8.853544,8853.543879,9703.000000,9780.000000,6300.000000
std,1.756056e+07,NaN,NaN,0 days 00:09:02.447636368,1.291084,1291.084222,1480.261539,1795.808949,542.447636


In [236]:
get_stats(light_rain_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,801,801
sum,666900.0,1303.715435
mean,832.58427,1.62761
median,600.0,1.279666
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.105359 2 0.10850...
max,6300.0,8.853544
min,300.0,0.100174
std_dev,542.447636,1.291084
variance,294249.438202,1.666898
skewness,3.67362,1.738287


In [237]:
# get entries where trips start and end at station
light_rain_delay_tier_station_station = light_rain_delay_tier[light_rain_delay_tier['end_stop_id'].notna() & light_rain_delay_tier['start_stop_id'].notna()]
light_rain_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
3181011,782364568,2023-07-01 08:10:00,2023-07-01 08:20:00,e-scooter,0 days 00:10:00,0.463702,463.702206,1102.0,Bonn Friedensplatz,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,600.0,POINT (7.09733 50.73576),POINT (7.10123 50.73724)
2965749,782283914,2023-07-01 08:15:00,2023-07-01 08:25:00,e-scooter,0 days 00:10:00,0.860218,860.218080,1110.0,Bonn Wilhelmsplatz,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,600.0,POINT (7.10001 50.73960),POINT (7.09792 50.73209)
2992763,782292503,2023-07-01 08:15:00,2023-07-01 08:35:00,e-scooter,0 days 00:20:00,3.576558,3576.557780,1292.0,Bonn Kiefernweg,801621 POLYGON ((7.157885983913106 50.7227...,686.0,Bonn Universität/Markt,801621 ...,1200.0,POINT (7.10081 50.70051),POINT (7.10097 50.73293)
2968244,782284567,2023-07-01 08:25:00,2023-07-01 08:35:00,e-scooter,0 days 00:10:00,0.806855,806.855012,1234.0,Bonn Am Klostergarten,801621 POLYGON ((7.157885983913106 50.7227...,1231.0,Bonn Sebastianstr.,801621 ...,600.0,POINT (7.07304 50.72288),POINT (7.07996 50.72506)
3249348,782427569,2023-07-01 08:25:00,2023-07-01 08:40:00,e-scooter,0 days 00:15:00,2.831303,2831.302754,1511.0,Bonn Am Ledenhof,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,900.0,POINT (7.12411 50.75135),POINT (7.10263 50.73757)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422531,832205256,2023-07-31 23:25:00,2023-08-01 00:00:00,e-scooter,0 days 00:35:00,1.787106,1787.106290,1153.0,Bonn Heerstr.,801621 POLYGON ((7.157885983913106 50.7227...,1190.0,Bonn Zeisigweg,801621 ...,2100.0,POINT (7.09015 50.73736),POINT (7.07489 50.74243)
3485603,832265823,2023-07-31 23:25:00,2023-08-01 00:00:00,e-scooter,0 days 00:35:00,1.774445,1774.445116,1153.0,Bonn Heerstr.,801621 POLYGON ((7.157885983913106 50.7227...,1190.0,Bonn Zeisigweg,801621 ...,2100.0,POINT (7.09011 50.73738),POINT (7.07496 50.74244)
3398184,832181971,2023-08-01 00:10:00,2023-08-01 00:20:00,e-scooter,0 days 00:10:00,0.649892,649.892254,1255.0,Bonn Graf-Stauffenberg-Str.,801621 POLYGON ((7.157885983913106 50.7227...,6994.0,Bonn Heinrich-Lützeler-Str.,801621 ...,600.0,POINT (7.10419 50.71648),POINT (7.09839 50.71719)
3326567,832125594,2023-08-01 01:45:00,2023-08-01 02:05:00,e-scooter,0 days 00:20:00,0.308480,308.479763,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1111.0,Bonn Beethovenhalle Und Swb,801621 ...,1200.0,POINT (7.10401 50.73732),POINT (7.10429 50.74010)


In [238]:
light_rain_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.440000e+02,344,344,344,344.000000,344.000000,344.000000,344.000000,344.000000
mean,8.109499e+08,2023-07-22 01:52:39.593023232,2023-07-22 02:05:44.476744192,0 days 00:13:04.883720930,1.365259,1365.258667,1448.171512,1608.430233,784.883721
min,7.822652e+08,2023-07-01 08:10:00,2023-07-01 08:20:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,8.086381e+08,2023-07-15 16:23:45,2023-07-15 16:33:45,0 days 00:10:00,0.662005,662.005488,1103.000000,1103.000000,600.000000
50%,8.087572e+08,2023-07-26 03:22:30,2023-07-26 03:32:30,0 days 00:10:00,1.042557,1042.556899,1144.500000,1150.000000,600.000000
75%,8.321488e+08,2023-07-29 16:31:15,2023-07-29 16:40:00,0 days 00:15:00,1.881181,1881.180616,1240.250000,1310.000000,900.000000
max,8.323070e+08,2023-08-01 01:45:00,2023-08-01 02:05:00,0 days 01:10:00,5.892370,5892.369649,9703.000000,9780.000000,4200.000000
std,1.765691e+07,NaN,NaN,0 days 00:07:30.465892124,1.010960,1010.959642,1421.686457,1886.592030,450.465892


In [239]:
# get statistics for the tier trips that start and end at a station
get_stats(light_rain_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,344,344
sum,270000.0,469.648982
mean,784.883721,1.365259
median,600.0,1.042557
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.105359 2 0.11190...
max,4200.0,5.89237
min,300.0,0.100174
std_dev,450.465892,1.01096
variance,202919.519967,1.022039
skewness,3.038344,1.330171


In [240]:
light_rain_delay_next = check_micromobility_datasets('next', light_rain_delay)

In [241]:
light_rain_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.904000e+03,1904,1904,1904,1904.000000,1904.000000,1338.000000,1095.000000,1904.000000
mean,3.794011e+07,2023-07-22 07:07:45.409664,2023-07-22 07:22:24.768907264,0 days 00:14:39.359243697,1.713596,1713.596088,1419.420777,1353.648402,879.359244
min,3.771669e+07,2023-07-01 08:06:00,2023-07-01 08:16:00,0 days 00:05:00,0.100260,100.260461,43.000000,43.000000,300.000000
25%,3.782881e+07,2023-07-15 23:16:00,2023-07-15 23:34:45,0 days 00:10:00,0.763960,763.959975,1103.000000,695.000000,600.000000
50%,3.794172e+07,2023-07-25 09:01:00,2023-07-25 09:13:30,0 days 00:15:00,1.345903,1345.902649,1151.000000,1144.000000,900.000000
75%,3.804559e+07,2023-07-28 05:27:15,2023-07-28 05:36:00,0 days 00:15:00,2.298826,2298.826178,1231.000000,1221.000000,900.000000
max,3.817130e+07,2023-08-01 00:41:00,2023-08-01 00:46:00,0 days 02:00:00,12.559234,12559.233706,9780.000000,9780.000000,7200.000000
std,1.282128e+05,NaN,NaN,0 days 00:09:11.536723978,1.393916,1393.916443,1444.260621,1449.253613,551.536724


In [242]:
get_stats(light_rain_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,1904,1904
sum,1674300.0,3262.686952
mean,879.359244,1.713596
median,900.0,1.345903
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 3.592849 Name: trip_distance_kilometers, ..."
max,7200.0,12.559234
min,300.0,0.10026
std_dev,551.536724,1.393916
variance,304192.757897,1.943003
skewness,3.564971,1.992206


In [243]:
# get entries where trips start and end at station
light_rain_delay_next_station_station = light_rain_delay_next[light_rain_delay_next['end_stop_id'].notna() & light_rain_delay_next['start_stop_id'].notna()]
light_rain_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1559867,38114219,2023-07-01 08:06:00,2023-07-01 08:21:00,True,False,bike,0 days 00:15:00,1.664427,1664.427043,"b'\x01\x01\x00\x00\x00\xbf,\xed\xd4\\N\x1c@\xa...",b'\x01\x01\x00\x00\x00*\x19\x00\xaa\xb8A\x1c@+...,8507.0,Bonn Auerberger Mitte,215356 ...,1432.0,Bonn Magdeburger Str.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1269783,37830051,2023-07-01 08:06:00,2023-07-01 08:16:00,False,False,bike,0 days 00:10:00,1.324039,1324.039495,b'\x01\x01\x00\x00\x00\x0c\xcdu\x1aiY\x1c@~t\x...,b'\x01\x01\x00\x00\x00!sePmP\x1c@\x90J\xb1\xa3...,1145.0,Bonn Haydnstr.,215356 ...,1192.0,Bonn Ellerstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1580255,37749677,2023-07-01 08:06:00,2023-07-01 08:16:00,False,False,bike,0 days 00:10:00,1.331120,1331.120321,b'\x01\x01\x00\x00\x00\x89\x97\xa7sEY\x1c@\xa8...,b'\x01\x01\x00\x00\x00\x0b(\xd4\xd3GP\x1c@\x8a...,1145.0,Bonn Haydnstr.,215356 ...,1192.0,Bonn Ellerstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1547628,38095504,2023-07-01 08:16:00,2023-07-01 08:26:00,False,False,bike,0 days 00:10:00,1.320237,1320.237232,b'\x01\x01\x00\x00\x00\x10\x95F\xcc\xecc\x1c@\...,b'\x01\x01\x00\x00\x00\xf8q4GVn\x1c@\x97\xc9p<...,1102.0,Bonn Friedensplatz,215356 ...,685.0,Bonn Juridicum,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1284019,37837211,2023-07-01 08:16:00,2023-07-01 08:41:00,False,False,bike,0 days 00:25:00,0.776580,776.579959,b'\x01\x01\x00\x00\x00r\xc2\x84\xd1\xacl\x1c@\...,b'\x01\x01\x00\x00\x00whX\x8c\xbaf\x1c@\x16i\x...,685.0,Bonn Juridicum,215356 ...,686.0,Bonn Universität/Markt,215356 POLYGON ((7.120193524138334 50.7395...,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333820,37861619,2023-07-31 22:16:00,2023-07-31 22:26:00,False,False,bike,0 days 00:10:00,0.864220,864.220203,b'\x01\x01\x00\x00\x00h\x06\xf1\x81\x1d_\x1c@\...,b'\x01\x01\x00\x00\x00\x1fh\x05\x86\xac^\x1c@\...,1104.0,Bonn Stadthaus,215356 ...,1175.0,Bonn Lvr-Klinik,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1476595,38040111,2023-07-31 22:26:00,2023-07-31 22:41:00,False,True,bike,0 days 00:15:00,1.658928,1658.927837,b'\x01\x01\x00\x00\x00\xb1\xa3q\xa8\xdfe\x1c@r...,b'\x01\x01\x00\x00\x00\x1c`\xe6;\xf8Y\x1c@.\xe...,1143.0,Bonn Poppelsdorfer Allee,215356 ...,1240.0,Bonn Poppelsdorfer Platz,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1443891,38014658,2023-07-31 22:51:00,2023-07-31 23:06:00,True,False,bike,0 days 00:15:00,2.202721,2202.720792,b'\x01\x01\x00\x00\x00\x06.\x8f5#c\x1c@\x9dJ\x...,b'\x01\x01\x00\x00\x00\x0fc\xd2\xdfKQ\x1c@~\x1...,1102.0,Bonn Friedensplatz,215356 ...,1620.0,Bonn Max-Bruch-Str.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1441769,38012536,2023-07-31 23:21:00,2023-07-31 23:41:00,False,False,bike,0 days 00:20:00,3.058475,3058.475311,b'\x01\x01\x00\x00\x00}\xca1Y\xdc\x7f\x1c@ILP\...,b'\x01\x01\x00\x00\x00\x8b\xc0X\xdf\xc0d\x1c@&...,1502.0,Bonn Beueler Bahnhofsplatz,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,1200.0


In [244]:
light_rain_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,8.480000e+02,848,848,848,848.000000,848.000000,848.000000,848.000000,848.000000
mean,3.794405e+07,2023-07-22 12:09:31.910377472,2023-07-22 12:23:07.712264192,0 days 00:13:35.801886792,1.437645,1437.644645,1418.971698,1384.847877,815.801887
min,3.771678e+07,2023-07-01 08:06:00,2023-07-01 08:16:00,0 days 00:05:00,0.100260,100.260461,683.000000,43.000000,300.000000
25%,3.783914e+07,2023-07-15 23:24:45,2023-07-15 23:42:15,0 days 00:10:00,0.715293,715.293390,1104.000000,1102.000000,600.000000
50%,3.794217e+07,2023-07-25 20:51:00,2023-07-25 21:01:00,0 days 00:10:00,1.169516,1169.516259,1145.000000,1150.000000,600.000000
75%,3.804554e+07,2023-07-28 05:23:30,2023-07-28 05:33:30,0 days 00:15:00,1.864103,1864.102825,1224.000000,1221.000000,900.000000
max,3.817128e+07,2023-07-31 23:31:00,2023-07-31 23:56:00,0 days 01:35:00,7.346171,7346.170563,9780.000000,9780.000000,5700.000000
std,1.260102e+05,NaN,NaN,0 days 00:08:47.349137675,1.111498,1111.497899,1411.652212,1467.626362,527.349138


In [245]:
# get statistics for the tier trips that start and end at a station
get_stats(light_rain_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,848,848
sum,691800.0,1219.122659
mean,815.801887,1.437645
median,600.0,1.169516
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 3.592849 Name: trip_distance_kilometers, ..."
max,5700.0,7.346171
min,300.0,0.10026
std_dev,527.349138,1.111498
variance,278097.113007,1.235428
skewness,4.137129,1.744676


In [246]:
light_rain_no_delay = no_delay[no_delay['current_description'].str.contains('light rain') | no_delay['current_description'].str.contains('light intensity drizzle') | no_delay['current_description'].str.contains('light intensity shower rain')]
light_rain_no_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
8,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
9,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
10,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
11,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
12,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,157-551-008-687.2.36:024000-33-157_95DF7B11-CF...,Troisdorf Bf,0,...,0,0,1,0,0,1,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564809,688,6,N8,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6880005-688-006-687.2.32:263500-36-1_5350170A-...,Bonn Hbf,0,...,0,0,1,early morning,8,0,early morning,8,0,0
2564811,687,6,N7,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6870005-687-006-687.2.24:263500-34-1_F0CE2D00-...,Bad Godesberg Bf / Rheinallee,0,...,0,0,1,early morning,8,0,early morning,8,0,0
2564817,684,6,N4,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,179,6840005-684-006-687.2.32:263500-36-1_7F56FE12-...,Am Waldrand,0,...,0,0,1,early morning,8,0,early morning,8,0,0
2564834,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,early morning,8,0,early morning,8,0,0


In [247]:
get_stats(light_rain_no_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,69130,69130,69130,69130,69130,69130
sum,1750,1355,3539,2998,1340370.19,30861.5
mean,0.025315,0.019601,0.051193,0.043368,19.389125,0.471053
median,0.0,0.0,0.0,0.0,18.77,0.32
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 17.11 Name: current_temp, dtype: float64","0 0.21 Name: current_precipitation_volume, ..."
max,5,5,6,6,31.96,3.46
min,0,0,0,0,12.56,0.11
std_dev,0.180799,0.15582,0.271291,0.248823,3.147162,0.504615
variance,0.032688,0.02428,0.073599,0.061913,9.904627,0.254636
skewness,8.912907,9.986671,6.994661,7.61133,1.449636,4.024358


In [248]:
light_rain_no_delay_tier = check_micromobility_datasets('tier', light_rain_no_delay)

In [249]:
light_rain_no_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,7.050000e+02,705,705,705,705.000000,705.000000,553.000000,347.000000,705.000000
mean,8.075939e+08,2023-07-20 13:25:52.851063808,2023-07-20 13:39:45.957446912,0 days 00:13:53.106382978,1.652094,1652.094188,1106.104882,1445.121037,833.106383
min,7.822653e+08,2023-07-01 04:35:00,2023-07-01 04:50:00,0 days 00:05:00,0.100174,100.173813,43.000000,161.000000,300.000000
25%,7.824568e+08,2023-07-11 22:00:00,2023-07-11 22:10:00,0 days 00:10:00,0.767267,767.266769,687.000000,1102.500000,600.000000
50%,8.087165e+08,2023-07-24 13:40:00,2023-07-24 13:55:00,0 days 00:10:00,1.262710,1262.709519,1103.000000,1146.000000,600.000000
75%,8.088075e+08,2023-07-27 21:15:00,2023-07-27 21:30:00,0 days 00:15:00,2.067390,2067.390203,1190.000000,1240.000000,900.000000
max,8.323115e+08,2023-08-01 00:10:00,2023-08-01 00:20:00,0 days 01:35:00,9.215283,9215.283180,9702.000000,9780.000000,5700.000000
std,1.749954e+07,NaN,NaN,0 days 00:09:39.006419022,1.386036,1386.035551,929.333845,1519.113151,579.006419


In [250]:
get_stats(light_rain_no_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,705,705
sum,587340.0,1164.726402
mean,833.106383,1.652094
median,600.0,1.26271
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.101896 2 0.10487...
max,5700.0,9.215283
min,300.0,0.100174
std_dev,579.006419,1.386036
variance,335248.433269,1.921095
skewness,3.54322,2.052277


In [251]:
# get entries where trips start and end at station
light_rain_no_delay_tier_station_station = light_rain_no_delay_tier[light_rain_no_delay_tier['end_stop_id'].notna() & light_rain_no_delay_tier['start_stop_id'].notna()]
light_rain_no_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
2973348,782286043,2023-07-01 04:35:00,2023-07-01 04:50:00,e-scooter,0 days 00:15:00,3.658388,3658.388003,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1433.0,Bonn Paulusplatz,801621 ...,900.0,POINT (7.09660 50.73269),POINT (7.06609 50.74510)
3181011,782364568,2023-07-01 08:10:00,2023-07-01 08:20:00,e-scooter,0 days 00:10:00,0.463702,463.702206,1102.0,Bonn Friedensplatz,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,600.0,POINT (7.09733 50.73576),POINT (7.10123 50.73724)
2965749,782283914,2023-07-01 08:15:00,2023-07-01 08:25:00,e-scooter,0 days 00:10:00,0.860218,860.218080,1110.0,Bonn Wilhelmsplatz,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,600.0,POINT (7.10001 50.73960),POINT (7.09792 50.73209)
3276902,782452654,2023-07-01 08:30:00,2023-07-01 08:40:00,e-scooter,0 days 00:10:00,2.290244,2290.244376,1103.0,Bonn Markt,801621 POLYGON ((7.157885983913106 50.7227...,1172.0,Bonn Bataverweg,801621 ...,600.0,POINT (7.10306 50.73465),POINT (7.09418 50.75338)
3181016,782364573,2023-07-01 08:40:00,2023-07-01 08:50:00,e-scooter,0 days 00:10:00,0.617976,617.975863,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,600.0,POINT (7.10114 50.73721),POINT (7.09828 50.73241)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485596,832265817,2023-07-31 22:25:00,2023-07-31 22:45:00,e-scooter,0 days 00:20:00,1.171579,1171.578745,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,1200.0,POINT (7.10025 50.73326),POINT (7.09058 50.73749)
3518645,832297525,2023-07-31 22:55:00,2023-07-31 23:00:00,e-scooter,0 days 00:05:00,0.704362,704.361980,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1161.0,Bonn An Der Esche,801621 ...,300.0,POINT (7.10309 50.73744),POINT (7.10055 50.74329)
3318827,832120317,2023-07-31 23:10:00,2023-08-01 00:20:00,e-scooter,0 days 01:10:00,0.117849,117.848655,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,4200.0,POINT (7.10346 50.73716),POINT (7.10245 50.73747)
3421744,832204504,2023-07-31 23:15:00,2023-07-31 23:25:00,e-scooter,0 days 00:10:00,1.002491,1002.491082,1504.0,Bonn Beuel Bf,801621 POLYGON ((7.157885983913106 50.7227...,1126.0,Bonn Gerhardstr.,801621 ...,600.0,POINT (7.12768 50.73829),POINT (7.13061 50.74688)


In [252]:
light_rain_no_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.160000e+02,316,316,316,316.000000,316.000000,316.000000,316.000000,316.000000
mean,8.075965e+08,2023-07-20 10:14:47.848101120,2023-07-20 10:28:07.974683648,0 days 00:13:20.126582278,1.413588,1413.587930,1163.594937,1489.145570,800.126582
min,7.822653e+08,2023-07-01 04:35:00,2023-07-01 04:50:00,0 days 00:05:00,0.100174,100.173813,371.000000,161.000000,300.000000
25%,8.086300e+08,2023-07-14 16:15:00,2023-07-14 16:25:00,0 days 00:10:00,0.657036,657.036049,687.000000,1103.000000,600.000000
50%,8.087232e+08,2023-07-24 09:35:00,2023-07-24 09:47:30,0 days 00:10:00,1.134797,1134.796738,1106.000000,1150.000000,600.000000
75%,8.088061e+08,2023-07-27 20:35:00,2023-07-27 21:00:00,0 days 00:15:00,1.852335,1852.334911,1164.500000,1240.000000,900.000000
max,8.323095e+08,2023-08-01 00:10:00,2023-08-01 00:20:00,0 days 01:35:00,6.184656,6184.655879,9073.000000,9780.000000,5700.000000
std,1.687797e+07,NaN,NaN,0 days 00:08:55.168256991,1.083083,1083.083206,1043.302315,1579.460114,535.168257


In [253]:
# get statistics for the tier trips that start and end at a station
get_stats(light_rain_no_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,316,316
sum,252840.0,446.693786
mean,800.126582,1.413588
median,600.0,1.134797
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.117849 2 0.12164...
max,5700.0,6.184656
min,300.0,0.100174
std_dev,535.168257,1.083083
variance,286405.063291,1.173069
skewness,4.923818,1.61574


In [254]:
light_rain_no_delay_next = check_micromobility_datasets('next', light_rain_no_delay)

In [255]:
light_rain_no_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.589000e+03,1589,1589,1589,1589.000000,1589.000000,1300.000000,888.000000,1589.000000
mean,3.793711e+07,2023-07-22 04:37:25.147891968,2023-07-22 04:51:53.052233984,0 days 00:14:27.904342353,1.664662,1664.662420,1134.618462,1320.013514,867.904342
min,3.771685e+07,2023-07-01 05:01:00,2023-07-01 05:16:00,0 days 00:05:00,0.100260,100.260461,43.000000,43.000000,300.000000
25%,3.782097e+07,2023-07-15 22:46:00,2023-07-15 23:01:00,0 days 00:10:00,0.808278,808.277617,687.000000,1102.000000,600.000000
50%,3.793412e+07,2023-07-25 08:56:00,2023-07-25 09:11:00,0 days 00:10:00,1.284725,1284.724637,1111.000000,1150.000000,600.000000
75%,3.804867e+07,2023-07-28 06:01:00,2023-07-28 06:11:00,0 days 00:15:00,2.129125,2129.124599,1221.000000,1221.000000,900.000000
max,3.817174e+07,2023-08-01 00:46:00,2023-08-01 01:06:00,0 days 02:00:00,10.667707,10667.707174,9780.000000,9780.000000,7200.000000
std,1.305105e+05,NaN,NaN,0 days 00:10:00.368863033,1.357146,1357.146127,1075.720339,1286.632352,600.368863


In [256]:
get_stats(light_rain_no_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,1589,1589
sum,1379100.0,2645.148585
mean,867.904342,1.664662
median,600.0,1.284725
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.222838 1 0.982570 2 2.724472 Name...
max,7200.0,10.667707
min,300.0,0.10026
std_dev,600.368863,1.357146
variance,360442.7717,1.841846
skewness,5.061051,2.070582


In [257]:
# get entries where trips start and end at station
light_rain_no_delay_next_station_station = light_rain_no_delay_next[light_rain_no_delay_next['end_stop_id'].notna() & light_rain_no_delay_next['start_stop_id'].notna()]
light_rain_no_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1242513,37729643,2023-07-01 08:06:00,2023-07-01 08:21:00,False,False,bike,0 days 00:15:00,1.423977,1423.976893,"b'\x01\x01\x00\x00\x00\xf2""\x13\xf0kd\x1c@sf\x...",b'\x01\x01\x00\x00\x00\xf5\xb8o\xb5Nl\x1c@sK\x...,1133.0,Bonn Luisenstr.,215356 ...,685.0,Bonn Juridicum,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1284019,37837211,2023-07-01 08:16:00,2023-07-01 08:41:00,False,False,bike,0 days 00:25:00,0.776580,776.579959,b'\x01\x01\x00\x00\x00r\xc2\x84\xd1\xacl\x1c@\...,b'\x01\x01\x00\x00\x00whX\x8c\xbaf\x1c@\x16i\x...,685.0,Bonn Juridicum,215356 ...,686.0,Bonn Universität/Markt,215356 POLYGON ((7.120193524138334 50.7395...,1500.0
1547628,38095504,2023-07-01 08:16:00,2023-07-01 08:26:00,False,False,bike,0 days 00:10:00,1.320237,1320.237232,b'\x01\x01\x00\x00\x00\x10\x95F\xcc\xecc\x1c@\...,b'\x01\x01\x00\x00\x00\xf8q4GVn\x1c@\x97\xc9p<...,1102.0,Bonn Friedensplatz,215356 ...,685.0,Bonn Juridicum,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1549193,38097069,2023-07-01 08:16:00,2023-07-01 08:26:00,False,False,bike,0 days 00:10:00,0.906862,906.861743,b'\x01\x01\x00\x00\x00\xfed\x8c\x0f\xb3g\x1c@\...,"b""\x01\x01\x00\x00\x00\x89'\xbb\x99\xd1_\x1c@=...",686.0,Bonn Universität/Markt,215356 ...,1144.0,Bonn Beethovenstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1603151,38140404,2023-07-01 08:21:00,2023-07-01 08:31:00,False,False,bike,0 days 00:10:00,0.597576,597.576449,b'\x01\x01\x00\x00\x00\xdb\xbeG\xfd\xf5j\x1c@\...,b'\x01\x01\x00\x00\x00B\xaf?\x89\xcfm\x1c@\xfe...,1103.0,Bonn Markt,215356 ...,685.0,Bonn Juridicum,215356 POLYGON ((7.120193524138334 50.7395...,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539405,38089363,2023-07-31 23:31:00,2023-07-31 23:41:00,False,False,bike,0 days 00:10:00,1.037686,1037.686290,b'\x01\x01\x00\x00\x00\xf6z\xf7\xc7{e\x1c@\xfe...,b'\x01\x01\x00\x00\x00\x12\xdar.\xc5e\x1c@r\xf...,687.0,Bonn Hbf,215356 ...,1133.0,Bonn Luisenstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1466477,38032070,2023-07-31 23:36:00,2023-07-31 23:56:00,False,False,bike,0 days 00:20:00,2.853534,2853.534454,b'\x01\x01\x00\x00\x00\xb2\xb8\xff\xc8th\x1c@\...,b'\x01\x01\x00\x00\x00=Fy\xe6\xe5P\x1c@i5$\xee...,1103.0,Bonn Markt,215356 ...,1231.0,Bonn Sebastianstr.,215356 POLYGON ((7.120193524138334 50.7395...,1200.0
1360121,37933007,2023-07-31 23:51:00,2023-07-31 23:56:00,False,True,bike,0 days 00:05:00,0.126078,126.077777,b'\x01\x01\x00\x00\x00\xa7\xe7\xddXPh\x1c@`\x1...,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,686.0,Bonn Universität/Markt,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,300.0
1360122,37933008,2023-07-31 23:56:00,2023-08-01 00:01:00,True,False,bike,0 days 00:05:00,0.115054,115.054399,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,b'\x01\x01\x00\x00\x00\xc4AB\x94/h\x1c@D\xbf\x...,1103.0,Bonn Markt,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,300.0


In [258]:
light_rain_no_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,7.630000e+02,763,763,763,763.000000,763.000000,763.000000,763.000000,763.000000
mean,3.793738e+07,2023-07-22 11:12:33.971166208,2023-07-22 11:26:08.650065664,0 days 00:13:34.678899082,1.359682,1359.682398,1139.052425,1354.357798,814.678899
min,3.771685e+07,2023-07-01 08:06:00,2023-07-01 08:21:00,0 days 00:05:00,0.100260,100.260461,43.000000,43.000000,300.000000
25%,3.782626e+07,2023-07-18 12:06:00,2023-07-18 12:16:00,0 days 00:10:00,0.707411,707.410581,687.000000,1103.000000,600.000000
50%,3.793153e+07,2023-07-26 10:11:00,2023-07-26 10:21:00,0 days 00:10:00,1.121532,1121.532005,1115.000000,1151.000000,600.000000
75%,3.803992e+07,2023-07-28 07:36:00,2023-07-28 07:46:00,0 days 00:15:00,1.633317,1633.317236,1182.000000,1221.000000,900.000000
max,3.817076e+07,2023-08-01 00:36:00,2023-08-01 00:41:00,0 days 02:00:00,8.255968,8255.968494,9780.000000,9780.000000,7200.000000
std,1.288739e+05,NaN,NaN,0 days 00:10:34.563933942,1.087984,1087.984174,983.068278,1321.819767,634.563934


In [259]:
# get statistics for the tier trips that start and end at a station
get_stats(light_rain_no_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,763,763
sum,621600.0,1037.43767
mean,814.678899,1.359682
median,600.0,1.121532
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.222838 1 0.982570 2 2.724472 Name...
max,7200.0,8.255968
min,300.0,0.10026
std_dev,634.563934,1.087984
variance,402671.38626,1.18371
skewness,5.488719,2.290497


In [260]:
light_rain_nan_delay = nan_delay[nan_delay['current_description'].str.contains('light rain') | nan_delay['current_description'].str.contains('light intensity drizzle') | nan_delay['current_description'].str.contains('light intensity shower rain')]
light_rain_nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
950,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,26817,660010-66-006-689.1.12:035500-13-178_67FF02EF-...,Ramersdorf,1,...,0,0,1,0,0,2,0,0,-1,1
957,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,26817,660010-66-006-689.1.12:035500-13-178_67FF02EF-...,Ramersdorf,1,...,0,0,1,0,0,2,0,0,-1,1
962,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,26817,660010-66-006-689.1.12:035500-13-178_67FF02EF-...,Ramersdorf,1,...,0,0,1,0,0,2,0,0,-1,1
968,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,26817,660010-66-006-689.1.12:035500-13-178_67FF02EF-...,Ramersdorf,1,...,0,0,1,0,0,2,0,0,-1,1
972,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,26817,660010-66-006-689.1.12:035500-13-178_67FF02EF-...,Ramersdorf,1,...,0,0,1,0,0,2,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564854,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564856,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564857,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1
2564858,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,0,0,1,0,0,0,0,0,-1,1


In [261]:
get_stats(light_rain_nan_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_2576417/931522703.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + p

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,122679,122679,122679,122679,122679,122679
sum,2085,1636,4478,3760,2401337.35,53269.95
mean,0.016996,0.013336,0.036502,0.030649,19.574152,0.460697
median,0.0,0.0,0.0,0.0,19.05,0.28
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: tier_trips_end_at_station_count, ...","0 0 Name: nextbike_trips_count, dtype: int64",0 0 Name: nextbike_trips_end_at_station_cou...,"0 24.9 Name: current_temp, dtype: float64","0 0.21 Name: current_precipitation_volume, ..."
max,5,5,6,6,31.96,3.46
min,0,0,0,0,12.56,0.11
std_dev,0.143538,0.124521,0.222632,0.202383,3.185093,0.525084
variance,0.020603,0.015505,0.049565,0.040959,10.144818,0.275713
skewness,10.417761,10.916283,8.090536,8.774086,1.350133,3.929118


In [262]:
light_rain_nan_delay_tier = check_micromobility_datasets('tier', light_rain_nan_delay)

In [263]:
light_rain_nan_delay_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.087000e+03,1087,1087,1087,1087.000000,1087.000000,811.000000,537.000000,1087.000000
mean,8.090021e+08,2023-07-21 02:55:23.459061504,2023-07-21 03:09:33.505059584,0 days 00:14:10.045998160,1.621259,1621.259126,1325.088779,1503.988827,850.045998
min,7.822653e+08,2023-07-01 04:35:00,2023-07-01 04:50:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,8.086271e+08,2023-07-14 16:35:00,2023-07-14 16:47:30,0 days 00:10:00,0.714003,714.002877,687.000000,1102.000000,600.000000
50%,8.087300e+08,2023-07-25 08:00:00,2023-07-25 08:10:00,0 days 00:10:00,1.280671,1280.671167,1115.000000,1140.000000,600.000000
75%,8.321196e+08,2023-07-29 07:22:30,2023-07-29 07:32:30,0 days 00:15:00,2.121985,2121.984740,1240.000000,1241.000000,900.000000
max,8.323115e+08,2023-07-31 23:15:00,2023-07-31 23:30:00,0 days 01:35:00,9.800629,9800.629086,9703.000000,9780.000000,5700.000000
std,1.743809e+07,NaN,NaN,0 days 00:10:01.989869242,1.342548,1342.548418,1443.103130,1728.670391,601.989869


In [264]:
get_stats(light_rain_nan_delay_tier, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,1087,1087
sum,924000.0,1762.30867
mean,850.045998,1.621259
median,600.0,1.280671
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.101896 2 0.10...
max,5700.0,9.800629
min,300.0,0.100174
std_dev,601.989869,1.342548
variance,362391.80267,1.802436
skewness,3.736043,2.033482


In [265]:
# get entries where trips start and end at station
light_rain_nan_delay_tier_station_station = light_rain_nan_delay_tier[light_rain_nan_delay_tier['end_stop_id'].notna() & light_rain_nan_delay_tier['start_stop_id'].notna()]
light_rain_nan_delay_tier_station_station

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds,start_location,end_location
2973348,782286043,2023-07-01 04:35:00,2023-07-01 04:50:00,e-scooter,0 days 00:15:00,3.658388,3658.388003,687.0,Bonn Hbf,801621 POLYGON ((7.157885983913106 50.7227...,1433.0,Bonn Paulusplatz,801621 ...,900.0,POINT (7.09660 50.73269),POINT (7.06609 50.74510)
3181011,782364568,2023-07-01 08:10:00,2023-07-01 08:20:00,e-scooter,0 days 00:10:00,0.463702,463.702206,1102.0,Bonn Friedensplatz,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,600.0,POINT (7.09733 50.73576),POINT (7.10123 50.73724)
2965749,782283914,2023-07-01 08:15:00,2023-07-01 08:25:00,e-scooter,0 days 00:10:00,0.860218,860.218080,1110.0,Bonn Wilhelmsplatz,801621 POLYGON ((7.157885983913106 50.7227...,687.0,Bonn Hbf,801621 ...,600.0,POINT (7.10001 50.73960),POINT (7.09792 50.73209)
2968244,782284567,2023-07-01 08:25:00,2023-07-01 08:35:00,e-scooter,0 days 00:10:00,0.806855,806.855012,1234.0,Bonn Am Klostergarten,801621 POLYGON ((7.157885983913106 50.7227...,1231.0,Bonn Sebastianstr.,801621 ...,600.0,POINT (7.07304 50.72288),POINT (7.07996 50.72506)
3120374,782332920,2023-07-01 08:25:00,2023-07-01 08:35:00,e-scooter,0 days 00:10:00,2.150683,2150.683217,1304.0,Bonn Buchholzstr.,801621 POLYGON ((7.157885983913106 50.7227...,1310.0,Bonn Mühlenhof,801621 ...,600.0,POINT (7.08030 50.69778),POINT (7.07249 50.71561)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422524,832205249,2023-07-31 22:25:00,2023-07-31 22:45:00,e-scooter,0 days 00:20:00,1.197806,1197.806121,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,1200.0,POINT (7.10022 50.73309),POINT (7.09036 50.73745)
3485596,832265817,2023-07-31 22:25:00,2023-07-31 22:45:00,e-scooter,0 days 00:20:00,1.171579,1171.578745,686.0,Bonn Universität/Markt,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,1200.0,POINT (7.10025 50.73326),POINT (7.09058 50.73749)
3518645,832297525,2023-07-31 22:55:00,2023-07-31 23:00:00,e-scooter,0 days 00:05:00,0.704362,704.361980,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1161.0,Bonn An Der Esche,801621 ...,300.0,POINT (7.10309 50.73744),POINT (7.10055 50.74329)
3420883,832203680,2023-07-31 23:00:00,2023-07-31 23:05:00,e-scooter,0 days 00:05:00,0.369227,369.226899,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 POLYGON ((7.157885983913106 50.7227...,1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,801621 ...,300.0,POINT (7.10130 50.73766),POINT (7.10442 50.73650)


In [266]:
light_rain_nan_delay_tier_station_station.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,4.680000e+02,468,468,468,468.000000,468.000000,468.000000,468.000000,468.000000
mean,8.093367e+08,2023-07-21 06:38:25.128205056,2023-07-21 06:51:38.717948416,0 days 00:13:13.589743589,1.426557,1426.557124,1361.972222,1518.057692,793.589744
min,7.822653e+08,2023-07-01 04:35:00,2023-07-01 04:50:00,0 days 00:05:00,0.100174,100.173813,43.000000,43.000000,300.000000
25%,8.086312e+08,2023-07-14 17:02:30,2023-07-14 17:15:00,0 days 00:10:00,0.620540,620.540313,688.000000,1102.000000,600.000000
50%,8.087299e+08,2023-07-25 05:42:30,2023-07-25 05:52:30,0 days 00:10:00,1.132374,1132.373634,1115.000000,1144.000000,600.000000
75%,8.321086e+08,2023-07-28 19:37:30,2023-07-28 19:47:30,0 days 00:15:00,1.880894,1880.894427,1224.000000,1240.250000,900.000000
max,8.323070e+08,2023-07-31 23:15:00,2023-07-31 23:25:00,0 days 01:35:00,8.437513,8437.512724,9703.000000,9780.000000,5700.000000
std,1.685607e+07,NaN,NaN,0 days 00:08:27.250997406,1.132004,1132.003625,1459.252857,1700.235144,507.250997


In [267]:
# get statistics for the tier trips that start and end at a station
get_stats(light_rain_nan_delay_tier_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,468,468
sum,371400.0,667.628734
mean,793.589744,1.426557
median,600.0,1.132374
mode,"0 600.0 Name: trip_duration_seconds, dtype:...",0 0.100174 1 0.111902 2 0.11489...
max,5700.0,8.437513
min,300.0,0.100174
std_dev,507.250997,1.132004
variance,257303.57437,1.281432
skewness,4.277736,1.747984


In [268]:
light_rain_nan_delay_next = check_micromobility_datasets('next', light_rain_nan_delay)

In [269]:
light_rain_nan_delay_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,2.356000e+03,2356,2356,2356,2356.000000,2356.000000,1844.000000,1373.000000,2356.000000
mean,3.794035e+07,2023-07-22 15:56:03.183361536,2023-07-22 16:10:44.083191808,0 days 00:14:40.899830220,1.668638,1668.637752,1232.317245,1330.224326,880.899830
min,3.771669e+07,2023-07-01 08:11:00,2023-07-01 08:21:00,0 days 00:05:00,0.100260,100.260461,43.000000,43.000000,300.000000
25%,3.783099e+07,2023-07-15 23:34:45,2023-07-15 23:44:45,0 days 00:10:00,0.756197,756.197168,687.000000,1102.000000,600.000000
50%,3.794141e+07,2023-07-26 11:01:00,2023-07-26 11:28:30,0 days 00:10:00,1.285610,1285.610232,1115.000000,1150.000000,600.000000
75%,3.804747e+07,2023-07-28 15:11:00,2023-07-28 15:21:00,0 days 00:15:00,2.129638,2129.638450,1221.000000,1221.000000,900.000000
max,3.817130e+07,2023-08-01 03:21:00,2023-08-01 03:31:00,0 days 02:00:00,13.373991,13373.990879,9780.000000,9780.000000,7200.000000
std,1.280415e+05,NaN,NaN,0 days 00:10:09.744122435,1.383306,1383.305997,1189.507992,1333.742707,609.744122


In [270]:
get_stats(light_rain_nan_delay_next, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,2356,2356
sum,2075400.0,3931.310544
mean,880.89983,1.668638
median,600.0,1.28561
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.98257 Name: trip_distance_kilometers, d..."
max,7200.0,13.373991
min,300.0,0.10026
std_dev,609.744122,1.383306
variance,371787.894845,1.913535
skewness,4.406981,2.042103


In [271]:
# get entries where trips start and end at station
light_rain_nan_delay_next_station_station = light_rain_nan_delay_next[light_rain_nan_delay_next['end_stop_id'].notna() & light_rain_nan_delay_next['start_stop_id'].notna()]
light_rain_nan_delay_next_station_station

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,trip_duration_seconds
1378336,37917898,2023-07-01 08:11:00,2023-07-01 08:26:00,False,False,bike,0 days 00:15:00,1.898308,1898.307744,b'\x01\x01\x00\x00\x00\x8fQ\x9ey9l\x1c@\xdd\xc...,b'\x01\x01\x00\x00\x00(\x81\xcd9xf\x1c@z\xaaCn...,685.0,Bonn Juridicum,215356 ...,1160.0,Bonn Nordstr.,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1287475,37872103,2023-07-01 08:11:00,2023-07-01 08:36:00,False,False,bike,0 days 00:25:00,1.342696,1342.696492,b'\x01\x01\x00\x00\x00\xce\x16\x10Z\x0f_\x1c@\...,"b""\x01\x01\x00\x00\x00c\x9a\xe9^'e\x1c@\xac\x8...",1182.0,Bonn Chlodwigplatz,215356 ...,687.0,Bonn Hbf,215356 POLYGON ((7.120193524138334 50.7395...,1500.0
1549193,38097069,2023-07-01 08:16:00,2023-07-01 08:26:00,False,False,bike,0 days 00:10:00,0.906862,906.861743,b'\x01\x01\x00\x00\x00\xfed\x8c\x0f\xb3g\x1c@\...,"b""\x01\x01\x00\x00\x00\x89'\xbb\x99\xd1_\x1c@=...",686.0,Bonn Universität/Markt,215356 ...,1144.0,Bonn Beethovenstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1324747,37856247,2023-07-01 08:46:00,2023-07-01 09:01:00,False,False,bike,0 days 00:15:00,1.201746,1201.745977,"b'\x01\x01\x00\x00\x00\xaeg\x08\xc7,[\x1c@\xe1...",b'\x01\x01\x00\x00\x00\xfb\xce/J\xd0_\x1c@\x9b...,1240.0,Bonn Poppelsdorfer Platz,215356 ...,1142.0,Bonn Colmantstr./Hbf,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1398079,37952663,2023-07-01 08:46:00,2023-07-01 08:56:00,False,False,bike,0 days 00:10:00,0.826507,826.506620,"b'\x01\x01\x00\x00\x00\xd7\xfa""\xa1-w\x1c@\x8a...",b'\x01\x01\x00\x00\x00\n\x14\xb1\x88aw\x1c@\xe...,1500.0,Bonn Konrad-Adenauer-Platz,215356 ...,1514.0,Bonn Schwarzrheindorf Schule,215356 POLYGON ((7.120193524138334 50.7395...,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539405,38089363,2023-07-31 23:31:00,2023-07-31 23:41:00,False,False,bike,0 days 00:10:00,1.037686,1037.686290,b'\x01\x01\x00\x00\x00\xf6z\xf7\xc7{e\x1c@\xfe...,b'\x01\x01\x00\x00\x00\x12\xdar.\xc5e\x1c@r\xf...,687.0,Bonn Hbf,215356 ...,1133.0,Bonn Luisenstr.,215356 POLYGON ((7.120193524138334 50.7395...,600.0
1592910,37753445,2023-07-31 23:31:00,2023-07-31 23:56:00,False,False,bike,0 days 00:25:00,4.197185,4197.185304,b'\x01\x01\x00\x00\x00\xa5\xa2\xb1\xf6wv\x1c@\...,b'\x01\x01\x00\x00\x00\xbb%9`WS\x1c@#2\xac\xe2...,1500.0,Bonn Konrad-Adenauer-Platz,215356 ...,1231.0,Bonn Sebastianstr.,215356 POLYGON ((7.120193524138334 50.7395...,1500.0
1250204,37733796,2023-07-31 23:56:00,2023-08-01 00:11:00,False,False,bike,0 days 00:15:00,2.822479,2822.478797,b'\x01\x01\x00\x00\x00\x8a \xce\xc3\t\\\x1c@Id...,b'\x01\x01\x00\x00\x00\x12/O\xe7\x8aB\x1c@\x02...,1153.0,Bonn Heerstr.,215356 ...,9702.0,Bonn Im Tannenbusch,215356 POLYGON ((7.120193524138334 50.7395...,900.0
1360122,37933008,2023-07-31 23:56:00,2023-08-01 00:01:00,True,False,bike,0 days 00:05:00,0.115054,115.054399,b'\x01\x01\x00\x00\x00\xa6}s\x7f\xf5h\x1c@\x92...,b'\x01\x01\x00\x00\x00\xc4AB\x94/h\x1c@D\xbf\x...,1103.0,Bonn Markt,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,300.0


In [272]:
light_rain_nan_delay_next_station_station.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.155000e+03,1155,1155,1155,1155.000000,1155.000000,1155.000000,1155.000000,1155.000000
mean,3.793724e+07,2023-07-22 20:41:10.129870080,2023-07-22 20:55:03.116883200,0 days 00:13:52.987012987,1.406350,1406.350436,1227.719481,1371.124675,832.987013
min,3.771675e+07,2023-07-01 08:11:00,2023-07-01 08:26:00,0 days 00:05:00,0.100260,100.260461,43.000000,43.000000,300.000000
25%,3.783099e+07,2023-07-18 12:26:00,2023-07-18 12:41:00,0 days 00:10:00,0.710658,710.658055,687.000000,1103.000000,600.000000
50%,3.793342e+07,2023-07-26 10:56:00,2023-07-26 11:11:00,0 days 00:10:00,1.137642,1137.641793,1115.000000,1151.000000,600.000000
75%,3.803792e+07,2023-07-28 15:11:00,2023-07-28 15:21:00,0 days 00:15:00,1.743991,1743.991188,1221.000000,1221.000000,900.000000
max,3.817128e+07,2023-08-01 03:21:00,2023-08-01 03:31:00,0 days 02:00:00,8.255968,8255.968494,9780.000000,9780.000000,7200.000000
std,1.253226e+05,NaN,NaN,0 days 00:10:35.621664339,1.144021,1144.021364,1132.163232,1376.744132,635.621664


In [273]:
# get statistics for the tier trips that start and end at a station
get_stats(light_rain_nan_delay_next_station_station, ['trip_duration_seconds', 'trip_distance_kilometers'])

column,trip_duration_seconds,trip_distance_kilometers
rows,1155,1155
sum,962100.0,1624.334754
mean,832.987013,1.40635
median,600.0,1.137642
mode,"0 600.0 Name: trip_duration_seconds, dtype:...","0 0.98257 Name: trip_distance_kilometers, d..."
max,7200.0,8.255968
min,300.0,0.10026
std_dev,635.621664,1.144021
variance,404014.900178,1.308785
skewness,5.02192,2.095317


In [274]:
light_rain_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,108219.000000,108219.000000,108219.000000,108219.000000,108219.000000,108219.000000,108219.000000,104815,108219,108219.000000,...,108219.000000,108219.000000,108219.000000,108219.0,108219.0,108219.000000,108219.000000,108219.000000,108219.000000,108219.0
mean,560.843678,6.005507,2.730103,3001.565991,0.511306,3605.058456,2307.699018,2023-07-23 20:43:22.156656896,2023-07-23 21:11:57.006949376,139275.059065,...,0.695521,142291.348192,146808.555383,0.0,1.0,3.176836,0.850276,3.278611,0.630813,0.0
min,18.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,2023-07-01 04:03:00,2023-07-01 02:40:00,10304.000000,...,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,-1.000000,0.0
25%,602.000000,6.000000,3.000000,64.000000,0.000000,1918.000000,1212.000000,2023-07-23 07:11:15,2023-07-23 07:20:07.500000,93305.000000,...,0.250000,94630.000000,100715.000000,0.0,1.0,1.000000,0.000000,1.000000,0.000000,0.0
50%,607.000000,6.000000,3.000000,64.000000,1.000000,2717.000000,1432.000000,2023-07-27 09:07:30,2023-07-27 09:09:15,143804.000000,...,0.500000,151830.000000,153315.000000,0.0,1.0,3.000000,0.000000,3.000000,1.000000,0.0
75%,611.000000,6.000000,3.000000,130.000000,1.000000,5633.000000,1682.000000,2023-07-29 15:10:17.500000,2023-07-29 15:12:45,183305.000000,...,0.750000,192500.000000,193115.000000,0.0,1.0,5.000000,2.000000,5.000000,1.000000,0.0
max,843.000000,12.000000,3.000000,27690.000000,1.000000,10396.000000,9780.000000,2023-08-01 03:19:00,2023-08-01 03:19:15,235804.000000,...,51.500000,235930.000000,235950.000000,0.0,1.0,8.000000,3.000000,8.000000,1.000000,0.0
std,159.143621,0.160701,0.858402,8362.012327,0.499874,2768.566471,2406.936576,NaN,NaN,52881.499561,...,1.914801,59811.088170,54847.786565,0.0,0.0,2.202455,1.145599,2.158195,0.543875,0.0


In [275]:
light_rain_no_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,69130.000000,69130.000000,69130.000000,69130.000000,69130.000000,69130.000000,69130.000000,49450,69130,69130.000000,...,69130.000000,69130.000000,69130.000000,69130.0,69130.0,69130.000000,69130.000000,69130.000000,69130.000000,69130.0
mean,554.226834,6.971214,2.576711,5631.488746,0.485375,4200.953088,2172.912209,2023-07-23 15:59:27.359676160,2023-07-23 16:16:16.875524352,138122.892652,...,0.040361,104429.649298,145647.389628,0.0,1.0,2.352032,0.854434,3.255576,-0.280775,0.0
min,16.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,2023-07-01 04:01:45,2023-07-01 02:40:00,10304.000000,...,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,-1.000000,0.0
25%,551.000000,6.000000,3.000000,64.000000,0.000000,2021.000000,1175.000000,2023-07-21 13:39:45,2023-07-21 13:40:00,91804.000000,...,0.000000,0.000000,95500.000000,0.0,1.0,0.000000,0.000000,1.000000,-1.000000,0.0
50%,606.000000,6.000000,3.000000,64.000000,0.000000,3189.000000,1406.000000,2023-07-26 13:52:45,2023-07-26 13:48:00,142808.000000,...,0.000000,113545.000000,152300.000000,0.0,1.0,1.000000,0.000000,3.000000,0.000000,0.0
75%,613.000000,6.000000,3.000000,151.000000,1.000000,6377.000000,1702.000000,2023-07-29 07:38:45,2023-07-29 07:28:00,183306.000000,...,0.000000,175245.000000,193300.000000,0.0,1.0,4.000000,2.000000,5.000000,0.000000,0.0
max,884.000000,12.000000,3.000000,28777.000000,1.000000,10396.000000,9780.000000,2023-08-01 03:15:45,2023-08-01 03:16:00,235804.000000,...,43.000000,235945.000000,235945.000000,0.0,1.0,8.000000,3.000000,8.000000,1.000000,0.0
std,221.639836,2.087279,1.044371,10963.870210,0.499790,2854.700479,2306.243589,NaN,NaN,53843.790279,...,0.443435,81324.077706,55867.147637,0.0,0.0,2.399885,1.142042,2.200986,0.457990,0.0


In [276]:
light_rain_nan_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,122679.000000,122679.000000,122679.000000,122679.000000,122679.000000,122679.000000,122679.000000,0,0,122679.000000,...,0.0,122679.0,122679.0,122679.0,122679.0,122679.0,122679.000000,122679.0,122679.0,122679.0
mean,553.014575,6.284213,2.669161,3804.731657,0.497445,4034.211511,2231.293530,NaT,NaT,134600.148925,...,NaN,0.0,0.0,0.0,1.0,0.0,0.727663,0.0,-1.0,1.0
min,16.000000,1.000000,0.000000,3.000000,0.000000,6.000000,43.000000,NaT,NaT,11805.000000,...,NaN,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,-1.0,1.0
25%,601.000000,6.000000,3.000000,64.000000,0.000000,1976.000000,1211.000000,NaT,NaT,95804.000000,...,NaN,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,-1.0,1.0
50%,607.000000,6.000000,3.000000,64.000000,0.000000,2952.000000,1455.000000,NaT,NaT,142305.000000,...,NaN,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,-1.0,1.0
75%,612.000000,6.000000,3.000000,130.000000,1.000000,6327.000000,1680.000000,NaT,NaT,171305.000000,...,NaN,0.0,0.0,0.0,1.0,0.0,2.000000,0.0,-1.0,1.0
max,884.000000,12.000000,3.000000,28777.000000,1.000000,10396.000000,9780.000000,NaT,NaT,235804.000000,...,NaN,0.0,0.0,0.0,1.0,0.0,3.000000,0.0,-1.0,1.0
std,182.241095,1.152619,0.939718,9304.412321,0.499996,2914.060312,2318.226205,NaN,NaN,49270.218696,...,NaN,0.0,0.0,0.0,0.0,0.0,1.032198,0.0,0.0,0.0


In [277]:
snow_delay = no_delay[no_delay['current_description'].str.contains('snow')]
snow_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip


In [278]:
get_stats(snow_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,0,0,0,0,0,0
sum,0,0,0,0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
median,NaN,NaN,NaN,NaN,NaN,NaN
mode,"Series([], Name: tier_trips_count, dtype: int64)","Series([], Name: tier_trips_end_at_station_cou...","Series([], Name: nextbike_trips_count, dtype: ...","Series([], Name: nextbike_trips_end_at_station...","Series([], Name: current_temp, dtype: float64)","Series([], Name: current_precipitation_volume,..."
max,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
std_dev,NaN,NaN,NaN,NaN,NaN,NaN
variance,NaN,NaN,NaN,NaN,NaN,NaN
skewness,NaN,NaN,NaN,NaN,NaN,NaN


In [279]:
snow_no_delay = no_delay[no_delay['current_description'].str.contains('snow')]
snow_no_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip


In [280]:
get_stats(snow_no_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,0,0,0,0,0,0
sum,0,0,0,0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
median,NaN,NaN,NaN,NaN,NaN,NaN
mode,"Series([], Name: tier_trips_count, dtype: int64)","Series([], Name: tier_trips_end_at_station_cou...","Series([], Name: nextbike_trips_count, dtype: ...","Series([], Name: nextbike_trips_end_at_station...","Series([], Name: current_temp, dtype: float64)","Series([], Name: current_precipitation_volume,..."
max,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
std_dev,NaN,NaN,NaN,NaN,NaN,NaN
variance,NaN,NaN,NaN,NaN,NaN,NaN
skewness,NaN,NaN,NaN,NaN,NaN,NaN


In [281]:
snow_nan_delay = nan_delay[nan_delay['current_description'].str.contains('snow')]
snow_nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip


In [282]:
get_stats(snow_nan_delay, ['tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'current_temp', 'current_precipitation_volume'])

column,tier_trips_count,tier_trips_end_at_station_count,nextbike_trips_count,nextbike_trips_end_at_station_count,current_temp,current_precipitation_volume
rows,0,0,0,0,0,0
sum,0,0,0,0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
median,NaN,NaN,NaN,NaN,NaN,NaN
mode,"Series([], Name: tier_trips_count, dtype: int64)","Series([], Name: tier_trips_end_at_station_cou...","Series([], Name: nextbike_trips_count, dtype: ...","Series([], Name: nextbike_trips_end_at_station...","Series([], Name: current_temp, dtype: float64)","Series([], Name: current_precipitation_volume,..."
max,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
std_dev,NaN,NaN,NaN,NaN,NaN,NaN
variance,NaN,NaN,NaN,NaN,NaN,NaN
skewness,NaN,NaN,NaN,NaN,NaN,NaN


In [283]:
snow_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [284]:
snow_no_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [285]:
snow_nan_delay.describe()

,route_id,agency_id,route_type,service_id,direction_id,shape_id,stop_id,actual_arrival_time,actual_departure_time,vrs_timestamp,...,departure_delay_float,actual_arrival_time_float,actual_departure_time_float,arrival_delay_span,weather_int,time_span_int_arrival,weekend,time_span_int,delay_category,cancelled_trip
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
